* https://www.pluralsight.com/guides/extracting-data-html-beautifulsoup

In [89]:
# importing the libraries
from bs4 import BeautifulSoup
import requests

url="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
# print(soup.prettify()) # print the parsed data of html

In [90]:
print(soup.title)

<title>List of countries by GDP (nominal) - Wikipedia</title>


In [91]:
print(soup.title.text)

List of countries by GDP (nominal) - Wikipedia


In [92]:
print(soup.title.string)

List of countries by GDP (nominal) - Wikipedia


In [93]:
for link in soup.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

Inner Text: 
Title: None
href: None
Inner Text: 
Title: This article is semi-protected.
href: /wiki/Wikipedia:Protection_policy#semi
Inner Text: Jump to navigation
Title: None
href: #mw-head
Inner Text: Jump to search
Title: None
href: #searchInput
Inner Text: List of countries by GDP (PPP)
Title: List of countries by GDP (PPP)
href: /wiki/List_of_countries_by_GDP_(PPP)
Inner Text: improve it
Title: None
href: https://en.wikipedia.org/w/index.php?title=List_of_countries_by_GDP_(nominal)&action=edit
Inner Text: talk page
Title: Talk:List of countries by GDP (nominal)
href: /wiki/Talk:List_of_countries_by_GDP_(nominal)
Inner Text: Learn how and when to remove these template messages
Title: Help:Maintenance template removal
href: /wiki/Help:Maintenance_template_removal
Inner Text: confusing or unclear
Title: Wikipedia:Vagueness
href: /wiki/Wikipedia:Vagueness
Inner Text: clarify the article
Title: Wikipedia:Please clarify
href: /wiki/Wikipedia:Please_clarify
Inner Text: the talk page
Titl

<br>

In [94]:
gdp_table = soup.find("table", attrs={"class": "wikitable"})
gdp_table_data = gdp_table.tbody.find_all("tr")  # contains 2 rows

# Get all the headings of Lists
headings = []
for td in gdp_table_data[0].find_all("td"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.b.text.replace('\n', ' ').strip())

print(headings)

['Per the International Monetary Fund (2020 estimates)', 'Per the World Bank (2019)', 'Per the United Nations (2018)']


In [95]:
data = {}
for table, heading in zip(gdp_table_data[1].find_all("table"), headings):
    # Get headers of table i.e., Rank, Country, GDP.
    t_headers = []
    for th in table.find_all("th"):
        # remove any newlines and extra spaces from left and right
        t_headers.append(th.text.replace('\n', ' ').strip())
    # Get all the rows of table
    table_data = []
    for tr in table.tbody.find_all("tr"): # find all tr's from table's tbody
        t_row = {}
        # Each table row is stored in the form of
        # t_row = {'Rank': '', 'Country/Territory': '', 'GDP(US$million)': ''}

        # find all td's(3) in tr and zip it with t_header
        for td, th in zip(tr.find_all("td"), t_headers): 
            t_row[th] = td.text.replace('\n', '').strip()
        table_data.append(t_row)

    # Put the data for the table with his heading.
    data[heading] = table_data

print(data)

{'Per the International Monetary Fund (2020 estimates)': [{}, {'Rank': '', 'Country/Territory': 'World[19]', 'GDP(US$million)': '83,844,988'}, {'Rank': '1', 'Country/Territory': 'United States', 'GDP(US$million)': '20,807,269'}, {'Rank': '2', 'Country/Territory': 'China[n 2][n 3]', 'GDP(US$million)': '14,860,775'}, {'Rank': '3', 'Country/Territory': 'Japan', 'GDP(US$million)': '4,910,580'}, {'Rank': '4', 'Country/Territory': 'Germany', 'GDP(US$million)': '3,780,553'}, {'Rank': '5', 'Country/Territory': 'United Kingdom', 'GDP(US$million)': '2,638,296'}, {'Rank': '6', 'Country/Territory': 'India', 'GDP(US$million)': '2,592,583'}, {'Rank': '7', 'Country/Territory': 'France', 'GDP(US$million)': '2,551,451'}, {'Rank': '8', 'Country/Territory': 'Italy', 'GDP(US$million)': '1,848,222'}, {'Rank': '9', 'Country/Territory': 'Canada', 'GDP(US$million)': '1,600,264'}, {'Rank': '10', 'Country/Territory': 'South Korea', 'GDP(US$million)': '1,586,786'}, {'Rank': '11', 'Country/Territory': 'Russia[n 4

In [96]:
import csv

for topic, table in data.items():
    # Create csv file for each table
    with open(f"{topic}.csv", 'w') as out_file:
        # Each 3 table has headers as following
        headers = [ 
            "Country/Territory",
            "GDP(US$million)",
            "Rank"
        ] # == t_headers
        writer = csv.DictWriter(out_file, headers)
        # write the header
        writer.writeheader()
        for row in table:
            if row:
                writer.writerow(row)

<br>

##### consolidated:

In [97]:


# importing the libraries
from bs4 import BeautifulSoup
import requests
import csv


# Step 1: Sending a HTTP request to a URL
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text


# Step 2: Parse the html content
soup = BeautifulSoup(html_content, "lxml")
# print(soup.prettify()) # print the parsed data of html


# Step 3: Analyze the HTML tag, where your content lives
# Create a data dictionary to store the data.
data = {}
#Get the table having the class wikitable
gdp_table = soup.find("table", attrs={"class": "wikitable"})
gdp_table_data = gdp_table.tbody.find_all("tr")  # contains 2 rows

# Get all the headings of Lists
headings = []
for td in gdp_table_data[0].find_all("td"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.b.text.replace('\n', ' ').strip())

# Get all the 3 tables contained in "gdp_table"
for table, heading in zip(gdp_table_data[1].find_all("table"), headings):
    # Get headers of table i.e., Rank, Country, GDP.
    t_headers = []
    for th in table.find_all("th"):
        # remove any newlines and extra spaces from left and right
        t_headers.append(th.text.replace('\n', ' ').strip())
    
    # Get all the rows of table
    table_data = []
    for tr in table.tbody.find_all("tr"): # find all tr's from table's tbody
        t_row = {}
        # Each table row is stored in the form of
        # t_row = {'Rank': '', 'Country/Territory': '', 'GDP(US$million)': ''}

        # find all td's(3) in tr and zip it with t_header
        for td, th in zip(tr.find_all("td"), t_headers): 
            t_row[th] = td.text.replace('\n', '').strip()
        table_data.append(t_row)

    # Put the data for the table with his heading.
    data[heading] = table_data


# Step 4: Export the data to csv
"""
For this example let's create 3 seperate csv for 
3 tables respectively
"""
for topic, table in data.items():
    # Create csv file for each table
    with open(f"{topic}.csv", 'w') as out_file:
        # Each 3 table has headers as following
        headers = [ 
            "Country/Territory",
            "GDP(US$million)",
            "Rank"
        ] # == t_headers
        writer = csv.DictWriter(out_file, headers)
        # write the header
        writer.writeheader()
        for row in table:
            if row:
                writer.writerow(row)
                
                

In [54]:

import re
import requests
#--------------------------------------------------
URL = 'https://www.49ers.com/team/players-roster/'
response = requests.get(URL)
#--------------------------------------------------
print(response.status_code)


200


In [55]:
print(response.content[:650])

b'\n\n\n<!DOCTYPE html>\n<html lang="en-US" dir="ltr">\n<head>\n  <meta charset="utf-8" />\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n  <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n  <link rel="canonical" href="https://www.49ers.com/team/players-roster/" />\n  <link rel="dns-prefetch" href="//nflenterprises.tt.omtrdc.net">\n  \n  \n\n\n<title>49ers Player Roster | San Francisco 49ers - 49ers.com</title>\n<meta name="description" content="San Francisco 49ers Player Roster: The official source of the latest 49ers player roster team information" />\n\n<meta property="og:title" content="San Francisco 49ers" />\n<'


In [56]:

from bs4 import BeautifulSoup
nfl = BeautifulSoup(response.content, 'html.parser')


In [87]:
# importing the libraries
from bs4 import BeautifulSoup
import requests

url="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
print(soup.prettify()) # print the parsed data of html


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of countries by GDP (nominal) - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"bc64007b-2278-4861-9e07-57d545132a15","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_countries_by_GDP_(nominal)","wgTitle":"List of countries by GDP (nominal)","wgCurRevisionId":991542097,"wgRevisionId":991542097,"wgArticleId":380845,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using the EasyTimeline extension","Wikipedia indefinitely semi-protected pages","Ar

In [57]:

# nfl.text


In [58]:

print(nfl.prettify())


<!DOCTYPE html>
<html dir="ltr" lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://www.49ers.com/team/players-roster/" rel="canonical"/>
  <link href="//nflenterprises.tt.omtrdc.net" rel="dns-prefetch"/>
  <title>
   49ers Player Roster | San Francisco 49ers - 49ers.com
  </title>
  <meta content="San Francisco 49ers Player Roster: The official source of the latest 49ers player roster team information" name="description"/>
  <meta content="San Francisco 49ers" property="og:title"/>
  <meta content="San Francisco 49ers Player Roster: The official source of the latest 49ers player roster and team information" property="og:description"/>
  <meta content="website" property="og:type"/>
  <meta content="https://www.49ers.com/team/players-roster/" property="og:url"/>
  <meta property="og:site_name"/>
  <meta content="en-US" property="og

In [59]:
#                   <tr>
#                     <td class="sorter-lastname">
#                       <div class="d3-o-media-object">
#                         <figure class="d3-o-media-object__figure">
#                           <a href="/team/players-roster/hroniss-grasu/" title="Hroniss Grasu">
#                             <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /><!--[if IE 9]></video><![endif]--><img alt="HronisGrassu" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /></picture>
#                           </a>
#                         </figure>
#                         <span class="nfl-o-roster__player-name" data-name="grasu,hroniss "><a href="/team/players-roster/hroniss-grasu/">Hroniss Grasu</a></span>
#                       </div>
#                     </td>
#                     <td data-append="1">50</td>
#                     <td data-append="1">OL</td>
#                     <td class="sorter-custom-height" data-name="grasu,hroniss ">6-3</td>
#                     <td data-append="1" data-value="301">301</td>
#                     <td data-append="1" data-value="29">29</td>
#                     <td data-append="1" data-value="6"><span>6</span></td>
#                     <td data-append="1" data-value="Oregon">Oregon</td>

In [60]:
print(nfl.a)

<a class="d3-u-block-bypass" href="#main-content" tabindex="0">
<span>Skip to main content</span>
</a>


In [61]:
print(nfl.p)

<p class="d3-o-footer__club-copyright">
      © 2020 Forty Niners Football Company LLC
    </p>


In [62]:
# print(nfl.body)

In [63]:
nfl_body = nfl.body

In [64]:
print(nfl_body.a)

<a class="d3-u-block-bypass" href="#main-content" tabindex="0">
<span>Skip to main content</span>
</a>


In [65]:
print(nfl_body.contents[0].next_sibling)

<a class="d3-u-block-bypass" href="#main-content" tabindex="0">
<span>Skip to main content</span>
</a>


In [66]:
nfl_div = nfl.find_all('div')
# print(nfl_div)

In [67]:
nfl_main = nfl.find(id='main-content')
# print(nfl_main.prettify())

#### quick checks:

In [68]:
nfl.title

<title>49ers Player Roster | San Francisco 49ers - 49ers.com</title>

In [69]:
nfl.title.string

'49ers Player Roster | San Francisco 49ers - 49ers.com'

In [70]:
nfl.title.parent.name

'head'

In [71]:
nfl.p

<p class="d3-o-footer__club-copyright">
      © 2020 Forty Niners Football Company LLC
    </p>

In [72]:
nfl.p['class']

['d3-o-footer__club-copyright']

In [73]:
for link in nfl.find_all('a'):
    print(link.get('href')) # http://example.com/elsi, # http://example.com/lacie

#main-content
https://www.49ers.com
https://www.49ers.com/news/
https://www.49ers.com/video/
https://www.49ers.com/audio/
https://www.49ers.com/photos/
https://www.49ers.com/schedule/
https://www.49ers.com/team/players-roster/
https://www.49ers.com/community/
https://www.49ers.com/fttb
#2ndlevel
https://www.49ers.com/cheerleaders/
http://www.levisstadium.com/
https://www.49ers.com/fans/
https://49ers.1rmg.com/
https://am.ticketmaster.com/49ersnew/
https://www.49ers.com/watch-live-games/ways-to-watch
https://www.49ers.com/tickets
https://www.49ers.com/tickets/suites
https://www.49ers.com/fans/team-stores
https://sports.yahoo.com/nfl/live-video/?is_retargeting=true&af_sub1=Acquisition&c=US_Acquisition_YMktg_NFLLive_NFLinapp_NFLteamappusers&pid=partnership&af_sub5=NFLteamapps__Static_&af_sub2=US_YMktg
/fttb
https://www.49ers.com/watch-live-games/ways-to-watch
https://www.49ers.com/tickets
https://www.49ers.com/tickets/suites
https://www.49ers.com/fans/team-stores
https://sports.yahoo.com/

In [74]:
# nfl.find_all('a')

In [75]:
nfl.div

<div id="oop" style="text-align: center;"></div>

In [76]:
# attribute value
# soup.select('a[href="http://example.com/elsie"]') # exact attribute
# soup.select('a[href^="http://example.com/"]') # negative match
# soup.select('a[href$="tillie"]') # end match
# soup.select('a[href*=".com/el"]') # middle match

nfl.select('a[href="/team/players-roster/jimmy-garoppolo/"]')


#     <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
#       <a 
#          data-link_module="Content Tray_Card carousel"
#          data-link_type="featured-players"
#          data-link_name="Jimmy Garoppolo"
#          data-link_url="/team/players-roster/jimmy-garoppolo/"
#          data-link_position="6:7"
#          href="/team/players-roster/jimmy-garoppolo/"
#          title="Jimmy Garoppolo"
#          data-id="31274802-8aae-4a0e-be0c-c11f14f39788"
         
#          aria-label="Jimmy Garoppolo">

[<a aria-label="Jimmy Garoppolo" data-id="31274802-8aae-4a0e-be0c-c11f14f39788" data-link_module="Content Tray_Card carousel" data-link_name="Jimmy Garoppolo" data-link_position="6:7" data-link_type="featured-players" data-link_url="/team/players-roster/jimmy-garoppolo/" href="/team/players-roster/jimmy-garoppolo/" title="Jimmy Garoppolo">
 <figure class="d3-o-media-object__figure">
 <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" media="(min-width:1024px)"/><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy

* http://akul.me/blog/2016/beautifulsoup-cheatsheet/

In [77]:
nfl.td

<td class="sorter-lastname">
<div class="d3-o-media-object">
<figure class="d3-o-media-object__figure">
<a href="/team/players-roster/chris-edwards/" title="Chris Edwards">
<picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg"/><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg"/><s

In [78]:
nfl.tr

<tr>
<th>Player</th>
<th class="{sorter: 'append'}">#</th>
<th class="{sorter: 'append'}">Pos</th>
<th>HT</th>
<th class="{sorter: 'append'}">WT</th>
<th class="{sorter: 'append'}">Age</th>
<th class="{sorter: 'append'}">Exp</th>
<th class="{sorter: 'append'}">College</th>
</tr>

In [86]:
nfl.findAll('td')

[<td class="sorter-lastname">
 <div class="d3-o-media-object">
 <figure class="d3-o-media-object__figure">
 <a href="/team/players-roster/chris-edwards/" title="Chris Edwards">
 <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg"/><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg

# Raw:

```




<!DOCTYPE html>
<html lang="en-US" dir="ltr">
<head>
  <meta charset="utf-8" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <link rel="canonical" href="https://www.49ers.com/team/players-roster/" />
  <link rel="dns-prefetch" href="//nflenterprises.tt.omtrdc.net">
  
  


<title>49ers Player Roster | San Francisco 49ers - 49ers.com</title>
<meta name="description" content="San Francisco 49ers Player Roster: The official source of the latest 49ers player roster team information" />

<meta property="og:title" content="San Francisco 49ers" />
<meta property="og:description" content="San Francisco 49ers Player Roster: The official source of the latest 49ers player roster and team information" />
<meta property="og:type" content="website" />
<meta property="og:url" content="https://www.49ers.com/team/players-roster/" />
<meta property="og:site_name" />
<meta property="og:locale" content="en-US" />
<meta property="og:image" content="" />
<meta property="og:image:type" content="image/jpeg" />

<meta name="twitter:card" content="summary" />
<meta name="twitter:title" content="San Francisco 49ers" />
<meta name="twitter:description" content="The official source for San Francisco 49ers news, schedules, stats, scores and more." />
<meta name="twitter:image:src" content="" />
<meta name="twitter:site" content="@https://www.49ers.com/" />  <meta content="112852666874" property="fb:pages" />
  
  <link href="/manifest.json" rel="manifest" />
  


<link rel="icon" href="https://static.clubs.nfl.com/49ers/hvdnirmddxf6agpzuabq" />
<link href="https://www.49ers.com/compiledassets/css/base.css?_t=ffc8adcf94d155c26d31b235d151293f" rel="stylesheet" type="text/css" />
  <link href="https://www.49ers.com/compiledassets/theming/cea711b3a5b6ee59af984e44c9da3dc2" rel="stylesheet" type="text/css" />
<script>
  // Picture element HTML5 shiv
  document.createElement("picture");
  window.baseUrl = "https://www.49ers.com";
  window.fbappid = "193113944051897";
  window.marketoHostName = "app-sj11.marketo.com";
  window.gigyaUrl = './js/gigya.js?apikey=';
  window.gigyaApiKey = '';
</script>
  
  
  <script async defer src="https://securepubads.g.doubleclick.net/tag/js/gpt.js"></script>
  <script type="text/javascript" src="https://imasdk.googleapis.com/js/sdkloader/gpt_proxy.js"></script>
  

<script type="text/javascript">
  window.gptconfig = {};
  var googletag = googletag || {};
  googletag.cmd = googletag.cmd || [];
  window.gptSlots = [];
  var advPageInfo = {"Sections":["team","players-roster"],"AdUnitName":"team/players-roster","IsCustom":false};
  var adUnitName = advPageInfo.AdUnitName;
  var leagueAdvId = '/4595/nfl.clubsites/sf';
  var clubAdvId = '/4595/team.sf/';
  var testParameter = '';
  window.adUnit = clubAdvId + adUnitName;
  googletag.cmd.push(function() {
    gptconfig.responsiveMappings = {};
    gptconfig.responsiveMappings['leaderboard'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [728, 90],
        [970, 250],
        [970, 90]
      ]).addSize([100, 1],
      [
        [300, 250],
        [320, 50]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['leaderboardleague'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [728, 90]
      ]).addSize([100, 1],
      [
        [320, 50]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['mrec'] = googletag.sizeMapping().addSize([100, 1],
      [
        [300, 250]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['mrec-photogallery'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [300, 600],
        [300, 250]
      ]).addSize([100, 1],
      [
        [300, 250]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['mrec-photogallery-list'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [728, 90],
        [970, 250],
        [970, 90]
      ]).addSize([100, 1],
      [
        [300, 250],
        [320, 50]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['package-c'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [300, 600]
      ]).addSize([100, 1],
      [
        [300, 250]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['package-b'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [160, 600]
      ]).addSize([100, 1],
      [
        [300, 250]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['presented-by'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [100, 30]
      ]).addSize([100, 1],
      [
        [100, 30],
        [30, 30]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['club-premium'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [728, 90],
        [970, 90],
        [970, 250]
      ]).addSize([760, 1],
      [
        [728, 90],
        [320, 50]
      ]).addSize([100, 1],
      [
        [320, 50],
        [300, 100]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['companion'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [300, 60]
      ]).addSize([100, 1],
      [
        [300, 60]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['interstitials'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [1, 4]
      ]).addSize([760, 1],
      [
        [3, 4]
      ]).addSize([100, 1],
      [
        [3, 4]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['728x90-320x50-only'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [320, 50],
        [728, 90]
      ]).addSize([100, 1],
      [
        [320, 50]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['300x250-only'] = googletag.sizeMapping().addSize([100, 1],
      [
        [300, 250]
      ]).addSize([0, 0], []).build();
    gptconfig.responsiveMappings['728x90-320x50-300x250'] = googletag.sizeMapping().addSize([1024, 1],
      [
        [300, 250],
        [320, 50],
        [728, 90],
      ]).addSize([100, 1],
      [
        [300, 250],
        [320, 50]
        ]).addSize([0, 0], []).build();

    gptconfig.responsiveMappings["draft-strip"] = googletag.sizeMapping()
      .addSize([0, 0], [
        [220, 32]
    ]).build();

    gptconfig.adSizes = [
      [728, 90],
      [970, 250],
      [970, 90],
      [300, 250],
      [320, 50],
      [300, 600],
      [160, 600],
      [300, 60],
      [300, 100],
      [1, 4],
      [3, 4]
    ];
  });
</script>
  
  
  
  <script>
    if ('serviceWorker' in navigator) {
      window.addEventListener('load', function () {
        navigator.serviceWorker.getRegistrations().then(function (registeredSevices) {
          for (var i = 0; i < registeredSevices.length; i++) {
            var sW = registeredSevices[i];
            var scriptUrl = sW && sW.active && sW.active.scriptURL;

            if (scriptUrl && scriptUrl.indexOf("OneSignal") !== -1) {
              continue;
            }

            sW.unregister().then(function (unregistered) {
              console.debug('ServiceWorker unregistration: ', unregistered);
            });            
          }
        });
      });
    }
  </script>

  <script type="text/javascript">
    window._taboola = window._taboola || [];
    var trackingPageInfo = {"SiteSection":"team","SiteSubsection":"players-roster","PageDetail":"landing","PageDetailSuffix":""};
    var utag_data = {
      page_type:'landing',
      siteName: '49ers',
      teamAbbr: 'sf',
      optimizedFor: 'desktop',
      responsiveState: window.innerWidth < 1024 ? 'mobile' : 'desktop',
      siteLanguage: 'English',
      partner: 'Deltatre',
      siteSection: trackingPageInfo.SiteSection,
      siteSubsection: trackingPageInfo.SiteSubsection,
      pageDetail: trackingPageInfo.PageDetail,
      pageState: 'no cp',
      adBlock: 'false',
      adPlacement:'na',
      buildNumber: '2.0.0.0'
    };
    var identityProvider = localStorage.getItem('identityProvider');
    if (identityProvider && identityProvider.length) {
      utag_data['identityProvider'] = JSON.parse(identityProvider);
    }
  </script>

      <script src="https://cdn.onesignal.com/sdks/OneSignalSDK.js" async=""></script>
      <script>
        var OneSignal = window.OneSignal || [];
        OneSignal.push(function () {
          OneSignal.init({
            appId: "07773e8c-80f0-4a59-96cc-e925ed7475c9",
          });
        });
      </script>

  <script type="text/javascript">
    window.nflLibHostName = "p.nfltags.com";
    window.environment = "clubs";
  </script>

  <script type="text/javascript">
    var adBlockEnabled = false;
    try {
      var testAd = document.createElement('div');
      testAd.innerHTML = '&nbsp;';
      testAd.className = 'adsbox';
      document.body.appendChild(testAd);
      if (testAd.offsetHeight === 0) {
        adBlockEnabled = true;
      }
      testAd.remove();
    }
    catch (e) { }
    if (utag_data) {
      utag_data.adBlock = '' + adBlockEnabled;
    }
  </script>
  <script type="text/javascript">
    utag_data.pageDetail = trackingPageInfo.PageDetailSuffix ? (utag_data.pageDetail + ':' + trackingPageInfo.PageDetailSuffix) : utag_data.pageDetail;
    var utag_data = window.utag_data || {};
    var utagParts = [utag_data.siteName, utag_data.siteSection, utag_data.siteSubsection, utag_data.pageDetail];
    var pageName = [];
    for (var i = 0; i < utagParts.length; i++) {
      var part = utagParts[i];
      if (part != undefined) {
        pageName.push(part);
      }
    }
    utag_data.pageName = pageName.join(':');
  </script>
<script type="text/javascript">(function(win, doc, style, timeout) { var STYLE_ID = 'at-body-style'; function getParent() { return doc.getElementsByTagName('head')[0]; } function addStyle(parent, id, def) { if (!parent) {return;} var style = doc.createElement('style'); style.id = id; style.innerHTML = def; parent.appendChild(style); } function removeStyle(parent, id) { if (!parent) {return;} var style = doc.getElementById(id); if (!style) {return;} parent.removeChild(style); } addStyle(getParent(), STYLE_ID, style); setTimeout(function() { removeStyle(getParent(), STYLE_ID); }, timeout); }(window, document, "body {}", 3000));</script>    <script src="//assets.adobedtm.com/a5ea4e8f4344/4e25eb6c0ac7/launch-0d4f4dd403d4.min.js" async></script>
  <script type="text/javascript">
    if (localStorage.getItem('user.countryCode') === null) {
      localStorage.setItem('user.countryCode', 'US');
    }
  </script>
</head>
<body class=" ">
  <a class="d3-u-block-bypass" href="#main-content" tabindex="0">
    <span>Skip to main content</span>
  </a>
    <!-- Interstitial OOP SLOT  -->
    <div id="oop" style="text-align: center;"></div>
    <!-- Background-Skin SLOT -->
    <div id="background_skin" style="text-align: center;"></div>
  





<div class="d3-l-wrap">











<div class="d3-o-header__branded-header ">

    <picture is-lazy="/t_lazy">
      <!--[if IE 9]><video style="display: none;"><![endif]-->
      <source data-srcset="https://static.clubs.nfl.com/f_auto/49ers/twfo5tp6fmpazcgwyztl"
              media="(min-width: 1024px)">
      <source data-srcset="https://static.clubs.nfl.com/f_auto/49ers/z8e8s39uaelykhtbzyq2"
              media="(min-width: 320px)">
      <!--[if IE 9]></video><![endif]-->
      
    </picture>
      <div class="d3-o-header__branded-header__sponsor">
    <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['56699454-a159-44e7-8c84-a0702cc18b36'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, '56699454-a159-44e7-8c84-a0702cc18b36')
                  .setTargeting('slot', 'brandedspon')
                  .defineSizeMapping(gptconfig.responsiveMappings['presented-by'])
.addService(googletag.pubads())
              );});
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['56699454-a159-44e7-8c84-a0702cc18b36'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('56699454-a159-44e7-8c84-a0702cc18b36')
            });
          });
        }
      </script><div class="  d3-o-adv-block" data-id="56699454-a159-44e7-8c84-a0702cc18b36" id="56699454-a159-44e7-8c84-a0702cc18b36"></div><script></script>
    <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['0e3769a0-4d70-49ed-a660-de22a4400334'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, '0e3769a0-4d70-49ed-a660-de22a4400334')
                  .setTargeting('slot', 'brandedspon1')
                  .defineSizeMapping(gptconfig.responsiveMappings['presented-by'])
.addService(googletag.pubads())
              );});
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['0e3769a0-4d70-49ed-a660-de22a4400334'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('0e3769a0-4d70-49ed-a660-de22a4400334')
            });
          });
        }
      </script><div class="  d3-o-adv-block" data-id="0e3769a0-4d70-49ed-a660-de22a4400334" id="0e3769a0-4d70-49ed-a660-de22a4400334"></div><script></script>
  </div>
</div><header class="nfl-c-header d3-o-nav__has-secondary-nav "
        role="banner"
        data-require="modules/navigation">
  <div class="d3-o-nav__wrap" data-require="modules/search">
    <a class="d3-o-nav__logo" href="https://www.49ers.com" title="Link to club's homepage"
       data-event_name="click action"
       data-link_module="Header"
       data-link_type="Nav Logo"
       data-link_name="Nav Logo"
       data-link_url="/">
      

<picture><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" /><source media="(min-width:768px)" srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" /><source srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" /><!--[if IE 9]></video><![endif]--><img alt="San Francisco 49ers logo" class="img-responsive" src="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" /></picture>
    </a>

    <button class="d3-o-nav__hamburger" aria-pressed="false" aria-label="Toggle navigation">
      <span class="d3-o-hamburger">
        <span class="d3-o-hamburger__inner">
          <span class="d3-o-screen-reader-content">
            Open menu button
          </span>
        </span>
      </span>
    </button>

      <nav role="navigation" class="d3-o-nav--primary">
        <h2 class="d3-o-nav--title">
          Primary nav
        </h2>
        





<script type="text/javascript">
          window._DMAMenuItems = window._DMAMenuItems || [];
          window._DMAMenuItems = window._DMAMenuItems.concat([]);
        </script>    <ul class="d3-o-nav__list d3-o-list">
          <li class="d3-o-nav__item  "  data-js-hook="c95353d4-8e91-4440-a134-41f0ab00d248">
            
      <a href="https://www.49ers.com/news/"
         title="NEWS"
         
         aria-label="NEWS "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="NEWS"
         data-link_url="/news/"
         data-link_position="1:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          NEWS
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="a5c716b3-2774-41d6-9580-93394e3f809d">
            
      <a href="https://www.49ers.com/video/"
         title="VIDEO"
         
         aria-label="VIDEO "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="VIDEO"
         data-link_url="/video/"
         data-link_position="2:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          VIDEO
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="35fb6e59-df7c-465a-8af6-747f2144e5f9">
            
      <a href="https://www.49ers.com/audio/"
         title="PODCASTS"
         
         aria-label="PODCASTS "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="PODCASTS"
         data-link_url="https://www.49ers.com/audio/"
         data-link_position="3:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          PODCASTS
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="404ef83f-7bed-4e23-b42e-eb84c066ffe5">
            
      <a href="https://www.49ers.com/photos/"
         title="PHOTOS"
         
         aria-label="PHOTOS "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="PHOTOS"
         data-link_url="/photos/"
         data-link_position="4:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          PHOTOS
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="9472769f-6b72-428b-a8a9-4e4335aea58f">
            
      <a href="https://www.49ers.com/schedule/"
         title="SCHEDULE"
         
         aria-label="SCHEDULE "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="SCHEDULE"
         data-link_url="/schedule/"
         data-link_position="5:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          SCHEDULE
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item   d3-is-active"  data-js-hook="a3ad4105-c13b-47cc-ba9d-f8206d28531f">
            
      <a href="https://www.49ers.com/team/players-roster/"
         title="TEAM"
         
         aria-label="TEAM "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="TEAM"
         data-link_url="/team/players-roster/"
         data-link_position="6:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          TEAM
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="2b38f454-4a51-4338-aa71-72b70285d714">
            
      <a href="https://www.49ers.com/community/"
         title="COMMUNITY"
         
         aria-label="COMMUNITY "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="COMMUNITY"
         data-link_url="/community/"
         data-link_position="7:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          COMMUNITY
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="d893d770-b46e-4a51-ae67-a69a62b24f0d">
            
      <a href="https://www.49ers.com/fttb"
         title="#FTTB"
         
         aria-label="#FTTB "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="#FTTB"
         data-link_url="https://www.49ers.com/fttb"
         data-link_position="8:14"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          #FTTB
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  d3-o-nav__item--has-submenu "  data-js-hook="e0d0717b-79fa-4bf2-9acd-ca2e972f7828">
            
      <a href="#2ndlevel" title="..." data-value="dropdown_trigger" aria-label="Toggle navigation">
        <span class="d3-o-nav__item-label">
          



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--dots" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#dots'></use>
  </svg>
</span>

        </span>
        <span class="d3-o-nav__item-icon">
          



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--open" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#open'></use>
  </svg>
</span>

        </span>
      </a>
  





  <ul id="2ndlevel" class="d3-o-nav__dropdown d3-o-list d3-o-nav__list">
        <li class="d3-o-nav__item "  data-js-hook="3b9b7d45-a8f2-4b7f-abae-8845dfada5db">
          
      <a href="https://www.49ers.com/cheerleaders/"
         title="GOLD RUSH"
         
         aria-label="GOLD RUSH "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="GOLD RUSH"
         data-link_url="/cheerleaders/"
         data-link_position="10:14"
         data-custom-properties=""
          >
        <span class="d3-o-nav__item-label">
          GOLD RUSH
        </span>
      </a>
  





        </li>
        <li class="d3-o-nav__item "  data-js-hook="6be2e047-eb80-4410-987e-d8e60c3258c0">
          
      <a href="http://www.levisstadium.com/"
         title="LEVI&#x27;S&#xAE; STADIUM"
         
         aria-label="LEVI&#x27;S&#xAE; STADIUM "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="LEVI&#x27;S&#xAE; STADIUM"
         data-link_url="http://www.levisstadium.com/"
         data-link_position="11:14"
         data-custom-properties=""
          >
        <span class="d3-o-nav__item-label">
          LEVI&#x27;S&#xAE; STADIUM
        </span>
      </a>
  





        </li>
        <li class="d3-o-nav__item "  data-js-hook="5e9ca64a-8fd9-4648-8360-bd987a062347">
          
      <a href="https://www.49ers.com/fans/"
         title="FANS"
         
         aria-label="FANS "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="FANS"
         data-link_url="/fans/"
         data-link_position="12:14"
         data-custom-properties=""
          >
        <span class="d3-o-nav__item-label">
          FANS
        </span>
      </a>
  





        </li>
        <li class="d3-o-nav__item "  data-js-hook="62432e7b-5b4f-43f8-8f33-69a6f2c887fb">
          
      <a href="https://49ers.1rmg.com/"
         title="MEDIA"
         
         aria-label="MEDIA "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="MEDIA"
         data-link_url="https://49ers.1rmg.com/"
         data-link_position="13:14"
         data-custom-properties=""
          >
        <span class="d3-o-nav__item-label">
          MEDIA
        </span>
      </a>
  





        </li>
        <li class="d3-o-nav__item "  data-js-hook="6b9abe53-508e-4227-bba7-c88eb4f8d7f9">
          
      <a href="https://am.ticketmaster.com/49ersnew/"
         title="ACCOUNT MANAGER"
         
         aria-label="ACCOUNT MANAGER "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_content-primary-nav-links"
         data-link_name="ACCOUNT MANAGER"
         data-link_url="https://am.ticketmaster.com/49ersnew/"
         data-link_position="14:14"
         data-custom-properties=""
          >
        <span class="d3-o-nav__item-label">
          ACCOUNT MANAGER
        </span>
      </a>
  





        </li>
  </ul>
          </li>
    </ul>





<span style="display: none">
    <svg class="nfl-o-icon-included--user" data-viewbox="0 0 32 32">
      <use xlink:href='#user'></use>
    </svg>
</span>

        <div class="d3-o-nav--action-related--mobile">
          





<script type="text/javascript">
          window._DMAMenuItems = window._DMAMenuItems || [];
          window._DMAMenuItems = window._DMAMenuItems.concat([]);
        </script>    <ul class="d3-o-nav__list d3-o-list">
          <li class="d3-o-nav__item  "  data-js-hook="2b3a9e61-29c1-48d7-9788-0111d4110616">
            
      <a href="https://www.49ers.com/watch-live-games/ways-to-watch"
         title="WATCH"
         
         aria-label="WATCH "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="WATCH"
         data-link_url="https://www.49ers.com/watch-live-games/ways-to-watch"
         data-link_position="1:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          WATCH
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="a1f99d72-9083-4666-aac1-cf056feb890e">
            
      <a href="https://www.49ers.com/tickets"
         title="TICKETS"
         
         aria-label="TICKETS "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="TICKETS"
         data-link_url="/tickets"
         data-link_position="2:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          TICKETS
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="b92f6183-d2cd-4d06-b439-cc1dd8f12f46">
            
      <a href="https://www.49ers.com/tickets/suites"
         title="SUITES"
         
         aria-label="SUITES "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="SUITES"
         data-link_url="/tickets/suites"
         data-link_position="3:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          SUITES
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="8784b1a0-bcc4-429d-accf-91505e9e98b2">
            
      <a href="https://www.49ers.com/fans/team-stores"
         title="SHOP"
         
         aria-label="SHOP "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="SHOP"
         data-link_url="/fans/team-stores"
         data-link_position="4:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          SHOP
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="42e117ee-cb5e-4b30-8e71-261b415e0797">
            
      <a href="https://sports.yahoo.com/nfl/live-video/?is_retargeting=true&amp;af_sub1=Acquisition&amp;c=US_Acquisition_YMktg_NFLLive_NFLinapp_NFLteamappusers&amp;pid=partnership&amp;af_sub5=NFLteamapps__Static_&amp;af_sub2=US_YMktg"
         title="LIVE NFL GAMES"
         
         aria-label="LIVE NFL GAMES "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="LIVE NFL GAMES"
         data-link_url="https://sports.yahoo.com/nfl/live-video/?is_retargeting=true&amp;af_sub1=Acquisition&amp;c=US_Acquisition_YMktg_NFLLive_NFLinapp_NFLteamappusers&amp;pid=partnership&amp;af_sub5=NFLteamapps__Static_&amp;af_sub2=US_YMktg"
         data-link_position="5:5"
         data-custom-properties="{&quot;devicevisibility&quot;:&quot;ipad|mobile&quot;}"
         >
        <span class="d3-o-nav__item-label">
          LIVE NFL GAMES
        </span>
      </a>
  





          </li>
    </ul>





<span style="display: none">
    <svg class="nfl-o-icon-included--user" data-viewbox="0 0 32 32">
      <use xlink:href='#user'></use>
    </svg>
</span>

        </div>
      </nav>

      <div class="d3-o-nav__seasonal">
        
        
            <div class="d3-o-nav__seasonal-item">
              <a href="/fttb">#FTTB</a>
            </div>
      </div>

        <div class="d3-o-nav__search d3-o-nav__search">
          <div class="d3-o-search">
            <form action="/search/">
              <label class="d3-o-search__label" for="query">Search</label>
              <input class="d3-o-search__input" type="text" id="query" name="query" autocomplete="off" />
              <span class="d3-o-search__input--autocomplete" id="query-suggestion"></span>
              <button class="d3-o-search__open-btn" aria-pressed="false" aria-label="Search" type="submit">
                



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--search" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#search'></use>
  </svg>
</span>

              </button>
            </form>
            <button class="d3-o-search__close-btn" aria-pressed="true" aria-label="Close search form">
              



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--close" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#close'></use>
  </svg>
</span>

            </button>
          </div>
        </div>

      <nav role="navigation" class="d3-o-nav--action-related">
        <h2 class="d3-o-nav--title">
          Action related nav
        </h2>






<script type="text/javascript">
          window._DMAMenuItems = window._DMAMenuItems || [];
          window._DMAMenuItems = window._DMAMenuItems.concat([]);
        </script>    <ul class="d3-o-nav__list d3-o-list">
          <li class="d3-o-nav__item  "  data-js-hook="2b3a9e61-29c1-48d7-9788-0111d4110616">
            
      <a href="https://www.49ers.com/watch-live-games/ways-to-watch"
         title="WATCH"
         
         aria-label="WATCH "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="WATCH"
         data-link_url="https://www.49ers.com/watch-live-games/ways-to-watch"
         data-link_position="1:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          WATCH
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="a1f99d72-9083-4666-aac1-cf056feb890e">
            
      <a href="https://www.49ers.com/tickets"
         title="TICKETS"
         
         aria-label="TICKETS "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="TICKETS"
         data-link_url="/tickets"
         data-link_position="2:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          TICKETS
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="b92f6183-d2cd-4d06-b439-cc1dd8f12f46">
            
      <a href="https://www.49ers.com/tickets/suites"
         title="SUITES"
         
         aria-label="SUITES "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="SUITES"
         data-link_url="/tickets/suites"
         data-link_position="3:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          SUITES
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="8784b1a0-bcc4-429d-accf-91505e9e98b2">
            
      <a href="https://www.49ers.com/fans/team-stores"
         title="SHOP"
         
         aria-label="SHOP "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="SHOP"
         data-link_url="/fans/team-stores"
         data-link_position="4:5"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          SHOP
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="42e117ee-cb5e-4b30-8e71-261b415e0797">
            
      <a href="https://sports.yahoo.com/nfl/live-video/?is_retargeting=true&amp;af_sub1=Acquisition&amp;c=US_Acquisition_YMktg_NFLLive_NFLinapp_NFLteamappusers&amp;pid=partnership&amp;af_sub5=NFLteamapps__Static_&amp;af_sub2=US_YMktg"
         title="LIVE NFL GAMES"
         
         aria-label="LIVE NFL GAMES "
         data-link_module="Header"
         data-event_name="click action"
         data-link_type="_core-primary-nav-links"
         data-link_name="LIVE NFL GAMES"
         data-link_url="https://sports.yahoo.com/nfl/live-video/?is_retargeting=true&amp;af_sub1=Acquisition&amp;c=US_Acquisition_YMktg_NFLLive_NFLinapp_NFLteamappusers&amp;pid=partnership&amp;af_sub5=NFLteamapps__Static_&amp;af_sub2=US_YMktg"
         data-link_position="5:5"
         data-custom-properties="{&quot;devicevisibility&quot;:&quot;ipad|mobile&quot;}"
         >
        <span class="d3-o-nav__item-label">
          LIVE NFL GAMES
        </span>
      </a>
  





          </li>
    </ul>





<span style="display: none">
    <svg class="nfl-o-icon-included--user" data-viewbox="0 0 32 32">
      <use xlink:href='#user'></use>
    </svg>
</span>

      </nav>

      <nav role="navigation" class="d3-o-nav--secondary">
        





<script type="text/javascript">
          window._DMAMenuItems = window._DMAMenuItems || [];
          window._DMAMenuItems = window._DMAMenuItems.concat([]);
        </script>    <ul class="d3-o-nav__list d3-o-list">
          <li class="d3-o-nav__item   d3-is-active"  data-js-hook="bb8ecdf1-7471-44df-a1d5-8f90c58ad72d">
            
      <a href="https://www.49ers.com/team/players-roster/"
         title="Roster"
         
         aria-label="Roster "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="Roster"
         data-link_url="/team/players-roster/"
         data-link_position="1:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          Roster
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="231e0fe4-2d59-4af6-a9c3-500d1886b5f5">
            
      <a href="https://www.49ers.com/team/depth-chart"
         title="Depth Chart"
         
         aria-label="Depth Chart "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="Depth Chart"
         data-link_url="/team/depth-chart"
         data-link_position="2:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          Depth Chart
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="3b9f6c8c-7ef6-4040-834f-553642b2226b">
            
      <a href="https://www.49ers.com/team/coaches-roster/"
         title="Coaches"
         
         aria-label="Coaches "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="Coaches"
         data-link_url="/team/coaches-roster/"
         data-link_position="3:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          Coaches
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="9954eda5-29bd-4de5-9efe-94918d8d3264">
            
      <a href="https://www.49ers.com/team/stats/"
         title="Stats"
         
         aria-label="Stats "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="Stats"
         data-link_url="/team/stats/"
         data-link_position="4:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          Stats
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="25be5dd3-eaf6-41f9-9d65-8319e3d03a4e">
            
      <a href="https://www.49ers.com/team/injury-report/"
         title="Injury Report"
         
         aria-label="Injury Report "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="Injury Report"
         data-link_url="/team/injury-report/"
         data-link_position="5:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          Injury Report
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="561c3983-7533-4e8e-8e92-1443a9db5ca1">
            
      <a href="https://www.49ers.com/team/front-office-roster/"
         title="Front Office"
         
         aria-label="Front Office "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="Front Office"
         data-link_url="/team/front-office-roster/"
         data-link_position="6:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          Front Office
        </span>
      </a>
  





          </li>
          <li class="d3-o-nav__item  "  data-js-hook="c79f0d7e-3f7d-4d1a-9b67-356b69724a0b">
            
      <a href="https://www.49ers.com/history/"
         title="History"
         
         aria-label="History "
         data-link_module="Second-Level-menu"
         data-event_name="click action"
         data-link_type="_team-secondary-nav-links"
         data-link_name="History"
         data-link_url="/history/"
         data-link_position="7:7"
         data-custom-properties=""
         >
        <span class="d3-o-nav__item-label">
          History
        </span>
      </a>
  





          </li>
    </ul>





<span style="display: none">
    <svg class="nfl-o-icon-included--user" data-viewbox="0 0 32 32">
      <use xlink:href='#user'></use>
    </svg>
</span>

      </nav>

    <div class="d3-o-nav__search-result">
    </div>
  </div>

</header>

  
        <section class="d3-l-grid--outer d3-l-adv-row">
          <h2 class="d3-o-section-title"> Advertising</h2>
          <div class="d3-l-grid--inner">
            <script></script><div class="  d3-o-adv-block" data-id="adv_club_premium" id="adv_club_premium"></div><script></script>
          </div>
        </section>

  <main role="main" id="main-content">




<h1 aria-label="San Francisco 49ers Player Roster" class="nfl-o-page-title nfl-o-page-title--visuallyhidden">San Francisco 49ers Player Roster</h1>    
      <section class="d3-l-grid--outer d3-l-section-row">
        <div class="d3-l-grid--inner">
          

<div class="d3-l-col__col-12" >
  <div class="d3-l-section-header">

        <h2 class="d3-o-section-title ">
              <span>
                Team Roster
              </span>
          </h2>
      

      <div class="d3-o-section-adv">
          <span class="d3-o-section-adv__label">
            Presented by
          </span>

        <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['23e1eb1c-16ae-412d-bb1f-65bf3ab00d22'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, '23e1eb1c-16ae-412d-bb1f-65bf3ab00d22')
                  .setTargeting('partner', 'Intel_Roster')
                  .setTargeting('slot', 'logo')
                  .defineSizeMapping(gptconfig.responsiveMappings['presented-by']).addService(googletag.pubads())
              );
            });
          
              window['23e1eb1c-16ae-412d-bb1f-65bf3ab00d22_refreshAdv'] = function() {
                googletag.pubads().refresh([slot['23e1eb1c-16ae-412d-bb1f-65bf3ab00d22']],{changeCorrelator: false});
              };
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['23e1eb1c-16ae-412d-bb1f-65bf3ab00d22'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('23e1eb1c-16ae-412d-bb1f-65bf3ab00d22')
            });
          });
        }
      </script><div class="d3-o-adv-block--refreshable d3-o-adv-block js-lightbox-refresh-ignore" id="23e1eb1c-16ae-412d-bb1f-65bf3ab00d22"></div><script></script>
      </div>
</div>
</div>

          <script>window.partnerCampaignID = window.partnerCampaignID || 'Intel_Roster';</script>
        </div>
      </section>

    



  <section class="d3-l-grid--outer d3-l-section-row">
    <div class="d3-l-grid--inner ">
      



                <div class="d3-l-col__col-12">
            



<section class="d3-l-grid--outer d3-l-section-row d3-o-cta-bar d3-o-cta-bar--horizontal">
  <div class="d3-l-grid--inner">

      <div class="d3-l-col__col-4 d3-o-cta-bar__item--button-color">
        <a href="/team/depth-chart"
           title="Depth Chart"
           target="_self"
           data-link_module="CTA Bar"
           data-event_name="click action"
           data-link_type="roster"
           data-link_name="Depth Chart"
           data-link_url="/team/depth-chart"
           data-link_position="1:3">
          Depth Chart
        </a>
      </div>
      <div class="d3-l-col__col-4 d3-o-cta-bar__item--button-color">
        <a href="/team/injury-report/"
           title="Injury Report"
           target="_self"
           data-link_module="CTA Bar"
           data-event_name="click action"
           data-link_type="roster"
           data-link_name="Injury Report"
           data-link_url="/team/injury-report/"
           data-link_position="2:3">
          Injury Report
        </a>
      </div>
      <div class="d3-l-col__col-4 d3-o-cta-bar__item--button-color">
        <a href="https://static.clubs.nfl.com/image/upload/v1606325174/49ers/gk5zgvieqisp4sqey31f.pdf"
           title="Printable Roster"
           target="_self"
           data-link_module="CTA Bar"
           data-event_name="click action"
           data-link_type="roster"
           data-link_name="Printable Roster"
           data-link_url="https://static.clubs.nfl.com/image/upload/v1606325174/49ers/gk5zgvieqisp4sqey31f.pdf"
           data-link_position="3:3">
          Printable Roster
        </a>
      </div>

  </div>
</section>
          </div>
    </div>
  </section>




  <section class="d3-l-grid--outer d3-l-section-row">
    <div class="d3-l-grid--inner ">
      

<div class="d3-l-col__col-12" >
  <div class="d3-l-section-header">

        <h2 class="d3-o-section-title ">
              <span>
                Team Captains
              </span>
          </h2>
      

</div>
</div>


                <div class="d3-l-col__col-12">
            



<section class="d3-l-grid--outer d3-l-section-row">
  <div class="d3-l-grid--inner " data-require=modules/carousel  data-jsonID="3c339fb3-1b81-4c95-8b5c-83ac04c77f13" >

    


<div class="d3-l-col__col-12 d3-o-carousel owl-carousel
           d3-o-carousel--arrows-outside"
           data-is-content-tray="true"
           data-count="7"
           data-items="4"
           data-cycle="False"
           role="region"
           aria-label="carousel with 7 items">
        


    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="Fred Warner"
         data-link_url="/team/players-roster/fred-warner/"
         data-link_position="1:7"
         href="/team/players-roster/fred-warner/"
         title="Fred Warner"
         data-id="a96e8de7-4b62-406b-9ce5-cf57deb402af"
         
         aria-label="Fred Warner">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          Fred Warner
        </h3>
                      </div>
      </a>
    </div>



    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="Richard Sherman"
         data-link_url="/team/players-roster/richard-sherman/"
         data-link_position="2:7"
         href="/team/players-roster/richard-sherman/"
         title="Richard Sherman"
         data-id="fa38d522-7034-4f77-92ff-cba2d385fe49"
         
         aria-label="Richard Sherman">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          Richard Sherman
        </h3>
                      </div>
      </a>
    </div>



    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="Trent Williams"
         data-link_url="/team/players-roster/trent-williams/"
         data-link_position="3:7"
         href="/team/players-roster/trent-williams/"
         title="Trent Williams"
         data-id="1d135fa5-b8f9-47a2-8eee-017f28a969ef"
         
         aria-label="Trent Williams">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          Trent Williams
        </h3>
                      </div>
      </a>
    </div>



    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="Arik Armstead"
         data-link_url="/team/players-roster/arik-armstead/"
         data-link_position="4:7"
         href="/team/players-roster/arik-armstead/"
         title="Arik Armstead"
         data-id="04fb5107-a252-4184-a44e-a3a7469f3364"
         
         aria-label="Arik Armstead">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          Arik Armstead
        </h3>
                      </div>
      </a>
    </div>



    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="Mike McGlinchey"
         data-link_url="/team/players-roster/mike-mcglinchey/"
         data-link_position="5:7"
         href="/team/players-roster/mike-mcglinchey/"
         title="Mike McGlinchey"
         data-id="b3bb7d33-d7e3-4614-ac9f-3ab59134f0c9"
         
         aria-label="Mike McGlinchey">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" media="(min-width:1024px)" /><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" media="(min-width:768px)" /><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" data-src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          Mike McGlinchey
        </h3>
                      </div>
      </a>
    </div>



    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="Jimmy Garoppolo"
         data-link_url="/team/players-roster/jimmy-garoppolo/"
         data-link_position="6:7"
         href="/team/players-roster/jimmy-garoppolo/"
         title="Jimmy Garoppolo"
         data-id="31274802-8aae-4a0e-be0c-c11f14f39788"
         
         aria-label="Jimmy Garoppolo">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" media="(min-width:1024px)" /><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" media="(min-width:768px)" /><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" data-src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          Jimmy Garoppolo
        </h3>
                      </div>
      </a>
    </div>



    <div class="d3-o-media-object d3-o-media-object--vertical d3-o-content-tray__card">
      <a 
         data-link_module="Content Tray_Card carousel"
         data-link_type="featured-players"
         data-link_name="George Kittle"
         data-link_url="/team/players-roster/george-kittle/"
         data-link_position="7:7"
         href="/team/players-roster/george-kittle/"
         title="George Kittle"
         data-id="961acf92-25a9-4af0-849b-42a63465b5ca"
         
         aria-label="George Kittle">
        



<figure class="d3-o-media-object__figure">
  <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_2x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_3_4_desktop_3x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" media="(min-width:1024px)" /><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_2x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_tablet_3x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" media="(min-width:768px)" /><source data-srcset="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 1x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_2x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 2x, https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile_3x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" /><!--[if IE 9]></video><![endif]--><img alt="" class="img-responsive" data-src="https://static.clubs.nfl.com/image/private/t_editorial_landscape_8_desktop_mobile/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==" /></picture>
  



  </figure>



    <div class="d3-o-media-object__body">
                    <h3 class="d3-o-media-object__title">
          George Kittle
        </h3>
                      </div>
      </a>
    </div>
      </div>
    

  </div>
  



<span style="display: none">
    <svg class="nfl-o-icon-included--down" data-viewbox="0 0 24 24">
      <use xlink:href='#down'></use>
    </svg>
</span>


</section>
          </div>
    </div>
  </section>



<section class="d3-l-grid--outer d3-l-section-row">
  <div class="d3-l-grid--inner">
          <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Active</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Active
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/chris-edwards/" title="Chris Edwards">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg" /><!--[if IE 9]></video><![endif]--><img alt="S Chris Edwards" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ws1y3obsnrxeirtcpa5k.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="edwards,chris "><a href="/team/players-roster/chris-edwards/">Chris Edwards</a></span>
                      </div>
                    </td>
                    <td data-append="1">38</td>
                    <td data-append="1">S</td>
                    <td class="sorter-custom-height" data-name="edwards,chris ">6-2</td>
                    <td data-append="1" data-value="215">215</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="Idaho">Idaho</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/taybor-pepper/" title="Taybor Pepper">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Taybor_Pepper" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lulfxy32mthkplsdtnfc.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="pepper,taybor "><a href="/team/players-roster/taybor-pepper/">Taybor Pepper</a></span>
                      </div>
                    </td>
                    <td data-append="1">46</td>
                    <td data-append="1">LS</td>
                    <td class="sorter-custom-height" data-name="pepper,taybor ">6-4</td>
                    <td data-append="1" data-value="245">245</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="MichiganState">Michigan State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/ken-webster/" title="Ken Webster">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg" /><!--[if IE 9]></video><![endif]--><img alt="KenWebster" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m9llk571phsdnbw54rkx.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="webster,kendarius "><a href="/team/players-roster/ken-webster/">Ken Webster</a></span>
                      </div>
                    </td>
                    <td data-append="1">40</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="webster,kendarius ">5-11</td>
                    <td data-append="1" data-value="202">202</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Mississippi">Mississippi</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/river-cracraft/" title="River Cracraft">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg" /><!--[if IE 9]></video><![endif]--><img alt="River,CraCraft" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/drecwkidhskaguib9n0l.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="cracraft,tanner "><a href="/team/players-roster/river-cracraft/">River Cracraft</a></span>
                      </div>
                    </td>
                    <td data-append="1">86</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="cracraft,tanner ">6-0</td>
                    <td data-append="1" data-value="198">198</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="WashingtonState">Washington State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/hroniss-grasu/" title="Hroniss Grasu">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /><!--[if IE 9]></video><![endif]--><img alt="HronisGrassu" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/matd1rdjgnmqslylme3h.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="grasu,hroniss "><a href="/team/players-roster/hroniss-grasu/">Hroniss Grasu</a></span>
                      </div>
                    </td>
                    <td data-append="1">50</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="grasu,hroniss ">6-3</td>
                    <td data-append="1" data-value="301">301</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Oregon">Oregon</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jordan-reed/" title="Jordan Reed">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg" /><!--[if IE 9]></video><![endif]--><img alt="JordanReed" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mh3crk6dqvxhb5ktkjun.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="reed,jordan "><a href="/team/players-roster/jordan-reed/">Jordan Reed</a></span>
                      </div>
                    </td>
                    <td data-append="1">81</td>
                    <td data-append="1">TE</td>
                    <td class="sorter-custom-height" data-name="reed,jordan ">6-2</td>
                    <td data-append="1" data-value="242">242</td>
                    <td data-append="1" data-value="30">30</td>
                    <td data-append="1" data-value="8"><span>8</span></td>
                    <td data-append="1" data-value="Florida">Florida</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/dion-jordan/" title="Dion Jordan">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Jordan_Dion" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/aikkt0iz4ttrg4kzec60.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="jordan,dion "><a href="/team/players-roster/dion-jordan/">Dion Jordan</a></span>
                      </div>
                    </td>
                    <td data-append="1">96</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="jordan,dion ">6-6</td>
                    <td data-append="1" data-value="284">284</td>
                    <td data-append="1" data-value="30">30</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Oregon">Oregon</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jamar-taylor/" title="Jamar Taylor">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Taylor_Jamar" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xtandrkx1hmz3ina0gg7.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="taylor,jamar "><a href="/team/players-roster/jamar-taylor/">Jamar Taylor</a></span>
                      </div>
                    </td>
                    <td data-append="1">47</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="taylor,jamar ">5-11</td>
                    <td data-append="1" data-value="192">192</td>
                    <td data-append="1" data-value="30">30</td>
                    <td data-append="1" data-value="8"><span>8</span></td>
                    <td data-append="1" data-value="BoiseState">Boise State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/charlie-woerner/" title="Charlie Woerner">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Woerner_Charlie" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rhz2s2r7p06l2chjfszu.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="woerner,charles "><a href="/team/players-roster/charlie-woerner/">Charlie Woerner</a></span>
                      </div>
                    </td>
                    <td data-append="1">89</td>
                    <td data-append="1">TE</td>
                    <td class="sorter-custom-height" data-name="woerner,charles ">6-5</td>
                    <td data-append="1" data-value="241">241</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Georgia">Georgia</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/colton-mckivitz/" title="Colton McKivitz">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg" /><!--[if IE 9]></video><![endif]--><img alt="McKiVitz_Colton" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b4a2jgpc4l6fygn86ydf.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="mckivitz,colton "><a href="/team/players-roster/colton-mckivitz/">Colton McKivitz</a></span>
                      </div>
                    </td>
                    <td data-append="1">68</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="mckivitz,colton ">6-6</td>
                    <td data-append="1" data-value="301">301</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="WestVirginia">West Virginia</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/trent-williams/" title="Trent Williams">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Williams_Trent" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bnq8i5urjualxre5caqz.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="williams,trenton "><a href="/team/players-roster/trent-williams/">Trent Williams</a></span>
                      </div>
                    </td>
                    <td data-append="1">71</td>
                    <td data-append="1">T</td>
                    <td class="sorter-custom-height" data-name="williams,trenton ">6-5</td>
                    <td data-append="1" data-value="320">320</td>
                    <td data-append="1" data-value="32">32</td>
                    <td data-append="1" data-value="10"><span>10</span></td>
                    <td data-append="1" data-value="Oklahoma">Oklahoma</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/javon-kinlaw/" title="Javon Kinlaw">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Kinlaw_Javon" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/bvkayls170qxhfvjkgen.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="kinlaw,javon "><a href="/team/players-roster/javon-kinlaw/">Javon Kinlaw</a></span>
                      </div>
                    </td>
                    <td data-append="1">99</td>
                    <td data-append="1">DT</td>
                    <td class="sorter-custom-height" data-name="kinlaw,javon ">6-5</td>
                    <td data-append="1" data-value="319">319</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="SouthCarolina">South Carolina</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/tom-compton/" title="Tom Compton">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Compton_Tom" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qxubbnk5o13qxbqsjrjd.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="compton,thomas "><a href="/team/players-roster/tom-compton/">Tom Compton</a></span>
                      </div>
                    </td>
                    <td data-append="1">66</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="compton,thomas ">6-6</td>
                    <td data-append="1" data-value="315">315</td>
                    <td data-append="1" data-value="31">31</td>
                    <td data-append="1" data-value="8"><span>8</span></td>
                    <td data-append="1" data-value="SouthDakota">South Dakota</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kerry-hyder/" title="Kerry Hyder Jr.&#xA0;">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Hyder-Jr._Kerry" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ddc9rya8ddqq1nvozms7.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="hyder,kerry "><a href="/team/players-roster/kerry-hyder/">Kerry Hyder Jr.&#xA0;</a></span>
                      </div>
                    </td>
                    <td data-append="1">92</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="hyder,kerry ">6-2</td>
                    <td data-append="1" data-value="275">275</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="5"><span>5</span></td>
                    <td data-append="1" data-value="TexasTech">Texas Tech</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/joe-walker/" title="Joe Walker">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Walker_Joe" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hq6t73z0e0xqwcecrkan.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="walker,joseph "><a href="/team/players-roster/joe-walker/">Joe Walker</a></span>
                      </div>
                    </td>
                    <td data-append="1">59</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="walker,joseph ">6-2</td>
                    <td data-append="1" data-value="236">236</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="5"><span>5</span></td>
                    <td data-append="1" data-value="Oregon">Oregon</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/demetrius-flannigan-fowles/" title="Demetrius Flannigan-Fowles">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Flannigan-Fowles" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/usqtlsukitoxs4qc2x9s.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="flannigan-fowles,demetrius "><a href="/team/players-roster/demetrius-flannigan-fowles/">Demetrius Flannigan-Fowles</a></span>
                      </div>
                    </td>
                    <td data-append="1">45</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="flannigan-fowles,demetrius ">6-2</td>
                    <td data-append="1" data-value="210">210</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="Arizona">Arizona</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/azeez-al-shaair/" title="Azeez Al-Shaair">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Azeez" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xaixahfhzzp9j3h5nwjt.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="al-shaair,azeez "><a href="/team/players-roster/azeez-al-shaair/">Azeez Al-Shaair</a></span>
                      </div>
                    </td>
                    <td data-append="1">51</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="al-shaair,azeez ">6&#x27;2</td>
                    <td data-append="1" data-value="228">228</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="FloridaAtlantic">Florida Atlantic</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kevin-givens/" title="Kevin Givens">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Givens" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ew0lpx6udtzasobdqw9b.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="givens,kevin "><a href="/team/players-roster/kevin-givens/">Kevin Givens</a></span>
                      </div>
                    </td>
                    <td data-append="1">90</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="givens,kevin ">6&#x27;1</td>
                    <td data-append="1" data-value="285">285</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="PennState">Penn State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/austin-walter/" title="Austin Walter">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Walter" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wfsviasbhkjmr50c5lsi.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="walter,austin "><a href="/team/players-roster/austin-walter/">Austin Walter</a></span>
                      </div>
                    </td>
                    <td data-append="1">48</td>
                    <td data-append="1">RB</td>
                    <td class="sorter-custom-height" data-name="walter,austin ">5-8</td>
                    <td data-append="1" data-value="202">202</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="Rice">Rice</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/justin-skule/" title="Justin Skule">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Skule" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lzk6j0egzxfm4iec8cdq.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="skule,justin "><a href="/team/players-roster/justin-skule/">Justin Skule</a></span>
                      </div>
                    </td>
                    <td data-append="1">67</td>
                    <td data-append="1">OT</td>
                    <td class="sorter-custom-height" data-name="skule,justin ">6&#x27;6</td>
                    <td data-append="1" data-value="315">315</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Vanderbilt">Vanderbilt</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/dre-greenlaw/" title="Dre Greenlaw">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Greenlaw" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cc6muuym4vrixp0sg4zg.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="greenlaw,keaundre "><a href="/team/players-roster/dre-greenlaw/">Dre Greenlaw</a></span>
                      </div>
                    </td>
                    <td data-append="1">57</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="greenlaw,keaundre ">6&#x27;0</td>
                    <td data-append="1" data-value="230">230</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Arkansas">Arkansas</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/mitch-wishnowsky/" title="Mitch Wishnowsky">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Wish" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ke3qmnb4oeygblelkrue.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="wishnowsky,mitch "><a href="/team/players-roster/mitch-wishnowsky/">Mitch Wishnowsky</a></span>
                      </div>
                    </td>
                    <td data-append="1">6</td>
                    <td data-append="1">P</td>
                    <td class="sorter-custom-height" data-name="wishnowsky,mitch ">6&#x27;2</td>
                    <td data-append="1" data-value="220">220</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Utah">Utah</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/deebo-samuel/" title="Deebo Samuel">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Deebo" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pmmt5zfuhaehrsdfzp9u.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="samuel,tyshun "><a href="/team/players-roster/deebo-samuel/">Deebo Samuel</a></span>
                      </div>
                    </td>
                    <td data-append="1">19</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="samuel,tyshun ">6-0</td>
                    <td data-append="1" data-value="215">215</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="SouthCarolina">South Carolina</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/daniel-brunskill/" title="Daniel Brunskill">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Brunskill-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/rdv1s94uzuajpfidxj3u.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="brunskill,daniel "><a href="/team/players-roster/daniel-brunskill/">Daniel Brunskill</a></span>
                      </div>
                    </td>
                    <td data-append="1">60</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="brunskill,daniel ">6-5</td>
                    <td data-append="1" data-value="300">300</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="SanDiegoState">San Diego State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/tevin-coleman/" title="Tevin Coleman">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Coleman-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/spfssblaxhw3oeqoj6vs.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="coleman,tevin "><a href="/team/players-roster/tevin-coleman/">Tevin Coleman</a></span>
                      </div>
                    </td>
                    <td data-append="1">26</td>
                    <td data-append="1">RB</td>
                    <td class="sorter-custom-height" data-name="coleman,tevin ">6-1</td>
                    <td data-append="1" data-value="210">210</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Indiana">Indiana</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jason-verrett/" title="Jason Verrett">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Verrett_Jason" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nrarutulj15q9polkrsb.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="verrett,jason "><a href="/team/players-roster/jason-verrett/">Jason Verrett</a></span>
                      </div>
                    </td>
                    <td data-append="1">22</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="verrett,jason ">5-10</td>
                    <td data-append="1" data-value="188">188</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="TexasChristian">Texas Christian</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jeff-wilson/" title="Jeff Wilson Jr.">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Wilson, Jeffery" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pkwpucwwcubzcjubdhf4.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="wilson,jeffery jr."><a href="/team/players-roster/jeff-wilson/">Jeff Wilson Jr.</a></span>
                      </div>
                    </td>
                    <td data-append="1">30</td>
                    <td data-append="1">RB</td>
                    <td class="sorter-custom-height" data-name="wilson,jeffery jr.">6-0</td>
                    <td data-append="1" data-value="213">213</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="NorthTexas">North Texas</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/emmanuel-moseley/" title="Emmanuel Moseley">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Moseley, Emmanuel" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jlfsfj1olzxua7g8n4tf.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="moseley,emmanuel "><a href="/team/players-roster/emmanuel-moseley/">Emmanuel Moseley</a></span>
                      </div>
                    </td>
                    <td data-append="1">41</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="moseley,emmanuel ">5-11</td>
                    <td data-append="1" data-value="190">190</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="Tennessee">Tennessee</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/ross-dwelley/" title="Ross Dwelley">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Dwelley, Ross" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/eqspkl3ua5delftrxmyt.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="dwelley,ross "><a href="/team/players-roster/ross-dwelley/">Ross Dwelley</a></span>
                      </div>
                    </td>
                    <td data-append="1">82</td>
                    <td data-append="1">TE</td>
                    <td class="sorter-custom-height" data-name="dwelley,ross ">6-5</td>
                    <td data-append="1" data-value="235">235</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="SanDiego">San Diego</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/richie-james/" title="Richie James">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg" /><!--[if IE 9]></video><![endif]--><img alt="James, Richie" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jmzslnuczxvisuvmjb1b.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="james,richard "><a href="/team/players-roster/richie-james/">Richie James</a></span>
                      </div>
                    </td>
                    <td data-append="1">13</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="james,richard ">5-9</td>
                    <td data-append="1" data-value="185">185</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="MiddleTennesseeState">Middle Tennessee State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/marcell-harris/" title="Marcell Harris">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Harris, Marcell" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/j6olsxxc9efxgbaom0pm.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="harris,marcell "><a href="/team/players-roster/marcell-harris/">Marcell Harris</a></span>
                      </div>
                    </td>
                    <td data-append="1">36</td>
                    <td data-append="1">S</td>
                    <td class="sorter-custom-height" data-name="harris,marcell ">6-0</td>
                    <td data-append="1" data-value="215">215</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="Florida">Florida</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kentavius-street/" title="Kentavius Street">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg" /><!--[if IE 9]></video><![endif]--><img alt="This is a 2016 photo of Asa Jackson of the Arizona Cardinals NFL football team. This image reflects the Arizona Cardinals active roster as of Monday, June 6, 2016 when this image was taken. (AP Photo)" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fjso1bo8iuic7fskoadj.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="street,kentavius "><a href="/team/players-roster/kentavius-street/">Kentavius Street</a></span>
                      </div>
                    </td>
                    <td data-append="1">95</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="street,kentavius ">6-2</td>
                    <td data-append="1" data-value="287">287</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="N.C.State">N.C. State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/tarvarius-moore/" title="Tarvarius Moore">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Moore, Tarvarius" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gb43uym1wuwetggvwdxq.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="moore,tarvarius "><a href="/team/players-roster/tarvarius-moore/">Tarvarius Moore</a></span>
                      </div>
                    </td>
                    <td data-append="1">33</td>
                    <td data-append="1">DB</td>
                    <td class="sorter-custom-height" data-name="moore,tarvarius ">6-2</td>
                    <td data-append="1" data-value="200">200</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="SouthernMiss">Southern Miss</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/fred-warner/" title="Fred Warner">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Warner, Fred" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/z1o1mamvknzcten0gmtk.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="warner,federico "><a href="/team/players-roster/fred-warner/">Fred Warner</a></span>
                      </div>
                    </td>
                    <td data-append="1">54</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="warner,federico ">6-3</td>
                    <td data-append="1" data-value="230">230</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="BYU">BYU</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/mike-mcglinchey/" title="Mike McGlinchey">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" /><!--[if IE 9]></video><![endif]--><img alt="McGlinchey, Mike" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/k0ogoua8oi6aeaxfogzc.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="mcglinchey,michael "><a href="/team/players-roster/mike-mcglinchey/">Mike McGlinchey</a></span>
                      </div>
                    </td>
                    <td data-append="1">69</td>
                    <td data-append="1">T</td>
                    <td class="sorter-custom-height" data-name="mcglinchey,michael ">6-8</td>
                    <td data-append="1" data-value="310">310</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="NotreDame">Notre Dame</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jerick-mckinnon/" title="Jerick McKinnon">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg" /><!--[if IE 9]></video><![endif]--><img alt="McKinnon, Jerick" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/v6vfleapzi7vi5x5307x.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="mckinnon,jerick "><a href="/team/players-roster/jerick-mckinnon/">Jerick McKinnon</a></span>
                      </div>
                    </td>
                    <td data-append="1">28</td>
                    <td data-append="1">RB</td>
                    <td class="sorter-custom-height" data-name="mckinnon,jerick ">5-9</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="GeorgiaSouthern">Georgia Southern</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/richard-sherman/" title="Richard Sherman">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Sherman_Richard" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/thxq7uskkz13xwnvod4f.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="sherman,richard "><a href="/team/players-roster/richard-sherman/">Richard Sherman</a></span>
                      </div>
                    </td>
                    <td data-append="1">25</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="sherman,richard ">6-3</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="32">32</td>
                    <td data-append="1" data-value="10"><span>10</span></td>
                    <td data-append="1" data-value="Stanford">Stanford</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/ahkello-witherspoon/" title="Ahkello Witherspoon">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Witherspoon, Ahkello" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qetejynuuhyx9oadmcwd.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="witherspoon,james "><a href="/team/players-roster/ahkello-witherspoon/">Ahkello Witherspoon</a></span>
                      </div>
                    </td>
                    <td data-append="1">23</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="witherspoon,james ">6-3</td>
                    <td data-append="1" data-value="195">195</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="Colorado">Colorado</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/k-waun-williams/" title="K&#x27;Waun Williams">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Williams, K&#x27;Waun" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/egjaji8quxreea9sqe3v.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="williams,k&#x27;waun "><a href="/team/players-roster/k-waun-williams/">K&#x27;Waun Williams</a></span>
                      </div>
                    </td>
                    <td data-append="1">24</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="williams,k&#x27;waun ">5-9</td>
                    <td data-append="1" data-value="185">185</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Pittsburgh">Pittsburgh</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jimmie-ward/" title="Jimmie Ward">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Ward, Jimmie" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oocscdtq1ldk47wgfzck.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="ward,james "><a href="/team/players-roster/jimmie-ward/">Jimmie Ward</a></span>
                      </div>
                    </td>
                    <td data-append="1">20</td>
                    <td data-append="1">DB</td>
                    <td class="sorter-custom-height" data-name="ward,james ">5-11</td>
                    <td data-append="1" data-value="195">195</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="NorthernIllinois">Northern Illinois</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/laken-tomlinson/" title="Laken Tomlinson">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Tomlinson, Laken" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/arvzdwjpcawmp7o8glux.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="tomlinson,laken "><a href="/team/players-roster/laken-tomlinson/">Laken Tomlinson</a></span>
                      </div>
                    </td>
                    <td data-append="1">75</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="tomlinson,laken ">6-3</td>
                    <td data-append="1" data-value="315">315</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Duke">Duke</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/trent-taylor/" title="Trent Taylor">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Taylor, Trent" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gyzgjsyhknqjzlbcq2yw.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="taylor,trent "><a href="/team/players-roster/trent-taylor/">Trent Taylor</a></span>
                      </div>
                    </td>
                    <td data-append="1">15</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="taylor,trent ">5-8</td>
                    <td data-append="1" data-value="180">180</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="LouisianaTech">Louisiana Tech</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/dontae-johnson/" title="Dontae Johnson">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg" /><!--[if IE 9]></video><![endif]--><img alt="This is a 2017 photo of Dontae Johnson of the San Francisco 49ers NFL football team. This image reflects the 2017 active roster as of Thursday, May 4, 2017 when this image was taken. (AP Photo)" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/phtrm7dlfoycwcpqrl1p.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="johnson,dontae "><a href="/team/players-roster/dontae-johnson/">Dontae Johnson</a></span>
                      </div>
                    </td>
                    <td data-append="1">27</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="johnson,dontae ">6-2</td>
                    <td data-append="1" data-value="200">200</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="NorthCarolinaState">North Carolina State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/raheem-mostert/" title="Raheem Mostert">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Mostert, Raheem" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/auzxyl7gg4ggz4wuo3zh.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="mostert,dominique "><a href="/team/players-roster/raheem-mostert/">Raheem Mostert</a></span>
                      </div>
                    </td>
                    <td data-append="1">31</td>
                    <td data-append="1">RB</td>
                    <td class="sorter-custom-height" data-name="mostert,dominique ">5-10</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Purdue">Purdue</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/robbie-gould/" title="Robbie Gould">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Gould, Robbie" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hs6epsahwvycfcqbziun.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="gould,robert "><a href="/team/players-roster/robbie-gould/">Robbie Gould</a></span>
                      </div>
                    </td>
                    <td data-append="1">9</td>
                    <td data-append="1">K</td>
                    <td class="sorter-custom-height" data-name="gould,robert ">6-0</td>
                    <td data-append="1" data-value="190">190</td>
                    <td data-append="1" data-value="37">37</td>
                    <td data-append="1" data-value="16"><span>16</span></td>
                    <td data-append="1" data-value="PennState">Penn State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/nick-mullens/" title="Nick Mullens">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Mullens, Nick" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ytfewq8zctzslnhnzgck.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="mullens,nicholas "><a href="/team/players-roster/nick-mullens/">Nick Mullens</a></span>
                      </div>
                    </td>
                    <td data-append="1">4</td>
                    <td data-append="1">QB</td>
                    <td class="sorter-custom-height" data-name="mullens,nicholas ">6-1</td>
                    <td data-append="1" data-value="210">210</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="SouthernMississippi">Southern Mississippi</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kendrick-bourne/" title="Kendrick Bourne">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Bourne, Kendrick" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lcvxqrl4yf3feh9azakk.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="bourne,kendrick "><a href="/team/players-roster/kendrick-bourne/">Kendrick Bourne</a></span>
                      </div>
                    </td>
                    <td data-append="1">84</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="bourne,kendrick ">6-1</td>
                    <td data-append="1" data-value="190">190</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="EasternWashington">Eastern Washington</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/c-j-beathard/" title="C.J. Beathard">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Bethard, C.J." class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mswdc5ljfjuiva4owaqn.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="beathard,casey "><a href="/team/players-roster/c-j-beathard/">C.J. Beathard</a></span>
                      </div>
                    </td>
                    <td data-append="1">3</td>
                    <td data-append="1">QB</td>
                    <td class="sorter-custom-height" data-name="beathard,casey ">6-2</td>
                    <td data-append="1" data-value="215">215</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="Iowa">Iowa</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/arik-armstead/" title="Arik Armstead">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Armstead, Arik" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/nxtroqdmu71blfr8rbdm.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="armstead,arik "><a href="/team/players-roster/arik-armstead/">Arik Armstead</a></span>
                      </div>
                    </td>
                    <td data-append="1">91</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="armstead,arik ">6-7</td>
                    <td data-append="1" data-value="290">290</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Oregon">Oregon</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kyle-juszczyk/" title="Kyle Juszczyk">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Juszczyk, Kyle" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fmhbvejysko5bky20pof.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="juszczyk,kyle "><a href="/team/players-roster/kyle-juszczyk/">Kyle Juszczyk</a></span>
                      </div>
                    </td>
                    <td data-append="1">44</td>
                    <td data-append="1">FB</td>
                    <td class="sorter-custom-height" data-name="juszczyk,kyle ">6-1</td>
                    <td data-append="1" data-value="235">235</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="8"><span>8</span></td>
                    <td data-append="1" data-value="Harvard">Harvard</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
        <div class="d3-l-col__col-12">
          <div class="d3-o-adv-block">
            <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['d53b2f26-1c9e-4b0f-87b0-327ce1388c26'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, 'd53b2f26-1c9e-4b0f-87b0-327ce1388c26')
                  .setTargeting('slot', 'roster_1')
                  .defineSizeMapping(gptconfig.responsiveMappings['leaderboard'])
.addService(googletag.pubads())
              );});
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['d53b2f26-1c9e-4b0f-87b0-327ce1388c26'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('d53b2f26-1c9e-4b0f-87b0-327ce1388c26')
            });
          });
        }
      </script><div class="  d3-o-adv-block" data-id="d53b2f26-1c9e-4b0f-87b0-327ce1388c26" id="d53b2f26-1c9e-4b0f-87b0-327ce1388c26"></div><script></script>
          </div>
        </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Reserve/Injured</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Reserve/Injured
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/ezekiel-ansah/" title="Ezekiel Ansah">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg" /><!--[if IE 9]></video><![endif]--><img alt="ZiggyAnsah" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uxt3rvo6uqgfesrxgacq.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="ansah,ezekiel "><a href="/team/players-roster/ezekiel-ansah/">Ezekiel Ansah</a></span>
                      </div>
                    </td>
                    <td data-append="1">94</td>
                    <td data-append="1">DE</td>
                    <td class="sorter-custom-height" data-name="ansah,ezekiel ">6-5</td>
                    <td data-append="1" data-value="275">275</td>
                    <td data-append="1" data-value="31">31</td>
                    <td data-append="1" data-value="8"><span>8</span></td>
                    <td data-append="1" data-value="BrighamYoung">Brigham Young</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/johnathan-cyprien/" title="Johnathan Cyprien">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Cyrprien-HS" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cde3op0bj6tcjntgouq7.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="cyprien,johnathan "><a href="/team/players-roster/johnathan-cyprien/">Johnathan Cyprien</a></span>
                      </div>
                    </td>
                    <td data-append="1">32</td>
                    <td data-append="1">S</td>
                    <td class="sorter-custom-height" data-name="cyprien,johnathan ">6-1</td>
                    <td data-append="1" data-value="211">211</td>
                    <td data-append="1" data-value="30">30</td>
                    <td data-append="1" data-value="8"><span>8</span></td>
                    <td data-append="1" data-value="FloridaInternational">Florida International</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jamycal-hasty/" title="JaMycal Hasty">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Hasty_JaMycal" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/wi6lrqtzghojvdpnuh0m.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="hasty,jamycal "><a href="/team/players-roster/jamycal-hasty/">JaMycal Hasty</a></span>
                      </div>
                    </td>
                    <td data-append="1">38</td>
                    <td data-append="1">RB</td>
                    <td class="sorter-custom-height" data-name="hasty,jamycal ">5-8</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Baylor">Baylor</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/chris-thompson/" title="Chris Thompson">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Thompson-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/olxnsskhhzg4d7c7lykw.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="thompson,christopher "><a href="/team/players-roster/chris-thompson/">Chris Thompson</a></span>
                      </div>
                    </td>
                    <td data-append="1">2</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="thompson,christopher ">6-0</td>
                    <td data-append="1" data-value="175">175</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Florida">Florida</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jalen-hurd/" title="Jalen Hurd">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Hurd" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xdungivwtweiotq8ne3n.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="hurd,jalen "><a href="/team/players-roster/jalen-hurd/">Jalen Hurd</a></span>
                      </div>
                    </td>
                    <td data-append="1">14</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="hurd,jalen ">6-5</td>
                    <td data-append="1" data-value="230">230</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Baylor">Baylor</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/nick-bosa/" title="Nick Bosa">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Bosa" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/dlinlnkfq1v2r6vvgehy.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="bosa,nicholas "><a href="/team/players-roster/nick-bosa/">Nick Bosa</a></span>
                      </div>
                    </td>
                    <td data-append="1">97</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="bosa,nicholas ">6&#x27;4</td>
                    <td data-append="1" data-value="266">266</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="OhioState">Ohio State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/ben-garland/" title="Ben Garland">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg" /><!--[if IE 9]></video><![endif]--><img alt="OL #63 Ben Garland" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/d72lgaeq6heecti9t5za.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="garland,benjamin "><a href="/team/players-roster/ben-garland/">Ben Garland</a></span>
                      </div>
                    </td>
                    <td data-append="1">63</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="garland,benjamin ">6-5</td>
                    <td data-append="1" data-value="304">304</td>
                    <td data-append="1" data-value="32">32</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="AirForce">Air Force</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/dee-ford/" title="Dee Ford">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Ford-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/pouqx9mklcas8j1qyyhs.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="ford,dedrick "><a href="/team/players-roster/dee-ford/">Dee Ford</a></span>
                      </div>
                    </td>
                    <td data-append="1">55</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="ford,dedrick ">6-2</td>
                    <td data-append="1" data-value="252">252</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="Auburn">Auburn</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/mark-nzeocha/" title="Mark Nzeocha">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Nzeocha, Mark" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/jpunsmlnaayhctrmznov.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="nzeocha,mark "><a href="/team/players-roster/mark-nzeocha/">Mark Nzeocha</a></span>
                      </div>
                    </td>
                    <td data-append="1">53</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="nzeocha,mark ">6-3</td>
                    <td data-append="1" data-value="235">235</td>
                    <td data-append="1" data-value="30">30</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Wyoming">Wyoming</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/solomon-thomas/" title="Solomon Thomas">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Thomas, Solomon" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/q2yokebtyzmhdlmeg3yr.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="thomas,solomon "><a href="/team/players-roster/solomon-thomas/">Solomon Thomas</a></span>
                      </div>
                    </td>
                    <td data-append="1">94</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="thomas,solomon ">6-3</td>
                    <td data-append="1" data-value="280">280</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="Stanford">Stanford</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jaquiski-tartt/" title="Jaquiski Tartt">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Tartt, Jaquiski" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/f4vdscu8vmq7am8vydfp.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="tartt,jaquiski "><a href="/team/players-roster/jaquiski-tartt/">Jaquiski Tartt</a></span>
                      </div>
                    </td>
                    <td data-append="1">29</td>
                    <td data-append="1">S</td>
                    <td class="sorter-custom-height" data-name="tartt,jaquiski ">6-1</td>
                    <td data-append="1" data-value="215">215</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="6"><span>6</span></td>
                    <td data-append="1" data-value="Samford">Samford</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/george-kittle/" title="George Kittle">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Kittle_George" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ywtxrom9p63pzy0rvwui.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="kittle,george "><a href="/team/players-roster/george-kittle/">George Kittle</a></span>
                      </div>
                    </td>
                    <td data-append="1">85</td>
                    <td data-append="1">TE</td>
                    <td class="sorter-custom-height" data-name="kittle,george ">6-4</td>
                    <td data-append="1" data-value="250">250</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="Iowa">Iowa</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jimmy-garoppolo/" title="Jimmy Garoppolo">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Garoppolo, Jimmy" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/b6jcwijaqzqvj8rrl0ed.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="garoppolo,james "><a href="/team/players-roster/jimmy-garoppolo/">Jimmy Garoppolo</a></span>
                      </div>
                    </td>
                    <td data-append="1">10</td>
                    <td data-append="1">QB</td>
                    <td class="sorter-custom-height" data-name="garoppolo,james ">6-2</td>
                    <td data-append="1" data-value="225">225</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="EasternIllinois">Eastern Illinois</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Reserve/Physically Unable to Perform</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Reserve/Physically Unable to Perform
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/weston-richburg/" title="Weston Richburg">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Richburg, Weston" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/lnzupowq6ngvwstanlk0.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="richburg,weston "><a href="/team/players-roster/weston-richburg/">Weston Richburg</a></span>
                      </div>
                    </td>
                    <td data-append="1">58</td>
                    <td data-append="1">C</td>
                    <td class="sorter-custom-height" data-name="richburg,weston ">6-4</td>
                    <td data-append="1" data-value="295">295</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="ColoradoState">Colorado State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/ronald-blair/" title="Ronald Blair III">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Blair III, Ronald" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/qfnfkic8ftrpmvetevmg.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="blair,ronald iii"><a href="/team/players-roster/ronald-blair/">Ronald Blair III</a></span>
                      </div>
                    </td>
                    <td data-append="1">98</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="blair,ronald iii">6-4</td>
                    <td data-append="1" data-value="270">270</td>
                    <td data-append="1" data-value="27">27</td>
                    <td data-append="1" data-value="5"><span>5</span></td>
                    <td data-append="1" data-value="AppalachianState">Appalachian State</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
        <div class="d3-l-col__col-12">
          <div class="d3-o-adv-block">
            <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['04baf269-41a4-4d28-96bb-7909ecb6ef99'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, '04baf269-41a4-4d28-96bb-7909ecb6ef99')
                  .setTargeting('slot', 'roster_3')
                  .defineSizeMapping(gptconfig.responsiveMappings['leaderboard'])
.addService(googletag.pubads())
              );});
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['04baf269-41a4-4d28-96bb-7909ecb6ef99'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('04baf269-41a4-4d28-96bb-7909ecb6ef99')
            });
          });
        }
      </script><div class="  d3-o-adv-block" data-id="04baf269-41a4-4d28-96bb-7909ecb6ef99" id="04baf269-41a4-4d28-96bb-7909ecb6ef99"></div><script></script>
          </div>
        </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Practice Squad</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Practice Squad
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/shawn-poindexter/" title="Shawn Poindexter">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Poindexter" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xn5iypmpjwd0gqjclvtp.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="poindexter,shawn "><a href="/team/players-roster/shawn-poindexter/">Shawn Poindexter</a></span>
                      </div>
                    </td>
                    <td data-append="1">1</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="poindexter,shawn ">6-5</td>
                    <td data-append="1" data-value="213">213</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Arizona">Arizona</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kai-nacua/" title="Kai Nacua">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg" /><!--[if IE 9]></video><![endif]--><img alt="112420-nacua-FB" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/o09yg6agnp1bzidzkqzr.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="nacua,kaimana "><a href="/team/players-roster/kai-nacua/">Kai Nacua</a></span>
                      </div>
                    </td>
                    <td data-append="1">48</td>
                    <td data-append="1">S</td>
                    <td class="sorter-custom-height" data-name="nacua,kaimana ">6-0</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="BYU">BYU</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/josh-johnson/" title="Josh Johnson">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Josh-Johnson-HS" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/xq3cvj3yplpemd2ladhu.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="johnson,joshua "><a href="/team/players-roster/josh-johnson/">Josh Johnson</a></span>
                      </div>
                    </td>
                    <td data-append="1">2</td>
                    <td data-append="1">QB</td>
                    <td class="sorter-custom-height" data-name="johnson,joshua ">6-3</td>
                    <td data-append="1" data-value="219">219</td>
                    <td data-append="1" data-value="34">34</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="SanDiego">San Diego</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/tony-bergstrom/" title="Tony Bergstrom">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Bergstrom-HS" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/u0s8zx6ejaw7yzxczw3g.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="bergstrom,steven "><a href="/team/players-roster/tony-bergstrom/">Tony Bergstrom</a></span>
                      </div>
                    </td>
                    <td data-append="1">62</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="bergstrom,steven ">6-5</td>
                    <td data-append="1" data-value="310">310</td>
                    <td data-append="1" data-value="34">34</td>
                    <td data-append="1" data-value="9"><span>9</span></td>
                    <td data-append="1" data-value="Utah">Utah</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/parnell-motley/" title="Parnell Motley">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg" /><!--[if IE 9]></video><![endif]--><img alt="MotleyParnell" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/iirxcxpt9nl0qc5d78ul.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="motley,parnell "><a href="/team/players-roster/parnell-motley/">Parnell Motley</a></span>
                      </div>
                    </td>
                    <td data-append="1">49</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="motley,parnell ">6-0</td>
                    <td data-append="1" data-value="180">180</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Oklahoma">Oklahoma</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/josiah-coatney/" title="Josiah Coatney">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Coatney-HS" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oq3trrryuqy5choayw4y.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="coatney,josiah "><a href="/team/players-roster/josiah-coatney/">Josiah Coatney</a></span>
                      </div>
                    </td>
                    <td data-append="1">76</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="coatney,josiah ">6-4</td>
                    <td data-append="1" data-value="325">325</td>
                    <td data-append="1" data-value="24">24</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Mississippi">Mississippi</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/dakoda-shepley/" title="Dakoda Shepley">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg" /><!--[if IE 9]></video><![endif]--><img alt="_00T7443" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/vna7otqczu9qrvr4grle.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="shepley,dakoda "><a href="/team/players-roster/dakoda-shepley/">Dakoda Shepley</a></span>
                      </div>
                    </td>
                    <td data-append="1">61</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="shepley,dakoda ">6-5</td>
                    <td data-append="1" data-value="290">290</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="BritishColumbia(Canada)">British Columbia (Canada)</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/kevin-white/" title="Kevin White">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg" /><!--[if IE 9]></video><![endif]--><img alt="White-HS" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/zj0bscq7cfghcapvps3j.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="white,kevin "><a href="/team/players-roster/kevin-white/">Kevin White</a></span>
                      </div>
                    </td>
                    <td data-append="1">14</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="white,kevin ">6-3</td>
                    <td data-append="1" data-value="216">216</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="5"><span>5</span></td>
                    <td data-append="1" data-value="WestVirginia">West Virginia</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jonas-griffith/" title="Jonas Griffith">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Griffith_Jonas" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/uei4zme4otmzv9fopsrh.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="griffith,jonas "><a href="/team/players-roster/jonas-griffith/">Jonas Griffith</a></span>
                      </div>
                    </td>
                    <td data-append="1">5</td>
                    <td data-append="1">LB</td>
                    <td class="sorter-custom-height" data-name="griffith,jonas ">6-4</td>
                    <td data-append="1" data-value="250">250</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="IndianaState">Indiana State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/josh-hokit/" title="Josh Hokit">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Hokit_Josh" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mmuzfk48rra7wciw9qkk.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="hokit,josh "><a href="/team/players-roster/josh-hokit/">Josh Hokit</a></span>
                      </div>
                    </td>
                    <td data-append="1">40</td>
                    <td data-append="1">FB</td>
                    <td class="sorter-custom-height" data-name="hokit,josh ">6-1</td>
                    <td data-append="1" data-value="225">225</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="FresnoState">Fresno State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/darrion-daniels/" title="Darrion Daniels">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Daniels_Darrion" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/mlrhrfwexezqnxqb0xxh.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="daniels,darrion "><a href="/team/players-roster/darrion-daniels/">Darrion Daniels</a></span>
                      </div>
                    </td>
                    <td data-append="1">65</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="daniels,darrion ">6-3</td>
                    <td data-append="1" data-value="311">311</td>
                    <td data-append="1" data-value="22">22</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Nebraska">Nebraska</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/willie-henry-jr/" title="Willie Henry Jr.&#xA0;">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Henry" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/m7ozvlgpdvsfbbovy5qi.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="henry,willie "><a href="/team/players-roster/willie-henry-jr/">Willie Henry Jr.&#xA0;</a></span>
                      </div>
                    </td>
                    <td data-append="1">61</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="henry,willie ">6-3</td>
                    <td data-append="1" data-value="288">288</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="Michigan">Michigan</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/alex-barrett/" title="Alex Barrett">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Barrett_Alex" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/oxkert0htgntg6vyg7nb.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="barrett,alejandro "><a href="/team/players-roster/alex-barrett/">Alex Barrett</a></span>
                      </div>
                    </td>
                    <td data-append="1">64</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="barrett,alejandro ">6-2</td>
                    <td data-append="1" data-value="250">250</td>
                    <td data-append="1" data-value="26">26</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="SanDiegoState">San Diego State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/tim-harris/" title="Tim Harris">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Harris" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/hpfhlv4nsktwqhmhhpvf.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="harris,tim "><a href="/team/players-roster/tim-harris/">Tim Harris</a></span>
                      </div>
                    </td>
                    <td data-append="1">35</td>
                    <td data-append="1">CB</td>
                    <td class="sorter-custom-height" data-name="harris,tim ">6-1</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="2"><span>2</span></td>
                    <td data-append="1" data-value="Virginia">Virginia</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jordan-matthews/" title="Jordan Matthews">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Matthews-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/s0jrguxohoto7c59rt7q.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="matthews,jordan "><a href="/team/players-roster/jordan-matthews/">Jordan Matthews</a></span>
                      </div>
                    </td>
                    <td data-append="1">18</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="matthews,jordan ">6-3</td>
                    <td data-append="1" data-value="215">215</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="7"><span>7</span></td>
                    <td data-append="1" data-value="Vanderbilt">Vanderbilt</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Practice Squad/Injured</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Practice Squad/Injured
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jared-mayden/" title="Jared Mayden">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Mayden_Jared" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/l5solfqc9hfsipzjyuay.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="mayden,jared "><a href="/team/players-roster/jared-mayden/">Jared Mayden</a></span>
                      </div>
                    </td>
                    <td data-append="1">43</td>
                    <td data-append="1">S</td>
                    <td class="sorter-custom-height" data-name="mayden,jared ">6-0</td>
                    <td data-append="1" data-value="205">205</td>
                    <td data-append="1" data-value="22">22</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Alabama">Alabama</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jauan-jennings/" title="Jauan Jennings">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Jennings_Jauan" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/tgozcmqntav1syfuzxdz.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="jennings,bennie "><a href="/team/players-roster/jauan-jennings/">Jauan Jennings</a></span>
                      </div>
                    </td>
                    <td data-append="1">17</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="jennings,bennie ">6-3</td>
                    <td data-append="1" data-value="212">212</td>
                    <td data-append="1" data-value="23">23</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="Tennessee">Tennessee</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
        <div class="d3-l-col__col-12">
          <div class="d3-o-adv-block">
            <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['5d5c7654-c98f-403f-b97e-45513fc8265a'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, '5d5c7654-c98f-403f-b97e-45513fc8265a')
                  .setTargeting('slot', 'roster_5')
                  .defineSizeMapping(gptconfig.responsiveMappings['leaderboard'])
.addService(googletag.pubads())
              );});
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['5d5c7654-c98f-403f-b97e-45513fc8265a'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('5d5c7654-c98f-403f-b97e-45513fc8265a')
            });
          });
        }
      </script><div class="  d3-o-adv-block" data-id="5d5c7654-c98f-403f-b97e-45513fc8265a" id="5d5c7654-c98f-403f-b97e-45513fc8265a"></div><script></script>
          </div>
        </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Reserve/COVID-19</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Reserve/COVID-19
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jordan-willis/" title="Jordan Willis">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Jordan-Willis" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/h1xcwyquqlsmw8lzxzwt.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="willis,jordan "><a href="/team/players-roster/jordan-willis/">Jordan Willis</a></span>
                      </div>
                    </td>
                    <td data-append="1">78</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="willis,jordan ">6-4</td>
                    <td data-append="1" data-value="270">270</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="KansasState">Kansas State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/brandon-aiyuk/" title="Brandon Aiyuk">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Aiyuk_Brandon" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/cxq1onmiqg1jfnsm2xji.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="aiyuk,brandon "><a href="/team/players-roster/brandon-aiyuk/">Brandon Aiyuk</a></span>
                      </div>
                    </td>
                    <td data-append="1">11</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="aiyuk,brandon ">6-0</td>
                    <td data-append="1" data-value="200">200</td>
                    <td data-append="1" data-value="22">22</td>
                    <td data-append="1" data-value="0"><span>R</span></td>
                    <td data-append="1" data-value="ArizonaState">Arizona State</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/d-j-jones/" title="D.J. Jones">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Jones, D.J." class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gxkyzmo0lasmf1hfocta.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="jones,david "><a href="/team/players-roster/d-j-jones/">D.J. Jones</a></span>
                      </div>
                    </td>
                    <td data-append="1">93</td>
                    <td data-append="1">DL</td>
                    <td class="sorter-custom-height" data-name="jones,david ">6-0</td>
                    <td data-append="1" data-value="305">305</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="4"><span>4</span></td>
                    <td data-append="1" data-value="Mississippi">Mississippi</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Reserve/Opt-Out</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Reserve/Opt-Out
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/travis-benjamin/" title="Travis Benjamin">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Benjamin-HS" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/gqpl74lagj6kopeflhru.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="benjamin,travis "><a href="/team/players-roster/travis-benjamin/">Travis Benjamin</a></span>
                      </div>
                    </td>
                    <td data-append="1">17</td>
                    <td data-append="1">WR</td>
                    <td class="sorter-custom-height" data-name="benjamin,travis ">5-10</td>
                    <td data-append="1" data-value="175">175</td>
                    <td data-append="1" data-value="30">30</td>
                    <td data-append="1" data-value="9"><span>9</span></td>
                    <td data-append="1" data-value="Miami(Fla.)">Miami (Fla.)</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/jake-brendel/" title="Jake Brendel">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Brendel_Jake" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/ss9dlewmzlqxig66xr9n.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="brendel,jacob "><a href="/team/players-roster/jake-brendel/">Jake Brendel</a></span>
                      </div>
                    </td>
                    <td data-append="1">64</td>
                    <td data-append="1">OL</td>
                    <td class="sorter-custom-height" data-name="brendel,jacob ">6-4</td>
                    <td data-append="1" data-value="299">299</td>
                    <td data-append="1" data-value="28">28</td>
                    <td data-append="1" data-value="3"><span>3</span></td>
                    <td data-append="1" data-value="UCLA">UCLA</td>
                  </tr>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/shon-coleman/" title="Shon Coleman">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Coleman-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/fy1zsnwklofrpllhmsaa.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="coleman,shalon "><a href="/team/players-roster/shon-coleman/">Shon Coleman</a></span>
                      </div>
                    </td>
                    <td data-append="1">78</td>
                    <td data-append="1">T</td>
                    <td class="sorter-custom-height" data-name="coleman,shalon ">6-5</td>
                    <td data-append="1" data-value="310">310</td>
                    <td data-append="1" data-value="29">29</td>
                    <td data-append="1" data-value="5"><span>5</span></td>
                    <td data-append="1" data-value="Auburn">Auburn</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
        <div class="d3-l-col__col-12">
          <div class="d3-o-adv-block">
            <script type="text/javascript">
            var googletag = window.googletag || {};
            googletag.cmd = window.googletag.cmd || [];
            var slot = slot || {}; //some modules call this more than once so we check if slot is already set.
            googletag.cmd.push(function () {
              gptSlots.push(
                slot['1d01eba6-58f5-4f85-8645-d67a06ad1772'] = googletag
                  .defineSlot(adUnit, gptconfig.adSizes, '1d01eba6-58f5-4f85-8645-d67a06ad1772')
                  .setTargeting('slot', 'roster_7')
                  .defineSizeMapping(gptconfig.responsiveMappings['leaderboard'])
.addService(googletag.pubads())
              );});
        if (typeof(__iasPET) !== 'undefined') {
          googletag.cmd.push(function() {
            var newSlot = window.slot['1d01eba6-58f5-4f85-8645-d67a06ad1772'];
            if(typeof(newSlot) === 'undefined'){
              return;
            }

            var iasPETSlots = [];
            var newSlotSizes = newSlot.getSizes().map(function(size) {
              if (size.getWidth && size.getHeight)
                return [size.getWidth(), size.getHeight()];
              else
                return [1, 1];
            });
 
            iasPETSlots.push({
              adSlotId: newSlot.getSlotElementId(),
              size: newSlotSizes,
              adUnitPath: newSlot.getAdUnitPath()
            });
 
            __iasPET.queue.push({
              adSlots: iasPETSlots,
              dataHandler: iasSingleSlotDataHandler('1d01eba6-58f5-4f85-8645-d67a06ad1772')
            });
          });
        }
      </script><div class="  d3-o-adv-block" data-id="1d01eba6-58f5-4f85-8645-d67a06ad1772" id="1d01eba6-58f5-4f85-8645-d67a06ad1772"></div><script></script>
          </div>
        </div>
      <div class="d3-l-col__col-12">
        <div class="nfl-o-roster">
          <h4 class="nfl-o-roster__title"><span class="nfl-o-roster__title-status">Practice Squad/COVID-19</span><span class="nfl-o-roster__title-advertiser"></span></h4>
          <div class="d3-o-table--horizontal-scroll">
            <table summary="Roster" class="d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-table--sortable {sortlist: [[0,0]]}" data-require="modules/tableSortable">
              <caption hidden class="d3-o-table__caption">
                Practice Squad/COVID-19
              </caption>
              <thead>
                <tr>
                  <th>Player</th>
                  <th class="{sorter: 'append'}">#</th>
                  <th class="{sorter: 'append'}">Pos</th>
                  <th>HT</th>
                  <th class="{sorter: 'append'}">WT</th>
                  <th class="{sorter: 'append'}">Age</th>
                  <th class="{sorter: 'append'}">Exp</th>
                  <th class="{sorter: 'append'}">College</th>
                </tr>
              </thead>
              <tbody>
                  <tr>
                    <td class="sorter-lastname">
                      <div class="d3-o-media-object">
                        <figure class="d3-o-media-object__figure">
                          <a href="/team/players-roster/daniel-helm/" title="Daniel Helm">
                            <picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg" /><source srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg" /><!--[if IE 9]></video><![endif]--><img alt="Helm-Headshot" class="img-responsive" src="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/49ers/csz4x37rnp0wtpqeqb0x.jpg" /></picture>
                          </a>
                        </figure>
                        <span class="nfl-o-roster__player-name" data-name="helm,daniel "><a href="/team/players-roster/daniel-helm/">Daniel Helm</a></span>
                      </div>
                    </td>
                    <td data-append="1">88</td>
                    <td data-append="1">TE</td>
                    <td class="sorter-custom-height" data-name="helm,daniel ">6-4</td>
                    <td data-append="1" data-value="255">255</td>
                    <td data-append="1" data-value="25">25</td>
                    <td data-append="1" data-value="1"><span>1</span></td>
                    <td data-append="1" data-value="Duke">Duke</td>
                  </tr>
              </tbody>
            </table>
          </div>
        </div>
      </div>
  </div>
  



<span style="display: none">
    <svg class="nfl-o-icon-included--down" data-viewbox="0 0 24 24">
      <use xlink:href='#down'></use>
    </svg>
</span>

</section>



  <section class="d3-l-grid--outer d3-l-section-row">
    <div class="d3-l-grid--inner ">
      



                <div class="d3-l-col__col-12">
            



  <div class="nfl-c-custom-promo nfl-c-custom-promo--has-color-schema-all-black-on-white nfl-c-custom-promo--has-image-position-top nfl-c-custom-promo--has-desktop-size-100 nfl-c-custom-promo--has-mobile-size-100 nfl-c-custom-promo--has-text-align-left nfl-c-custom-promo--has-background-size-default nfl-c-custom-promo--has-padding nfl-c-custom-promo--has-cta-size-small nfl-c-custom-promo--has-cta-color-schema-button-color nfl-c-custom-promo--no-section-title">
    


    

    <div class="d3-o-media-object d3-o-media-object--horizontal d3-o-media-object--vertical-center nfl-c-custom-promo__content" >

        <a class="nfl-c-custom-promo__cta-whole-active"
          data-link_module="Custom Promo"
          data-link_type="2020-04-24-season-tickets-team"
          data-link_name="2020-04-24-season-tickets-team 2020-04-24 Season Tickets Team"
          data-link_url="https://www.49ers.com/tickets/season-tickets?source=Roster_20200424"
          href="https://www.49ers.com/tickets/season-tickets?source=Roster_20200424"
          target="_self">
        </a>

        <figure class="d3-o-media-object__figure nfl-c-custom-promo__figure">
          <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/f_auto/49ers/ct5cxrlfdy0qicldq0dj" /><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/f_auto/49ers/ct5cxrlfdy0qicldq0dj" /><source srcset="https://static.clubs.nfl.com/image/private/f_auto/49ers/ct5cxrlfdy0qicldq0dj" /><!--[if IE 9]></video><![endif]--><img alt="20Home_Opponent_Header_940x116" class="img-responsive" src="https://static.clubs.nfl.com/image/private/f_auto/49ers/ct5cxrlfdy0qicldq0dj" /></picture>
        </figure>
          </div>
  
  </div>

          </div>
    </div>
  </section>


    
        <section class="d3-l-grid--outer d3-l-adv-row">
          <h2 class="d3-o-section-title"> Advertising</h2>
          <div class="d3-l-grid--inner">
            <script></script><div class="  d3-o-adv-block" data-id="adv_club" id="adv_club"></div><script></script>
          </div>
        </section>
  </main>




<footer class="d3-o-footer" role="contentinfo" data-require="modules/footer" >
  
    <ul class="d3-o-footer__nav" role="tablist">
      <li>
        <button id="d3-o-footer__tab-trigger-1" class="d3-o-footer__tab-trigger" href="#d3-o-footer__tab-panel-1" role="tab" aria-controls="d3-o-footer__tab-panel-1" aria-selected="true">
          <span>Club Links</span>
          



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--down" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#down'></use>
  </svg>
</span>

        </button>
        <div id="d3-o-footer__tab-panel-1" class="d3-o-footer__tab-panel d3-t-footer__panel-1" aria-hidden="false" aria-labelledby="d3-o-footer__tab-panel-1" role="tabpanel">
          <nav role="navigation">
            <h2 class="d3-o-nav__title">Club links nav</h2>
            



        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
CONNECT WITH US          </li>

              <li>

    <a href="https://www.49ers.com/about-us/contact-us"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Contact Us"
       data-link_url="/about-us/contact-us"
       data-link_position="1:4">
Contact Us    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/employment"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Employment"
       data-link_url="/employment"
       data-link_position="2:4">
Employment    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/fans/email-newsletter"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Email Newsletter"
       data-link_url="/fans/email-newsletter"
       data-link_position="3:4">
Email Newsletter    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/about-us/terms-and-conditions"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Terms &amp; Conditions"
       data-link_url="/about-us/terms-and-conditions"
       data-link_position="4:4">
Terms &amp; Conditions    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
NEWS          </li>

              <li>

    <a href="https://www.49ers.com/news"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Articles"
       data-link_url="/news"
       data-link_position="1:5">
Articles    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/video"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Videos"
       data-link_url="/video"
       data-link_position="2:5">
Videos    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/photos"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Photos"
       data-link_url="/photos"
       data-link_position="3:5">
Photos    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/news/trainingcamp"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Training Camp"
       data-link_url="https://www.49ers.com/news/trainingcamp"
       data-link_position="4:5">
Training Camp    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/sitemap/0"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Site Map"
       data-link_url="https://www.49ers.com/sitemap/0"
       data-link_position="5:5">
Site Map    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
LEVI&#x27;S&#xAE; STADIUM          </li>

              <li>

    <a href="http://www.levisstadium.com/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Stadium Website"
       data-link_url="http://www.levisstadium.com"
       data-link_position="1:6">
Stadium Website    </a>
              </li>
              <li>

    <a href="http://www.levisstadium.com/bagpolicy/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Bag Policy"
       data-link_url="http://www.levisstadium.com/bagpolicy/"
       data-link_position="2:6">
Bag Policy    </a>
              </li>
              <li>

    <a href="http://www.levisstadium.com/stadium-az-guide/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="A-Z Stadium Guide"
       data-link_url="http://www.levisstadium.com/stadium-az-guide/"
       data-link_position="3:6">
A-Z Stadium Guide    </a>
              </li>
              <li>

    <a href="http://www.levisstadium.com/guest-services/ada-parking/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Accessible Services"
       data-link_url="http://www.levisstadium.com/guest-services/ada-parking/"
       data-link_position="4:6">
Accessible Services    </a>
              </li>
              <li>

    <a href="http://www.levisstadium.com/plan-your-visit/parking/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Directions and Parking"
       data-link_url="http://www.levisstadium.com/plan-your-visit/parking/"
       data-link_position="5:6">
Directions and Parking    </a>
              </li>
              <li>

    <a href="http://www.levisstadium.com/security-policies/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Security Policies"
       data-link_url="http://www.levisstadium.com/security-policies/"
       data-link_position="6:6">
Security Policies    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">

    <a href="https://www.49ers.com/tickets/suites/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Suites"
       data-link_url="/tickets/suites/"
       data-link_position="1:3">
Suites    </a>
          </li>

              <li>

    <a href="https://www.49ers.com/tickets/suites/season-suites"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Full Season Suite Leases"
       data-link_url="/tickets/suites/season-suites"
       data-link_position="1:3">
Full Season Suite Leases    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/suites/single-game-suites"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Single Game Suite Rentals"
       data-link_url="/tickets/suites/single-game-suites"
       data-link_position="2:3">
Single Game Suite Rentals    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/suites/levis-stadium-events"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Levi&#x27;s&#xAE; Stadium Event Suite Interest"
       data-link_url="/tickets/suites/levis-stadium-events"
       data-link_position="3:3">
Levi&#x27;s&#xAE; Stadium Event Suite Interest    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">

    <a href="https://www.49ers.com/tickets/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="TICKETS"
       data-link_url="/tickets/"
       data-link_position="2:7">
TICKETS    </a>
          </li>

              <li>

    <a href="https://oss.ticketmaster.com/aps/49ersnew/EN/account/login"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Account Manager"
       data-link_url="https://oss.ticketmaster.com/aps/49ersnew/EN/account/login"
       data-link_position="1:7">
Account Manager    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/season-tickets"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Season Tickets"
       data-link_url="/tickets/season-tickets"
       data-link_position="2:7">
Season Tickets    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/ownersseats"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Citrix Owners Seats"
       data-link_url="/tickets/ownersseats"
       data-link_position="3:7">
Citrix Owners Seats    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/single-game"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Single Game Tickets"
       data-link_url="/tickets/single-game"
       data-link_position="4:7">
Single Game Tickets    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/group-tickets"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Group Tickets"
       data-link_url="/tickets/group-tickets"
       data-link_position="5:7">
Group Tickets    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/vivid-seats"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Fan Travel Packages"
       data-link_url="/tickets/vivid-seats"
       data-link_position="6:7">
Fan Travel Packages    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/tickets/policies"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="2020 Ticket Policies"
       data-link_url="/tickets/policies"
       data-link_position="7:7">
2020 Ticket Policies    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">

    <a href="https://www.49ers.com/team/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="TEAM INFO"
       data-link_url="/team/"
       data-link_position="3:5">
TEAM INFO    </a>
          </li>

              <li>

    <a href="https://www.49ers.com/team/players-roster/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Roster"
       data-link_url="/team/players-roster/"
       data-link_position="1:5">
Roster    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/schedule/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Schedule"
       data-link_url="/schedule/"
       data-link_position="2:5">
Schedule    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/team/depth-chart"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Depth Chart"
       data-link_url="/team/depth-chart"
       data-link_position="3:5">
Depth Chart    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/team/coaches-roster/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Coaches"
       data-link_url="/team/coaches-roster/"
       data-link_position="4:5">
Coaches    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/team/stats/"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_club-links"
       data-link_name="Stats"
       data-link_url="/team/stats/"
       data-link_position="5:5">
Stats    </a>
              </li>
        </ul>

          </nav>
        </div>
      </li>
      <li>
        <button id="d3-o-footer__tab-trigger-2" class="d3-o-footer__tab-trigger" href="#d3-o-footer__tab-panel-2" role="tab" aria-controls="d3-o-footer__tab-trigger-2" aria-selected="false">
          <span>NFL Clubs</span>
          



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--down" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#down'></use>
  </svg>
</span>

        </button>
        <div id="d3-o-footer__tab-panel-2" class="d3-o-footer__tab-panel d3-t-footer__panel-2" aria-hidden="true" aria-labelledby="d3-o-footer__tab-panel-2" style="display: none" role="tabpanel">
          <nav role="navigation">
            <h2 class="d3-o-nav__title">NFL Clubs links nav</h2>
            



        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
AFC North          </li>

              <li>

    <a href="https://www.baltimoreravens.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Baltimore Ravens"
       data-link_url="https://www.baltimoreravens.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/BAL" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/BAL" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/BAL" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.bengals.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Cincinnati Bengals"
       data-link_url="https://www.bengals.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CIN" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CIN" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CIN" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.clevelandbrowns.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Cleveland Browns"
       data-link_url="https://www.clevelandbrowns.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CLE" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CLE" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CLE" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.steelers.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Pittsburgh Steelers"
       data-link_url="https://www.steelers.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/PIT" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/PIT" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/PIT" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
AFC East          </li>

              <li>

    <a href="https://www.buffalobills.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Buffalo Bills"
       data-link_url="https://www.buffalobills.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/BUF" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/BUF" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/BUF" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.miamidolphins.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Miami Dolphins"
       data-link_url="https://www.miamidolphins.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/MIA" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/MIA" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/MIA" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.patriots.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="New England Patriots"
       data-link_url="https://www.patriots.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NE" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NE" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NE" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.newyorkjets.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="New York Jets"
       data-link_url="https://www.newyorkjets.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NYJ" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NYJ" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NYJ" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
AFC South          </li>

              <li>

    <a href="https://www.houstontexans.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Houston Texans"
       data-link_url="https://www.houstontexans.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/HOU" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/HOU" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/HOU" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.colts.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Indianapolis Colts"
       data-link_url="https://www.colts.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/IND" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/IND" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/IND" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.jaguars.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Jacksonville Jaguars"
       data-link_url="https://www.jaguars.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/JAX" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/JAX" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/JAX" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.tennesseetitans.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Tennessee Titans"
       data-link_url="https://www.tennesseetitans.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/TEN" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/TEN" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/TEN" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
AFC West          </li>

              <li>

    <a href="https://www.denverbroncos.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Denver Broncos"
       data-link_url="https://www.denverbroncos.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DEN" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DEN" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DEN" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.chiefs.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Kansas City Chiefs"
       data-link_url="https://www.chiefs.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/KC" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/KC" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/KC" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.chargers.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Los Angeles Chargers"
       data-link_url="https://www.chargers.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/LAC" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/LAC" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/LAC" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.raiders.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Oakland Raiders"
       data-link_url="https://www.raiders.com/"
       data-link_position="4:4">
<picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.nfl.com/static/content/public/static/wildcat/assets/img/logos/teams/OAK.svg" media="(min-width:1024px)" /><source data-srcset="https://static.nfl.com/static/content/public/static/wildcat/assets/img/logos/teams/OAK.svg" media="(min-width:768px)" /><source data-srcset="https://static.nfl.com/static/content/public/static/wildcat/assets/img/logos/teams/OAK.svg" /><!--[if IE 9]></video><![endif]--><img alt="Oakland Raiders" class="d3-o-footer__panel-icon img-responsive" data-src="https://static.nfl.com/static/content/public/static/wildcat/assets/img/logos/teams/OAK.svg" src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==" /></picture>    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
NFC North          </li>

              <li>

    <a href="https://www.chicagobears.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Chicago Bears"
       data-link_url="https://www.chicagobears.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CHI" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CHI" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CHI" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.detroitlions.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Detroit Lions"
       data-link_url="https://www.detroitlions.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DET" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DET" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DET" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.packers.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Green Bay Packers"
       data-link_url="https://www.packers.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/GB" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/GB" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/GB" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.vikings.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Minnesota Vikings"
       data-link_url="https://www.vikings.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/MIN" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/MIN" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/MIN" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
NFC East          </li>

              <li>

    <a href="https://www.dallascowboys.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Dallas Cowboys"
       data-link_url="https://www.dallascowboys.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DAL" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DAL" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/DAL" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.giants.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="New York Giants"
       data-link_url="https://www.giants.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NYG" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NYG" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NYG" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.philadelphiaeagles.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Philadelphia Eagles"
       data-link_url="https://www.philadelphiaeagles.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/PHI" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/PHI" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/PHI" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.washingtonfootball.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Washington Redskins"
       data-link_url="https://www.washingtonfootball.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/WAS" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/WAS" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/WAS" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
NFC South          </li>

              <li>

    <a href="https://www.atlantafalcons.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Atlanta Falcons"
       data-link_url="https://www.atlantafalcons.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/ATL" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/ATL" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/ATL" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.panthers.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Carolina Panthers"
       data-link_url="https://www.panthers.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CAR" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CAR" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/CAR" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.neworleanssaints.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="New Orleans Saints"
       data-link_url="https://www.neworleanssaints.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NO" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NO" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/NO" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.buccaneers.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Tampa Bay Buccaneers"
       data-link_url="https://www.buccaneers.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/TB" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/TB" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/TB" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>
        <ul class="d3-o-footer__panel-links">
          <li class="d3-o-footer__panel-link-category">
NFC West          </li>

              <li>

    <a href="https://www.azcardinals.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Arizona Cardinals"
       data-link_url="https://www.azcardinals.com/"
       data-link_position="1:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/ARI" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/ARI" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/ARI" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.therams.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Los Angeles Rams"
       data-link_url="https://www.therams.com/"
       data-link_position="2:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/LA" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/LA" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/LA" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.49ers.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="San Francisco 49ers"
       data-link_url="https://www.49ers.com/"
       data-link_position="3:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
              <li>

    <a href="https://www.seahawks.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_nfl_clubs"
       data-link_name="Seattle Seahawks"
       data-link_url="https://www.seahawks.com/"
       data-link_position="4:4">


      <span class="d3-o-footer__panel-icon">
        <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SEA" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SEA" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SEA" /><!--[if IE 9]></video><![endif]--></picture>
      </span>
    </a>
              </li>
        </ul>

          </nav>
        </div>
      </li>
      <li>
        <button id="d3-o-footer__tab-trigger-3" class="d3-o-footer__tab-trigger" href="#d3-o-footer__tab-panel-3" role="tab" aria-controls="d3-o-footer__tab-panel-3" aria-selected="false">
          <span>More NFL Sites</span>
          



<span class="nfl-o-icon nfl-o-icon--small ">
  <svg class="nfl-o-icon--down" viewBox="0 0 24 24"  aria-hidden="true" >
    <use xlink:href='#down'></use>
  </svg>
</span>

        </button>
        <div id="d3-o-footer__tab-panel-3" class="d3-o-footer__tab-panel d3-t-footer__panel-3" aria-hidden="true" aria-labelledby="d3-o-footer__tab-panel-3" style="display: none" role="tabpanel">
          <nav role="navigation">
            <h2 class="d3-o-nav__title">More NFL sites nav</h2>
            



    <ul class="d3-o-footer__panel-links">
        <li>

    <a href="https://operations.nfl.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-1st-column"
       data-link_name="NFL Football Operations"
       data-link_url="https://operations.nfl.com/"
       data-link_position="1:5">
NFL Football Operations    </a>
        </li>
        <li>

    <a href="https://www.nflshop.com/source/bm-nflcom-2017-BottomBanner-shopgeneric"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-1st-column"
       data-link_name="NFL Shop"
       data-link_url="https://www.nflshop.com/source/bm-nflcom-2017-BottomBanner-shopgeneric"
       data-link_position="2:5">
NFL Shop    </a>
        </li>
        <li>

    <a href="https://nflonlocation.com/?utm_campaign=OLE&amp;utm_medium=referral&amp;utm_source=nfl&amp;utm_content=footerlink"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-1st-column"
       data-link_name="NFL On Location"
       data-link_url="https://nflonlocation.com/?utm_campaign=OLE&amp;utm_medium=referral&amp;utm_source=nfl&amp;utm_content=footerlink"
       data-link_position="3:5">
NFL On Location    </a>
        </li>
        <li>

    <a href="https://www.profootballhof.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-1st-column"
       data-link_name="Pro Football Hall of Fame"
       data-link_url="https://www.profootballhof.com/"
       data-link_position="4:5">
Pro Football Hall of Fame    </a>
        </li>
        <li>

    <a href="https://www.nfl.info/NFLConsProd/Welcome/index.html"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-1st-column"
       data-link_name="Licensing"
       data-link_url="https://www.nfl.info/NFLConsProd/Welcome/index.html"
       data-link_position="5:5">
Licensing    </a>
        </li>
    </ul>




    <ul class="d3-o-footer__panel-links">
        <li>

    <a href="https://www.usafootball.com/"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-2nd-column"
       data-link_name="USA Football"
       data-link_url="https://www.usafootball.com/"
       data-link_position="1:3">
USA Football    </a>
        </li>
        <li>

    <a href="https://www.nflextrapoints.com/nfl-extra-points-credit-card/?campaignId=2038&amp;shopperid=&amp;cellNumber=706&amp;referrerid=NFLB070911&amp;referrerid=NFLB070911&amp;publisherid=&amp;advertisementid=#national-football-league"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-2nd-column"
       data-link_name="NFL Extra Points"
       data-link_url="https://www.nflextrapoints.com/nfl-extra-points-credit-card/?campaignId=2038&amp;shopperid=&amp;cellNumber=706&amp;referrerid=NFLB070911&amp;referrerid=NFLB070911&amp;publisherid=&amp;advertisementid=#national-football-league"
       data-link_position="2:3">
NFL Extra Points    </a>
        </li>
        <li>

    <a href="https://www.ticketmaster.com/nfl?wt.mc_id=NFL_LEAGUE_FOOTER_LINK&amp;utm_source=NFL.com&amp;utm_medium=client&amp;utm_campaign=NFL_LEAGUE&amp;utm_content=FOOTER_LINK"
       target="_blank"
       data-link_module="Footer"
       data-event_name="click action"
       data-link_type="_more-nfl-sites-2nd-column"
       data-link_name="NFL Ticket Exchange"
       data-link_url="https://www.ticketmaster.com/nfl?wt.mc_id=NFL_LEAGUE_FOOTER_LINK&amp;utm_source=NFL.com&amp;utm_medium=client&amp;utm_campaign=NFL_LEAGUE&amp;utm_content=FOOTER_LINK"
       data-link_position="3:3">
NFL Ticket Exchange    </a>
        </li>
    </ul>

          </nav>
        </div>
      </li>
    </ul>
  


<div class="d3-o-footer__apps">
    <span class="d3-o-footer__apps-label">
      Download Official Team Mobile App
    </span>
  <nav role="navigation">
    <h2 class="d3-o-nav__title">Download Apps nav</h2>
    <ul class="d3-o-footer__apps-links">
      <li>
        <a href="https://itunes.apple.com/app/id395859078?mt=8" target="_blank" aria-label="Download on the Apple Store - Opens new window">


          <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://www.49ers.com/compiledassets/assets/icons/app_store_badge.svg" media="(min-width:1024px)" /><source data-srcset="https://www.49ers.com/compiledassets/assets/icons/app_store_badge.svg" media="(min-width:768px)" /><source data-srcset="https://www.49ers.com/compiledassets/assets/icons/app_store_badge.svg" /><!--[if IE 9]></video><![endif]--><img alt="Download on the Apple Store" class="d3-o-footer__apps-links-icon-apple img-responsive" data-src="https://www.49ers.com/compiledassets/assets/icons/app_store_badge.svg" src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==" /></picture>
        </a>
      </li>
      <li>
        <a id="league-play-store-app" href="https://play.google.com/store/apps/details?id=com.yinzcam.nfl.sf&amp;hl=en" target="_blank" aria-label="Get it on Google Play - Opens new window">

          <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://play.google.com/intl/en_us/badges/images/generic/en_badge_web_generic.png" media="(min-width:1024px)" /><source data-srcset="https://play.google.com/intl/en_us/badges/images/generic/en_badge_web_generic.png" media="(min-width:768px)" /><source data-srcset="https://play.google.com/intl/en_us/badges/images/generic/en_badge_web_generic.png" /><!--[if IE 9]></video><![endif]--><img alt="Get it on Google Play" class="d3-o-footer__apps-links-icon-google img-responsive" data-src="https://play.google.com/intl/en_us/badges/images/generic/en_badge_web_generic.png" src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==" /></picture>
        </a>
      </li>
    </ul>
  </nav>
</div>
  


  <div class="d3-o-footer__club-legal">

  <div class="d3-o-footer__club-legal-logo">

    

    <a class="" href="https://www.49ers.com" aria-title="San Francisco 49ers">
      <picture><!--[if IE 9]><video style="display: none; "><![endif]--><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" media="(min-width:1024px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" media="(min-width:768px)" /><source data-srcset="https://static.www.nfl.com/t_q-best/league/api/clubs/logos/SF" /><!--[if IE 9]></video><![endif]--></picture>
    </a>

  </div>

    <p class="d3-o-footer__club-copyright">
      &#xA9; 2020 Forty Niners Football Company LLC
    </p>
    <nav role="navigation">
      <h2 class="d3-o-nav__title">Legal links nav</h2>
      <ul class="d3-o-footer__club-legal-links">
          <li>
            <a href="/about-us/privacy-policy" title="Privacy Policy">
              Privacy Policy
            </a>
          </li>
          <li>
            <a href="/about-us/accessibility-policy" title="Accessibility">
              Accessibility
            </a>
          </li>
          <li>
            <a href="/about-us/contact-us" title="Contact Us">
              Contact Us
            </a>
          </li>
          <li>
            <a href="https://www.49ers.com/about-us/ccpa-notice" title="CCPA Notice">
              CCPA Notice
            </a>
          </li>
      </ul>
    </nav>
  </div>
</footer>



<svg xmlns="http://www.w3.org/2000/svg" id="NFLicons" style="display: none">
<symbol id='bigLeft'><desc id='bigLeftDesc'>
      Big left arrow icon
    </desc><polygon id="bigLeftArrow" points="15.1,0 16,0.9 1.7,16 16,31.1 15.1,32 0,16  " /></symbol><symbol id='bigRight'><desc id='bigRightDesc'>
      Big right arrow icon
    </desc><polygon id="bigRightArrow" points="0.9,0 0,0.9 14.3,16 0,31.1 0.9,32 16,16   " /></symbol><symbol id='close'><desc id='closeDesc'>
      Close icon
    </desc><polygon id="closeCross" points="12,9.8 2.2,0 0,2.2 9.8,12 0,21.8 2.2,24 12,14.2 21.8,24 24,21.8 14.2,12 24,2.2 21.8,0 " /></symbol><symbol id='copyUrl'><desc id='copyUrlDesc'>Copy Url</desc><path d="M8,4V16H20V4ZM18,14H10V6h8Z" /><polygon points="6 8 4 8 4 18 4 20 6 20 16 20 16 18 6 18 6 8" /></symbol><symbol id='dots'><desc id='dotsDesc'>
      Three dots icon
    </desc><circle id="dotsDot3" cx="21.6" cy="12" r="2.4" /><circle id="dotsDot2" cx="12" cy="12" r="2.4" /><circle id="dotsDot1" cx="2.4" cy="12" r="2.4" /></symbol><symbol id='down'><desc id='downDesc'>
      Down arrow icon
    </desc><polygon id="downArrow" points="12,9.8 20.8,1 21.9,0 24,2.1 23,3.1 13.1,12.9 12,14 12,14 12,14 10.9,13 1.1,3.1 0,2.1 2.1,0 3.2,1   " /></symbol><symbol id='email'><desc id='emailDesc'>
      Email icon
    </desc><path class="emailBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="emailCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24s24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8 C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><g class="emailLetter">
  <polygon points="13.1,20.7 13.1,30.5 34.9,30.5 34.9,20.7 24,26.2 " />
  <polygon points="13.1,16.4 13.1,18.5 24,24 34.9,18.5 34.9,16.4 " />
</g><polygon points="13.1,20.7 13.1,30.5 34.9,30.5 34.9,20.7 24,26.2 " /><polygon points="13.1,16.4 13.1,18.5 24,24 34.9,18.5 34.9,16.4 " /></symbol><symbol id='email-no-circle'><desc id='email-no-circleDesc'>
      Email icon
    </desc><path class="emailBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><g class="emailLetter">
  <polygon points="13.1,20.7 13.1,30.5 34.9,30.5 34.9,20.7 24,26.2 " />
  <polygon points="13.1,16.4 13.1,18.5 24,24 34.9,18.5 34.9,16.4 " />
</g><polygon points="13.1,20.7 13.1,30.5 34.9,30.5 34.9,20.7 24,26.2 " /><polygon points="13.1,16.4 13.1,18.5 24,24 34.9,18.5 34.9,16.4 " /></symbol><symbol id='exitFullScreen'><desc id='exitFullScreenDesc'>
      Exit Fullscreen icon
    </desc><path id="exitFullScreenCorners" d="M18.7,5.2H24v1.3h-6.7V0h1.3V5.2z M18.7,15.8V21h-1.3v-6.6H24v1.3H18.7z M5.3,5.2 V0h1.3v6.6H0V5.2H5.3z M5.3,15.8H0v-1.3h6.7V21H5.3V15.8z" /></symbol><symbol id='external'><desc id='externalDesc'>
      External link icon
    </desc><polygon id="externalBox" points="11,13 0,13 0,2 5.5,2 5.5,3 1,3 1,12 10,11.9 10,7.5 11,7.5   " /><polygon id="externalArrow" points="11.3,1 8,1 8,0 13,0 13,5 12,5 12,1.7 5.9,7.8 5.2,7.1  " /></symbol><symbol id='facebook'><desc id='facebookDesc'>
      Facebook logo
    </desc><path class="facebookBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="facebookCircle_1_" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z M24,45.8 C12,45.8,2.2,36,2.2,24S12,2.2,24,2.2S45.8,12,45.8,24S36,45.8,24,45.8z" /><path class="facebookF" d="M28.8,24h-4v13.1H20V24h-4v-4.4h4V17c0-4,2.4-6.2,6-6.2c1.7,0,3.2,0.1,3.6,0.2v4.2h-2.5 c-1.9,0-2.3,0.9-2.3,2.3v2.1h4.6L28.8,24z" /></symbol><symbol id='football'><desc id='footballDesc'>Football icon</desc><path d="M10,15.5v-1h1.25v-2H10v-1h1.25v-2H10v-1h1.25V3.13C9.1,3.76,6,6.9,6,12s3.1,8.24,5.25,8.87V15.5Z" /><path d="M12.75,3.13V8.5H14v1H12.75v2H14v1H12.75v2H14v1H12.75v5.37C14.9,20.24,18,17.1,18,12S14.9,3.76,12.75,3.13Z" /><rect class="cls-1" x="10" y="11.5" width="1.25" height="1" /><rect class="cls-1" x="10" y="8.5" width="1.25" height="1" /><rect class="cls-1" x="12.75" y="14.5" width="1.25" height="1" /><rect class="cls-1" x="10" y="14.5" width="1.25" height="1" /><rect class="cls-1" x="12.75" y="11.5" width="1.25" height="1" /><rect class="cls-1" x="12.75" y="8.5" width="1.25" height="1" /></symbol><symbol id='facebook-no-circle'><desc id='facebook-no-circleDesc'>
      Facebook logo
    </desc><path class="facebookBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="facebookF" d="M28.8,24h-4v13.1H20V24h-4v-4.4h4V17c0-4,2.4-6.2,6-6.2c1.7,0,3.2,0.1,3.6,0.2v4.2h-2.5 c-1.9,0-2.3,0.9-2.3,2.3v2.1h4.6L28.8,24z" /></symbol><symbol id='instagram'><desc id='instagramDesc'>
      Instagram logo
    </desc><path class="instagramBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="instagramCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24c13.3,0,24-10.7,24-24 C48,10.7,37.3,0,24,0z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="instagramCamera" d="M34.8,19.5c-0.1-1.2-0.2-2-0.5-2.6c-0.3-0.7-0.7-1.3-1.3-1.9c-0.6-0.6-1.2-1-1.9-1.3 c-0.7-0.3-1.5-0.5-2.6-0.5c-1.2-0.1-1.5-0.1-4.5-0.1s-3.3,0-4.5,0.1c-1.2,0.1-2,0.2-2.6,0.5c-0.7,0.3-1.3,0.7-1.9,1.3 c-0.6,0.6-1,1.2-1.3,1.9c-0.3,0.7-0.5,1.5-0.5,2.6c-0.1,1.2-0.1,1.5-0.1,4.5s0,3.3,0.1,4.5c0.1,1.2,0.2,2,0.5,2.6 c0.3,0.7,0.7,1.3,1.3,1.9c0.6,0.6,1.2,1,1.9,1.3c0.7,0.3,1.5,0.5,2.6,0.5c1.2,0.1,1.5,0.1,4.5,0.1s3.3,0,4.5-0.1 c1.2-0.1,2-0.2,2.6-0.5c0.7-0.3,1.3-0.7,1.9-1.3c0.6-0.6,1-1.2,1.3-1.9c0.3-0.7,0.5-1.5,0.5-2.6c0.1-1.2,0.1-1.5,0.1-4.5 S34.9,20.7,34.8,19.5z M32.9,28.4c0,1.1-0.2,1.6-0.4,2c-0.2,0.5-0.4,0.9-0.8,1.3c-0.4,0.4-0.7,0.6-1.3,0.8c-0.4,0.1-1,0.3-2,0.4 c-1.1,0.1-1.5,0.1-4.4,0.1c-2.9,0-3.3,0-4.4-0.1c-1.1,0-1.6-0.2-2-0.4c-0.5-0.2-0.9-0.4-1.3-0.8c-0.4-0.4-0.6-0.7-0.8-1.3 c-0.1-0.4-0.3-1-0.4-2c-0.1-1.1-0.1-1.5-0.1-4.4s0-3.3,0.1-4.4c0-1.1,0.2-1.6,0.4-2c0.2-0.5,0.4-0.9,0.8-1.3 c0.4-0.4,0.7-0.6,1.3-0.8c0.4-0.1,1-0.3,2-0.4c1.1-0.1,1.5-0.1,4.4-0.1s3.3,0,4.4,0.1c1.1,0,1.6,0.2,2,0.4c0.5,0.2,0.9,0.4,1.3,0.8 c0.4,0.4,0.6,0.7,0.8,1.3c0.1,0.4,0.3,1,0.4,2c0.1,1.1,0.1,1.5,0.1,4.4S32.9,27.3,32.9,28.4z" /><path class="instagramLens" d="M24,18.5c-3,0-5.5,2.4-5.5,5.5c0,3,2.4,5.5,5.5,5.5s5.5-2.4,5.5-5.5 C29.5,21,27,18.5,24,18.5z M24,27.5c-2,0-3.5-1.6-3.5-3.5c0-2,1.6-3.5,3.5-3.5s3.5,1.6,3.5,3.5C27.5,26,26,27.5,24,27.5z" /></symbol><symbol id='snapchat'><desc id='snapchatDesc'>
      Snapchat logo
    </desc><path class="snapchatBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="snapchatCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24c13.3,0,24-10.7,24-24C48,10.7,37.3,0,24,0 z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="snapchatGhost" d="M35.7,29.2c-3-0.7-4.2-2.8-4.7-3.9c-0.1-0.3,0-0.7,0.3-0.8l1.6-0.8c0.4-0.2,0.7-0.6,0.8-1 c0.1-0.4,0-0.9-0.2-1.3c-0.4-0.6-1.1-0.8-1.7-0.6l-1.1,0.3v-3c0,0,0,0,0,0c-0.3-5.3-5-6-6.5-6.1c0,0-0.1,0-0.1,0 c-1.5,0.1-6.2,0.8-6.5,6.1c0,0,0,0,0,0v3l-1.1-0.3c-0.6-0.2-1.3,0.1-1.7,0.6c-0.3,0.4-0.3,0.8-0.2,1.3c0.1,0.4,0.4,0.8,0.8,1 l1.6,0.8c0.3,0.1,0.4,0.5,0.3,0.8c-0.5,1.2-1.7,3.2-4.7,3.9c-0.3,0.1-0.5,0.4-0.4,0.6c0,0.1,0.4,2,3.2,2.1c0.1,0.5,0.3,1.2,0.6,1.5 c0.2,0.2,0.4,0.3,0.6,0.2c0,0,1.4-0.5,2.6-0.2c0,0,1.5,0.3,2.4,1.2c0.4,0.4,1,0.7,1.5,0.9c0.3,0.1,0.7,0.1,1.1,0.1s0.7,0,1.1-0.1 c0.6-0.1,1.1-0.4,1.5-0.9c0.9-0.9,2.4-1.2,2.4-1.2c1.3-0.2,2.6,0.2,2.6,0.2c0.2,0.1,0.5,0,0.6-0.2c0.2-0.3,0.4-0.9,0.6-1.5 c2.9-0.1,3.2-2,3.2-2.1C36.2,29.6,36,29.3,35.7,29.2z" /></symbol><symbol id='youtube'><desc id='youtubeDesc'>
      YouTube logo
    </desc><path class="youtubeBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="youtubeCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24c13.3,0,24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="youtubeTube" d="M36.9,36.9c-0.3,1.4-1.5,2.5-2.9,2.6c-3.3,0.4-6.7,0.4-10,0.4c-3.3,0-6.7,0-10-0.4 c-1.4-0.2-2.6-1.2-2.9-2.6c-0.5-2-0.5-4.2-0.5-6.3c0-2.1,0-4.3,0.5-6.3c0.3-1.4,1.5-2.5,2.9-2.6c3.3-0.4,6.7-0.4,10-0.4 c3.3,0,6.7,0,10,0.4c1.4,0.2,2.6,1.2,2.9,2.6c0.5,2,0.5,4.2,0.5,6.3C37.4,32.6,37.4,34.8,36.9,36.9z M12.6,25.8h1.9v10.3h1.8V25.8 h1.9v-1.7h-5.6V25.8z M27.7,33.5c0,0.8-0.2,1.2-0.7,1.2c-0.3,0-0.5-0.1-0.8-0.4v-5.4c0.3-0.3,0.5-0.4,0.8-0.4 c0.5,0,0.7,0.4,0.7,1.2V33.5z M27.8,27c-0.6,0-1.2,0.3-1.7,1v-3.9h-1.6v12h1.6v-0.9c0.5,0.7,1.1,1,1.7,1c0.6,0,1.1-0.3,1.3-1 c0.1-0.4,0.2-1,0.2-1.8v-3.6c0-0.8-0.1-1.4-0.2-1.8C28.9,27.4,28.5,27,27.8,27z M21.5,34c-0.4,0.5-0.7,0.8-1,0.8 c-0.2,0-0.3-0.1-0.4-0.4c0-0.1,0-0.2,0-0.6v-6.6h-1.6v7.1c0,0.6,0.1,1.1,0.1,1.3c0.2,0.5,0.5,0.7,1,0.7c0.6,0,1.2-0.4,1.8-1.1v1 h1.6v-8.9h-1.6V34z M33.8,30.6h-1.6v-0.8c0-0.8,0.3-1.2,0.8-1.2c0.5,0,0.8,0.4,0.8,1.2V30.6z M33,27c-0.8,0-1.5,0.3-1.9,0.9 c-0.3,0.5-0.5,1.2-0.5,2.1v3.1c0,0.9,0.2,1.7,0.5,2.1c0.5,0.6,1.1,0.9,2,0.9c0.8,0,1.5-0.3,2-1c0.2-0.3,0.3-0.6,0.4-1 c0-0.2,0-0.5,0-1V33h-1.7c0,0.6,0,1,0,1.1c-0.1,0.4-0.3,0.6-0.7,0.6c-0.6,0-0.8-0.4-0.8-1.2v-1.6h3.2V30c0-1-0.2-1.7-0.5-2.1 C34.4,27.3,33.8,27,33,27z" /><path class="youtubeU" d="M26.7,11.2h1.6v6.6c0,0.4,0,0.6,0,0.6c0,0.3,0.2,0.4,0.4,0.4c0.3,0,0.7-0.3,1-0.8v-6.9h1.6v9 h-1.6v-1c-0.7,0.7-1.3,1.1-1.9,1.1c-0.5,0-0.9-0.2-1.1-0.7c-0.1-0.3-0.1-0.7-0.1-1.3V11.2z" /><path class="youtubeO" d="M20.5,14.2c0-1,0.2-1.7,0.5-2.1c0.5-0.6,1.1-0.9,1.9-0.9c0.8,0,1.5,0.3,1.9,0.9 c0.3,0.5,0.5,1.2,0.5,2.1v3.2c0,1-0.2,1.7-0.5,2.1c-0.5,0.6-1.1,0.9-1.9,0.9c-0.8,0-1.5-0.3-1.9-0.9c-0.3-0.5-0.5-1.2-0.5-2.1 V14.2z M22.9,18.9c0.5,0,0.8-0.4,0.8-1.3v-3.8c0-0.8-0.3-1.3-0.8-1.3c-0.5,0-0.8,0.4-0.8,1.3v3.8C22.1,18.5,22.4,18.9,22.9,18.9z" /><path class="youtubeY" d="M16.1,8.1l1.3,4.8l1.2-4.8h1.8l-2.2,7.2v4.9h-1.8v-4.9c-0.2-0.9-0.5-2.1-1.1-3.8 c-0.4-1.1-0.8-2.3-1.2-3.4H16.1z" /><g class="youtubeLogo">
  <path class="youtubeTube" d="M36.9,36.9c-0.3,1.4-1.5,2.5-2.9,2.6c-3.3,0.4-6.7,0.4-10,0.4c-3.3,0-6.7,0-10-0.4 c-1.4-0.2-2.6-1.2-2.9-2.6c-0.5-2-0.5-4.2-0.5-6.3c0-2.1,0-4.3,0.5-6.3c0.3-1.4,1.5-2.5,2.9-2.6c3.3-0.4,6.7-0.4,10-0.4 c3.3,0,6.7,0,10,0.4c1.4,0.2,2.6,1.2,2.9,2.6c0.5,2,0.5,4.2,0.5,6.3C37.4,32.6,37.4,34.8,36.9,36.9z M12.6,25.8h1.9v10.3h1.8V25.8 h1.9v-1.7h-5.6V25.8z M27.7,33.5c0,0.8-0.2,1.2-0.7,1.2c-0.3,0-0.5-0.1-0.8-0.4v-5.4c0.3-0.3,0.5-0.4,0.8-0.4 c0.5,0,0.7,0.4,0.7,1.2V33.5z M27.8,27c-0.6,0-1.2,0.3-1.7,1v-3.9h-1.6v12h1.6v-0.9c0.5,0.7,1.1,1,1.7,1c0.6,0,1.1-0.3,1.3-1 c0.1-0.4,0.2-1,0.2-1.8v-3.6c0-0.8-0.1-1.4-0.2-1.8C28.9,27.4,28.5,27,27.8,27z M21.5,34c-0.4,0.5-0.7,0.8-1,0.8 c-0.2,0-0.3-0.1-0.4-0.4c0-0.1,0-0.2,0-0.6v-6.6h-1.6v7.1c0,0.6,0.1,1.1,0.1,1.3c0.2,0.5,0.5,0.7,1,0.7c0.6,0,1.2-0.4,1.8-1.1v1 h1.6v-8.9h-1.6V34z M33.8,30.6h-1.6v-0.8c0-0.8,0.3-1.2,0.8-1.2c0.5,0,0.8,0.4,0.8,1.2V30.6z M33,27c-0.8,0-1.5,0.3-1.9,0.9 c-0.3,0.5-0.5,1.2-0.5,2.1v3.1c0,0.9,0.2,1.7,0.5,2.1c0.5,0.6,1.1,0.9,2,0.9c0.8,0,1.5-0.3,2-1c0.2-0.3,0.3-0.6,0.4-1 c0-0.2,0-0.5,0-1V33h-1.7c0,0.6,0,1,0,1.1c-0.1,0.4-0.3,0.6-0.7,0.6c-0.6,0-0.8-0.4-0.8-1.2v-1.6h3.2V30c0-1-0.2-1.7-0.5-2.1 C34.4,27.3,33.8,27,33,27z" />
  <path class="youtubeU" d="M26.7,11.2h1.6v6.6c0,0.4,0,0.6,0,0.6c0,0.3,0.2,0.4,0.4,0.4c0.3,0,0.7-0.3,1-0.8v-6.9h1.6v9 h-1.6v-1c-0.7,0.7-1.3,1.1-1.9,1.1c-0.5,0-0.9-0.2-1.1-0.7c-0.1-0.3-0.1-0.7-0.1-1.3V11.2z" />
  <path class="youtubeO" d="M20.5,14.2c0-1,0.2-1.7,0.5-2.1c0.5-0.6,1.1-0.9,1.9-0.9c0.8,0,1.5,0.3,1.9,0.9 c0.3,0.5,0.5,1.2,0.5,2.1v3.2c0,1-0.2,1.7-0.5,2.1c-0.5,0.6-1.1,0.9-1.9,0.9c-0.8,0-1.5-0.3-1.9-0.9c-0.3-0.5-0.5-1.2-0.5-2.1 V14.2z M22.9,18.9c0.5,0,0.8-0.4,0.8-1.3v-3.8c0-0.8-0.3-1.3-0.8-1.3c-0.5,0-0.8,0.4-0.8,1.3v3.8C22.1,18.5,22.4,18.9,22.9,18.9z" />
  <path class="youtubeY" d="M16.1,8.1l1.3,4.8l1.2-4.8h1.8l-2.2,7.2v4.9h-1.8v-4.9c-0.2-0.9-0.5-2.1-1.1-3.8 c-0.4-1.1-0.8-2.3-1.2-3.4H16.1z" />
</g></symbol><symbol id='tiktok'><desc id='tiktokDesc'>
      TikTok logo
    </desc><path class="tiktokBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="tiktokCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24c13.3,0,24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="tiktokLogo" d="m 30.973001,19.236 v 0 A 4.7689932,4.769 0 0 1 29.913003,18.85 4.7599932,4.76 0 0 1 28.652005,17.889 l -0.008,-0.011 0.012,0.011 -0.668999,-0.989 c 0,0.011 0.013,0.022 0.018,0.033 l -0.022,-0.033 v 0 a 3.7259947,3.726 0 0 1 -0.422,-1.9 h -0.857999 v 0 0 h -2.261996 v 11.954 a 3.5819949,3.582 0 0 1 -0.139,1.230999 2.4679965,2.468 0 0 1 -1.357998,1.49 2.6589962,2.659 0 0 1 -3.217996,-0.818 h -0.006 a 2.8749959,2.875 0 0 1 -0.505999,-1.737 2.6259962,2.626 0 0 1 3.399995,-2.307999 v -3.137 a 5.0999927,5.1 0 0 0 -0.899998,-0.056 v 0 a 5.3069924,5.307 0 0 0 -1.971998,0.413 5.588992,5.589 0 0 0 -1.587997,0.938 6.1289912,6.129 0 0 0 -1.335998,1.628 5.6339919,5.634 0 0 0 -0.688,2.6 6.1449912,6.145 0 0 0 0.343,2.041 v 0 a 6.0999913,6.1 0 0 0 0.849999,1.538999 6.299991,6.3 0 0 0 0.447999,0.506 l 0.002,0.016 v 0 a 6.3349909,6.335 0 0 0 0.899999,0.752 A 5.8669916,5.867 0 0 0 21.860014,33 a 5.6689919,5.669 0 0 0 2.362997,-0.518001 5.7599918,5.76 0 0 0 2.964996,-3.152 6.3999909,6.4 0 0 0 0.375999,-1.969 V 21.018 c 0.038,0.023 0.541999,0.353 0.541999,0.353 a 7.2419897,7.242 0 0 0 1.858998,0.762 10.785985,10.786 0 0 0 1.907997,0.259 v -3.07 a 2.9189958,2.919 0 0 1 -0.899999,-0.086 z" /></symbol><symbol id='spotify'><desc id='spotifyDesc'>
      Spotify logo
    </desc><path class="spotifyBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="spotifyCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24c13.3,0,24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="spotifyLogo" d="m 24,15 a 9,9 0 1 0 9,9 9,9 0 0 0 -9,-9 z m 4.127,12.981 a 0.561,0.561 0 0 1 -0.771,0.187 C 25.243,26.876 22.583,26.584 19.45,27.3 A 0.561,0.561 0 1 1 19.2,26.207 c 3.428,-0.784 6.369,-0.447 8.741,1 a 0.56,0.56 0 0 1 0.186,0.774 z m 1.1,-2.451 A 0.7,0.7 0 0 1 28.262,25.762 11.812,11.812 0 0 0 19.3,24.713 0.70165839,0.70165839 0 1 1 18.893,23.37 13.13,13.13 0 0 1 29,24.566 0.7,0.7 0 0 1 29.229,25.53 Z m 0.1,-2.551 C 26.423,21.256 21.637,21.1 18.868,21.938 a 0.842,0.842 0 1 1 -0.489,-1.611 c 3.179,-0.965 8.464,-0.779 11.8,1.2 a 0.842,0.842 0 0 1 -0.859,1.448 z" /></symbol><symbol id='linkedin'><desc id='linkedinDesc'>
      LinkedIn logo
    </desc><path class="linkedinBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="linkedinCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24c13.3,0,24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="linkedinLogo" d="M 31.672001,15 H 16.329 A 1.329,1.329 0 0 0 15,16.329 V 31.671 A 1.329,1.329 0 0 0 16.329,33 H 31.672001 A 1.328,1.328 0 0 0 33,31.671 V 16.329 A 1.328,1.328 0 0 0 31.672001,15 Z M 20.365,30.334 h -2.706 v -8.6 h 2.706 z m -1.353,-9.787 a 1.554,1.554 0 1 1 1.56,-1.554 1.532,1.532 0 0 1 -1.56,1.554 z m 11.330001,9.794 h -2.707 v -4.695 c 0,-1.386 -0.588,-1.813 -1.348001,-1.813 A 1.645,1.645 0 0 0 24.7,25.68 v 4.661 h -2.709 v -8.6 h 2.6 v 1.191 h 0.035 A 2.951,2.951 0 0 1 27.2,21.5 c 1.510001,0 3.142001,0.895 3.142001,3.521 z" /></symbol><symbol id='grid'><desc id='gridDesc'>
      Grid icon
    </desc><rect id="gridSquare_8" y="14" width="4" height="4" /><rect id="gridSquare_7" y="7" width="4" height="4" /><rect id="gridSquare_6" width="4" height="4" /><rect id="gridSquare_5" x="7" y="14" width="4" height="4" /><rect id="gridSquare_4" x="7" y="7" width="4" height="4" /><rect id="gridSquare_3" x="7" width="4" height="4" /><rect id="gridSquare_2" x="14" y="14" width="4" height="4" /><rect id="gridSquare_1" x="14" y="7" width="4" height="4" /><rect id="gridSquare" x="14" width="4" height="4" /></symbol><symbol id='key'><desc id='keyDesc'>
      Key icon
    </desc><path id="keyKey" d="M21.6,3.6H11.5C10.6,1.5,8.5,0,6,0C2.7,0,0,2.7,0,6s2.7,6,6,6c2.5,0,4.6-1.5,5.5-3.6h1.7l1.2-1.2 l1.2,1.2H18l1.2-1.2l1.2,1.2h1.2L24,6L21.6,3.6z" /></symbol><symbol id='left'><desc id='leftDesc'>
      Left arrow icon
    </desc><polygon id="leftArrow" points="4.2,12 13,20.8 14,21.9 11.9,24 10.9,23 1.1,13.1 0,12 0,12 0,12 1,10.9 10.9,1.1 11.9,0 14,2.1 13,3.2  " /></symbol><symbol id='link'><desc id='linkDesc'>
      Link icon
    </desc><path id="linkCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24s24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8 C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><g id="linkChain">
  <rect x="21.9" y="18.1" transform="matrix(0.7071 -0.7071 0.7071 0.7071 -9.7152 23.4544)" width="3.1" height="10.8" />
  <polygon points="29.5,20.7 27.3,22.9 32.7,28.4 28.4,32.7 22.9,27.3 20.7,29.5 28.4,37.1 37.1,28.4" />
  <polygon points="18.5,14.2 24,19.6 26.2,17.5 18.5,9.8 9.8,18.5 17.5,26.2 19.6,24 14.2,18.5" />
</g><polygon points="29.5,20.7 27.3,22.9 32.7,28.4 28.4,32.7 22.9,27.3 20.7,29.5 28.4,37.1 37.1,28.4" /><polygon points="18.5,14.2 24,19.6 26.2,17.5 18.5,9.8 9.8,18.5 17.5,26.2 19.6,24 14.2,18.5" /><rect x="21.9" y="18.1" transform="matrix(0.7071 -0.7071 0.7071 0.7071 -9.7152 23.4544)" width="3.1" height="10.8" /></symbol><symbol id='location'><desc id='locationDesc'>
      Location icon
    </desc><path id="locationPin" d="M6,3.2c-1.5,0-2.7,1.2-2.7,2.6c0,1.4,1.2,2.6,2.7,2.6s2.7-1.2,2.7-2.6C8.7,4.4,7.5,3.2,6,3.2 z M6,0L6,0c3.3,0,6,2.6,6,5.8c0,1.2-0.8,2.9-1.3,3.9L6,18L1.3,9.6C0.8,8.7,0,6.9,0,5.8C0,2.6,2.7,0,6,0z" /></symbol><symbol id='mail'><desc id='mailDesc'>
      Mail icon
    </desc><path id="mailEnvelope" d="M0.7,11h18.5V1.1l-9,7c-0.1,0.1-0.2,0.1-0.2,0.1s-0.2,0-0.2-0.1l-9-7V11z M18.6,0.7H1.4 L10,7.4L18.6,0.7z M20,0.3C20,0.3,20,0.3,20,0.3l0,11c0,0.2-0.2,0.3-0.4,0.3H0.4c-0.2,0-0.4-0.2-0.4-0.3v-11c0,0,0,0,0,0 c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0 c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0h19.3c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0 c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0c0,0,0,0,0,0 c0,0,0,0,0,0C20,0.2,20,0.3,20,0.3C20,0.3,20,0.3,20,0.3C20,0.3,20,0.3,20,0.3z" /></symbol><symbol id='menu'><desc id='menuDesc'>
      Menu icon
    </desc><path id="menuHamburger" d="M0,6h24v2H0V6z M0,11h24v2H0V11z M0,16h24v2H0V16z" /></symbol><symbol id='open'><desc id='openDesc'>
      Open icon
    </desc><polygon id="openArrow" points="13.5,18.3 13.5,0 10.5,0 10.5,18.3 2.2,10.2 0,12.3 12,24 24,12.3 21.8,10.2 " /></symbol><symbol id='phone'><desc id='phoneDesc'>
      Phone icon
    </desc><path id="phoneHandset" d="M14.8,14.3c-0.7,0.7-1.4,1.1-2.4,1.1c-1.7,0-4-1.1-6.6-3.2c-2.6-2-5.5-5.1-5-8.7 C0.9,2.4,1.6,1,2.4,0.7l0,0c0.2,0,0.6,0.3,0.8,0.5l0.1,0.1c0.4,0.3,0.8,0.7,1.2,1.1l0.2,0.2C5,2.7,5.2,2.9,5.3,3.2 c0.3,0.4,0.2,0.6-0.3,1.4C4.9,4.8,4.4,5.3,4.1,5.6l0,0l0,0C3.8,6.1,4,6.6,4.5,7.2C4.7,7.6,4.9,8,5,8.1c0.4,0.6,0.9,1.1,1.4,1.6 c0.5,0.5,1.1,0.9,1.7,1.3c0.1,0.1,1.5,1.1,2,1.1c0.2,0,0.4-0.1,0.5-0.2l0.7-0.7c0.1-0.1,0.2-0.2,0.3-0.3c0.2-0.2,0.4-0.5,0.6-0.5 c0.3,0,1.1,0.7,1.4,0.9c0.2,0.1,0.3,0.3,0.4,0.3c0.6,0.4,1.3,1,1.3,1.6C15.3,13.4,15.3,13.7,14.8,14.3 M14.4,11.1 c-0.5-0.4-1.6-1.4-2.2-1.4c-0.6,0-1,0.6-1.3,1l-0.7,0.7c-0.1,0.1-1.5-0.8-1.7-0.9c-0.6-0.4-1.1-0.8-1.6-1.3c-0.5-0.5-0.9-1-1.3-1.5 c-0.1-0.2-1-1.5-0.9-1.6c0,0,0.8-0.8,1-1.2c0.5-0.7,0.8-1.3,0.3-2.1C5.7,2.5,5.5,2.3,5.2,2C4.7,1.5,4.3,1.1,3.7,0.7 C3.5,0.5,3,0.1,2.5,0C2.4,0,2.3,0,2.2,0C1,0.5,0.2,2.2,0.1,3.4c-0.6,3.8,2.5,7.1,5.2,9.3c2.5,1.9,7.2,5.1,10,2 c0.4-0.4,0.8-0.9,0.8-1.5C15.9,12.3,15.1,11.6,14.4,11.1" /></symbol><symbol id='play'><desc id='playDesc'>
      Play icon
    </desc><path id="playBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z M19.6,32.7V15.3   L32.7,24L19.6,32.7z" /><polygon id="playForeground" points="19.6,32.7 32.7,24 19.6,15.3  " /></symbol><symbol id='radio'><desc id='radioDesc'>
      Radio icon
    </desc><path id="radioBody" d="M5,3.8V0H2.5v3.8H0v13.8h20V3.8H5z M5.6,13.8c-1.7,0-3.1-1.4-3.1-3.1s1.4-3.1,3.1-3.1 s3.1,1.4,3.1,3.1S7.4,13.8,5.6,13.8z M17.5,10H15V7.5h2.5V10z" /></symbol><symbol id='rewind'><desc id='rewindDesc'>
      Rewind icon
    </desc><path id="rewindArrow" d="M11.3,0c-3.5,0-6.7,1.4-9,3.7L0,1.4v5.6h5.6L3.3,4.7c2-2,4.9-3.3,8-3.3 c6.2,0,11.3,5.1,11.3,11.3S17.5,24,11.3,24c-3.1,0-5.9-1.3-8-3.3l-1,1c2.3,2.3,5.5,3.7,9,3.7c7,0,12.7-5.7,12.7-12.7S18.3,0,11.3,0 " /></symbol><symbol id='right'><desc id='rightDesc'>
      Right arrow icon
    </desc><polygon id="rightArrow" points="9.8,12 1,20.8 0,21.9 2.1,24 3.1,23 12.9,13.1 14,12 14,12 14,12 13,10.9 3.1,1.1 2.1,0 0,2.1 1,3.2   " /></symbol><symbol id='search'><desc id='searchDesc'>
      Search icon
    </desc><path id="searchLens" d="M9.7,17.7c4.3,0,7.9-3.5,7.9-7.9S14,1.8,9.7,1.8S1.8,5.4,1.8,9.7S5.3,17.7,9.7,17.7z M17.4,15.7l6.6,6.6L22.3,24l-6.6-6.6c-1.6,1.3-3.7,2.1-6,2.1C4.3,19.5,0,15.1,0,9.7S4.3,0,9.7,0c5.3,0,9.7,4.4,9.7,9.7 C19.4,12,18.6,14,17.4,15.7z" /></symbol><symbol id='select'><desc id='selectDesc'>
      Select icon
    </desc><path class="selectCircle" d="M12,23c6.1,0,11-4.9,11-11S18.1,1,12,1S1,5.9,1,12S5.9,23,12,23z M12,24C5.4,24,0,18.6,0,12 S5.4,0,12,0s12,5.4,12,12S18.6,24,12,24z" /><circle class="selectBackground" cx="12" cy="12" r="12" /></symbol><symbol id='selected'><desc id='selectedDesc'>
      Selected icon
    </desc><path class="selectedFlag" d="M10.1,16.8c2.9-2.9,5.8-5.9,8.7-8.9c-0.3-0.3-0.6-0.6-0.9-0.9c-2.6,2.6-5.2,5.3-7.8,8 c-1.1-1.1-2.1-2.2-3.2-3.2C6.6,12,6.3,12.3,6,12.6C7.4,14,8.7,15.4,10.1,16.8" /><circle class="selectedBackground" cx="12" cy="12" r="12" /></symbol><symbol id='tv'><desc id='tvDesc'>
      TV icon
    </desc><polygon id="tvScreen" points="0,0 0,11.2 8.8,11.2 8.8,13.8 5,13.8 5,15 15,15 15,13.8 11.2,13.8 11.2,11.2 20,11.2 20,0  " /></symbol><symbol id='twitter'><desc id='twitterDesc'>
      Twitter logo
    </desc><path class="twitterBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="twitterCircle" d="M24,0C10.7,0,0,10.7,0,24c0,13.3,10.7,24,24,24s24-10.7,24-24C48,10.7,37.3,0,24,0z M24,45.8C12,45.8,2.2,36,2.2,24C2.2,12,12,2.2,24,2.2S45.8,12,45.8,24C45.8,36,36,45.8,24,45.8z" /><path class="twitterBird" d="M34.7,17.3c-0.6,0.9-1.3,1.7-2.2,2.3c0,0.2,0,0.4,0,0.6c0,5.8-4.4,12.5-12.5,12.5 c-2.5,0-4.8-0.7-6.8-2c0.3,0,0.7,0.1,1.1,0.1c2.1,0,4-0.7,5.5-1.9c-1.9,0-3.6-1.3-4.1-3.1c0.3,0.1,0.5,0.1,0.8,0.1 c0.4,0,0.8-0.1,1.2-0.2c-2-0.4-3.5-2.2-3.5-4.3v-0.1c0.6,0.3,1.3,0.5,2,0.6c-1.2-0.8-2-2.1-2-3.7c0-0.8,0.2-1.6,0.6-2.2 c2.2,2.7,5.4,4.4,9.1,4.6c-0.1-0.3-0.1-0.7-0.1-1c0-2.4,2-4.4,4.4-4.4c1.3,0,2.4,0.5,3.2,1.4c1-0.2,1.9-0.6,2.8-1.1 c-0.3,1-1,1.9-1.9,2.4C33.1,17.9,33.9,17.7,34.7,17.3z" /></symbol><symbol id='twitter-no-circle'><desc id='twitter-no-circleDesc'>
      Twitter logo
    </desc><path class="twitterBackground" d="M24,0C10.7,0,0,10.7,0,24s10.7,24,24,24s24-10.7,24-24S37.3,0,24,0z" /><path class="twitterBird" d="M34.7,17.3c-0.6,0.9-1.3,1.7-2.2,2.3c0,0.2,0,0.4,0,0.6c0,5.8-4.4,12.5-12.5,12.5 c-2.5,0-4.8-0.7-6.8-2c0.3,0,0.7,0.1,1.1,0.1c2.1,0,4-0.7,5.5-1.9c-1.9,0-3.6-1.3-4.1-3.1c0.3,0.1,0.5,0.1,0.8,0.1 c0.4,0,0.8-0.1,1.2-0.2c-2-0.4-3.5-2.2-3.5-4.3v-0.1c0.6,0.3,1.3,0.5,2,0.6c-1.2-0.8-2-2.1-2-3.7c0-0.8,0.2-1.6,0.6-2.2 c2.2,2.7,5.4,4.4,9.1,4.6c-0.1-0.3-0.1-0.7-0.1-1c0-2.4,2-4.4,4.4-4.4c1.3,0,2.4,0.5,3.2,1.4c1-0.2,1.9-0.6,2.8-1.1 c-0.3,1-1,1.9-1.9,2.4C33.1,17.9,33.9,17.7,34.7,17.3z" /></symbol><symbol id='up'><desc id='upDesc'>
      Up arrow icon
    </desc><polygon id="upArrow" points="12,4.2 20.8,13 21.9,14 24,11.9 23,10.9 13.1,1.1 12,0 12,0 12,0 10.9,1 1.1,10.9 0,11.9 2.1,14 3.2,13 " /></symbol><symbol id='user'><desc id='userDesc'>
      User icon
    </desc><path id="userAvatar" d="M24.2,24.1c-0.9-0.9-2-1.6-3.1-2.2c-0.4-0.4-0.6-1-0.5-1.6c0.2-0.8,0.8-0.9,1.4-2.4 c0.4-1.4,0.6-2.9,0.5-4.4c0,0-0.1-3.2-2.4-4.8c-1.2-0.8-2.6-1.2-4-1.2c-1.4,0-2.7,0.4-3.9,1.2c-2.3,1.6-2.4,4.8-2.4,4.8 c-0.2,1.5,0,3,0.5,4.4c0.5,1.5,1.2,1.6,1.4,2.4c0.1,0.6-0.1,1.2-0.5,1.6c-1.2,0.6-2.2,1.3-3.1,2.2c-0.7,0.8-1.4,1.7-1.9,2.6 C3,23.9,1.3,20.1,1.3,16C1.3,7.9,7.9,1.3,16,1.3S30.7,7.9,30.7,16c0,4-1.7,7.9-4.6,10.7C25.5,25.7,24.9,24.9,24.2,24.1z M16,0 C7.2,0,0,7.2,0,16s7.2,16,16,16s16-7.2,16-16c0-4.2-1.7-8.3-4.7-11.3C24.3,1.7,20.2,0,16,0z" /></symbol><symbol id='audio'><desc id='audioDesc'>
      Audio icon
    </desc><path id="audioBackground" d="M12,1C5.9,1,1,5.9,1,12s4.9,11,11,11s11-4.9,11-11S18.1,1,12,1z" /><path id="audioForeground" d="M14.1,14.3c-0.5,0.5-1.1,0.8-1.8,0.9v1.1h1v0.6h-2.6v-0.6h1v-1.1c-0.7-0.1-1.3-0.4-1.8-0.9    C9.3,13.8,9,13,9,12.2l0.6,0c0,1.3,1.1,2.4,2.4,2.4s2.4-1.1,2.4-2.4l0.6,0C15,13,14.7,13.8,14.1,14.3z M14.1,9.4h-1V10h1v0.5h-1    v0.6h1v0.5h-1v0.6h1c0,1.1-0.9,2.1-2.1,2.1s-2.1-0.9-2.1-2.1h1v-0.6h-1v-0.5h1v-0.6h-1V10h1V9.4h-1V9.2c0-1.1,0.9-2.1,2.1-2.1    s2.1,0.9,2.1,2.1V9.4z" /></symbol><symbol id='tickets'><desc id='ticketsDesc'>Tickets icon</desc><path d="M13.2283,12.722 L13.9873,15.058 L12.0003,13.614 L10.0123,15.058 L10.7713,12.722 L8.7843,11.278 L11.2413,11.278 L12.0003,8.942 L12.7593,11.278 L15.2153,11.278 L13.2283,12.722 Z M14.0003,3 C14.0003,4.105 13.1043,5 12.0003,5 C10.8953,5 10.0003,4.105 10.0003,3 L7.0003,3 L7.0003,21 L10.0003,21 C10.0003,19.895 10.8953,19 12.0003,19 C13.1043,19 14.0003,19.895 14.0003,21 L17.0003,21 L17.0003,3 L14.0003,3 Z"></path></symbol><symbol id='addToCalendar'><desc id='addToCalendarDesc'>Add to calendar icon</desc><path d="M13.8507463,8.26268657 L13.8507463,1.91044776 L13.8507463,1.43283582 L10.9850746,1.43283582 L10.9850746,0 L10.5074627,0 L8.8358209,0 C8.66069652,0 8.50149254,0 8.35820896,0 L8.35820896,1.43283582 L5.49253731,1.43283582 L5.49253731,0 C5.34925373,0 5.19004975,0 5.01492537,0 L3.34328358,0 C3.1681592,0 3.00895522,0 2.86567164,0 L2.86567164,1.43283582 L0,1.43283582 L0,1.91044776 L0,12.8955224 L0,13.3731343 L8.74029851,13.3731343 C9.36119403,14.638806 10.6746269,15.5223881 12.1791045,15.5223881 C14.280597,15.5223881 16,13.8029851 16,11.7014925 C16,10.1970149 15.1164179,8.88358209 13.8507463,8.26268657 Z M9.31343284,0.955223881 L10.0298507,0.955223881 L10.0298507,2.86567164 L9.31343284,2.86567164 L9.31343284,0.955223881 Z M3.82089552,0.955223881 L4.53731343,0.955223881 L4.53731343,2.86567164 L3.82089552,2.86567164 L3.82089552,0.955223881 Z M2.86567164,2.3880597 L2.86567164,3.82089552 L3.34328358,3.82089552 L5.01492537,3.82089552 C5.19004975,3.82089552 5.34925373,3.82089552 5.49253731,3.82089552 L5.49253731,2.3880597 L8.35820896,2.3880597 L8.35820896,3.82089552 C8.50149254,3.82089552 8.66069652,3.82089552 8.8358209,3.82089552 L10.5074627,3.82089552 C10.6825871,3.82089552 10.841791,3.82089552 10.9850746,3.82089552 L10.9850746,2.3880597 L12.8955224,2.3880597 L12.8955224,4.7761194 L0.955223881,4.7761194 L0.955223881,2.3880597 L2.86567164,2.3880597 Z M0.955223881,12.4179104 L0.955223881,5.73134328 L12.8955224,5.73134328 L12.8955224,7.95223881 C12.6567164,7.90447761 12.4179104,7.88059701 12.1791045,7.88059701 C10.0776119,7.88059701 8.35820896,9.6 8.35820896,11.7014925 C8.35820896,11.9402985 8.38208955,12.1791045 8.42985075,12.4179104 L0.955223881,12.4179104 Z M12.1791045,14.5671642 C10.6029851,14.5671642 9.31343284,13.2776119 9.31343284,11.7014925 C9.31343284,10.1253731 10.6029851,8.8358209 12.1791045,8.8358209 C13.7552239,8.8358209 15.0447761,10.1253731 15.0447761,11.7014925 C15.0447761,13.2776119 13.7552239,14.5671642 12.1791045,14.5671642 Z M13.8507463,11.7014925 L13.8507463,12.1791045 L12.6567164,12.1791045 L12.6567164,13.3731343 L12.1791045,13.3731343 L11.7014925,13.3731343 L11.7014925,12.1791045 L10.5074627,12.1791045 L10.5074627,11.7014925 L10.5074627,11.2238806 L11.7014925,11.2238806 L11.7014925,10.0298507 L12.1791045,10.0298507 L12.6567164,10.0298507 L12.6567164,11.2238806 L13.8507463,11.2238806 L13.8507463,11.7014925 Z"></path></symbol><symbol id='NFC'><desc id='NFCDesc'>NFC icon</desc><path d="M0 0h500v500h-500z" /><path fill="#fff" d="M276.127 95.338v68.563l-65.7-68.563h-165.427v83.011l29.028 6.106v134.612l-27.907 6.928v78.554h189.703v-67.909l67.166 67.95h127.018v-220.539l24.992-5.903v-82.81z" /><path d="M300.744 283.954l7.439 23.488-20.08-15.205-20.067 14.415 7.767-22.965-18.901-15.205h23.495l7.706-21.979 7.679 21.979h24.43l-19.469 15.471zm47.035 19.514l7.692 22.036h24.469l-19.493 15.385 7.439 23.545-20.106-15.248-20.061 14.436 7.827-22.947-18.928-15.172h23.483l7.679-22.036zm-119.417-68.251l-20.094 14.436 7.778-22.945-18.864-15.19h23.47l7.71-22.014 7.622 22.014h24.493l-19.481 15.387 7.39 23.539-20.024-15.227zm-47.105-65.299l7.446 23.537-20.088-15.207-20.067 14.458 7.79-23.018-18.897-15.164h23.477l7.698-21.971 7.655 21.971h24.508l-19.52 15.393zm110.042-3.809l20.248 6.138v50.549l-107.594-112.285h-143.781v55.521l29.028 6.104v158.797l-27.907 6.93v51.511h159.359v-51.388l-23.844-6.996v-55.292l112.516 113.697h105.511v-217.357l24.992-5.901v-55.628h-148.529v55.597z" fill="#013369" /><path d="M176.27 154.525h24.508l-19.52 15.393 7.446 23.537-20.088-15.207-20.067 14.458 7.79-23.018-18.897-15.164h23.477l7.698-21.971 7.655 21.971zm84.208 56.993l-19.481 15.387 7.39 23.539-20.024-15.227-20.094 14.436 7.778-22.945-18.864-15.19h23.47l7.71-22.014 7.622 22.014h24.493zm27.626 34.986l7.679 21.979h24.43l-19.469 15.471 7.439 23.488-20.08-15.205-20.067 14.415 7.767-22.965-18.901-15.205h23.495l7.706-21.979zm47.441 94.173l-18.928-15.172h23.483l7.679-22.036 7.692 22.036h24.469l-19.493 15.385 7.439 23.545-20.106-15.248-20.061 14.436 7.827-22.947z" fill="#fff" /><g fill="none">
  <path d="M0 0h500v500h-500z" />
  <path fill="#fff" d="M276.127 95.338v68.563l-65.7-68.563h-165.427v83.011l29.028 6.106v134.612l-27.907 6.928v78.554h189.703v-67.909l67.166 67.95h127.018v-220.539l24.992-5.903v-82.81z" />
  <path d="M300.744 283.954l7.439 23.488-20.08-15.205-20.067 14.415 7.767-22.965-18.901-15.205h23.495l7.706-21.979 7.679 21.979h24.43l-19.469 15.471zm47.035 19.514l7.692 22.036h24.469l-19.493 15.385 7.439 23.545-20.106-15.248-20.061 14.436 7.827-22.947-18.928-15.172h23.483l7.679-22.036zm-119.417-68.251l-20.094 14.436 7.778-22.945-18.864-15.19h23.47l7.71-22.014 7.622 22.014h24.493l-19.481 15.387 7.39 23.539-20.024-15.227zm-47.105-65.299l7.446 23.537-20.088-15.207-20.067 14.458 7.79-23.018-18.897-15.164h23.477l7.698-21.971 7.655 21.971h24.508l-19.52 15.393zm110.042-3.809l20.248 6.138v50.549l-107.594-112.285h-143.781v55.521l29.028 6.104v158.797l-27.907 6.93v51.511h159.359v-51.388l-23.844-6.996v-55.292l112.516 113.697h105.511v-217.357l24.992-5.901v-55.628h-148.529v55.597z" fill="#013369" />
  <path d="M176.27 154.525h24.508l-19.52 15.393 7.446 23.537-20.088-15.207-20.067 14.458 7.79-23.018-18.897-15.164h23.477l7.698-21.971 7.655 21.971zm84.208 56.993l-19.481 15.387 7.39 23.539-20.024-15.227-20.094 14.436 7.778-22.945-18.864-15.19h23.47l7.71-22.014 7.622 22.014h24.493zm27.626 34.986l7.679 21.979h24.43l-19.469 15.471 7.439 23.488-20.08-15.205-20.067 14.415 7.767-22.965-18.901-15.205h23.495l7.706-21.979zm47.441 94.173l-18.928-15.172h23.483l7.679-22.036 7.692 22.036h24.469l-19.493 15.385 7.439 23.545-20.106-15.248-20.061 14.436 7.827-22.947z" fill="#fff" />
</g></symbol><symbol id='AFC'><desc id='AFCDesc'>AFC icon</desc><path d="M0 0h500v500h-500z" /><path fill="#fff" d="M217.375 264.305h37.842l-20.046-59.716-17.796 59.716" /><path fill="#fff" d="M431.992 317.729l-56.233-125.998 29.937-7.517v-88.645h-310.385v88.593l29.539 7.505-56.991 126.129-32.859 8.019v78.325h431v-78.436z" /><path d="M378.261 372.971l-19.436-14.734-19.384 14.027 7.519-22.221-18.302-14.682h22.752l7.415-21.205 7.418 21.205h23.647l-18.809 14.906 7.18 22.703zm-79.323-98.369h22.718l7.461-21.22 7.372 21.22h23.675l-18.83 14.917 7.206 22.692-19.423-14.719-19.436 14.036 7.555-22.245-18.298-14.682zm-28.976-62.4h22.757l7.424-21.213 7.396 21.213h23.66l-18.839 14.927 7.18 22.688-19.397-14.729-19.423 13.984 7.527-22.202-18.285-14.669zm-20.828-24.692l7.53-22.18-18.218-14.675h22.699l7.443-21.241 7.383 21.241h23.683l-18.858 14.895 7.198 22.714-19.406-14.734-19.453 13.98zm-31.758 76.796l17.796-59.716 20.046 59.716h-37.842zm137.2-82.863l35.965-9.035v-61.701h-280.072v61.684l35.631 9.052-67.464 149.308-28.478 6.949v51.301h171.527v-51.301l-24.306-6.915 7.782-21.864h60.521l7.241 21.907-22.421 6.872v51.301h200.348v-51.301l-29.636-6.949-66.637-149.308z" fill="#D50A0A" /><path d="M280.795 165.549l7.198 22.714-19.406-14.734-19.453 13.98 7.53-22.18-18.218-14.675h22.699l7.443-21.241 7.383 21.241h23.683l-18.858 14.895zm19.348 69.539l-19.423 13.984 7.527-22.202-18.285-14.669h22.757l7.424-21.213 7.396 21.213h23.66l-18.839 14.927 7.18 22.688-19.397-14.729zm28.974 62.405l-19.436 14.036 7.555-22.245-18.298-14.682h22.718l7.461-21.22 7.372 21.22h23.675l-18.83 14.917 7.206 22.692-19.423-14.719zm37.126 37.87h23.647l-18.809 14.906 7.18 22.703-19.436-14.734-19.384 14.027 7.519-22.221-18.302-14.682h22.752l7.415-21.205 7.418 21.205z" fill="#fff" /><g fill="none">
  <path d="M0 0h500v500h-500z" />
  <path fill="#fff" d="M217.375 264.305h37.842l-20.046-59.716-17.796 59.716" />
  <path fill="#fff" d="M431.992 317.729l-56.233-125.998 29.937-7.517v-88.645h-310.385v88.593l29.539 7.505-56.991 126.129-32.859 8.019v78.325h431v-78.436z" />
  <path d="M378.261 372.971l-19.436-14.734-19.384 14.027 7.519-22.221-18.302-14.682h22.752l7.415-21.205 7.418 21.205h23.647l-18.809 14.906 7.18 22.703zm-79.323-98.369h22.718l7.461-21.22 7.372 21.22h23.675l-18.83 14.917 7.206 22.692-19.423-14.719-19.436 14.036 7.555-22.245-18.298-14.682zm-28.976-62.4h22.757l7.424-21.213 7.396 21.213h23.66l-18.839 14.927 7.18 22.688-19.397-14.729-19.423 13.984 7.527-22.202-18.285-14.669zm-20.828-24.692l7.53-22.18-18.218-14.675h22.699l7.443-21.241 7.383 21.241h23.683l-18.858 14.895 7.198 22.714-19.406-14.734-19.453 13.98zm-31.758 76.796l17.796-59.716 20.046 59.716h-37.842zm137.2-82.863l35.965-9.035v-61.701h-280.072v61.684l35.631 9.052-67.464 149.308-28.478 6.949v51.301h171.527v-51.301l-24.306-6.915 7.782-21.864h60.521l7.241 21.907-22.421 6.872v51.301h200.348v-51.301l-29.636-6.949-66.637-149.308z" fill="#D50A0A" />
  <path d="M280.795 165.549l7.198 22.714-19.406-14.734-19.453 13.98 7.53-22.18-18.218-14.675h22.699l7.443-21.241 7.383 21.241h23.683l-18.858 14.895zm19.348 69.539l-19.423 13.984 7.527-22.202-18.285-14.669h22.757l7.424-21.213 7.396 21.213h23.66l-18.839 14.927 7.18 22.688-19.397-14.729zm28.974 62.405l-19.436 14.036 7.555-22.245-18.298-14.682h22.718l7.461-21.22 7.372 21.22h23.675l-18.83 14.917 7.206 22.692-19.423-14.719zm37.126 37.87h23.647l-18.809 14.906 7.18 22.703-19.436-14.734-19.384 14.027 7.519-22.221-18.302-14.682h22.752l7.415-21.205 7.418 21.205z" fill="#fff" />
</g></symbol><symbol id='NFL'><desc id='NFLDesc'>NFL icon</desc><path fill="#FFFFFF" d="M167.963,22.75c-6.259,4.49-18.23,10.61-33.632,6.815C107.64,22.99,100,0,100,0     s-7.639,22.994-34.331,29.565C50.26,33.36,38.291,27.238,32.038,22.75h-4.819v123.974c0.002,3.137,0.766,9.309,5.852,15.488     c6.281,7.63,16.935,12.578,31.658,14.705c11.729,1.697,20.4,5.384,26.516,11.269C95.737,192.511,100,200,100,200     s4.426-7.646,8.756-11.813c6.115-5.885,14.791-9.571,26.516-11.269c14.723-2.127,25.377-7.075,31.659-14.705     c5.082-6.175,5.847-12.354,5.851-15.488V22.75H167.963z" /><path fill="#013369" d="M168.414,27.584c0,0-14.59,11.355-35.143,6.22C109.309,27.815,100,9.868,100,9.868     s-9.308,17.947-33.271,23.936c-20.552,5.136-35.142-6.22-35.142-6.22v119.134c0.003,4.049,1.695,21.243,33.768,25.878     c12.663,1.833,22.122,5.903,28.918,12.443c2.448,2.348,4.308,4.739,5.728,6.933c1.424-2.193,3.276-4.585,5.721-6.933     c6.804-6.54,16.263-10.61,28.926-12.443c32.07-4.635,33.762-21.829,33.768-25.878V27.584z" /><path fill="#FFFFFF" enable-background="new    " d="M100,179.844c0,0,9.308-11.473,33.495-15.204     c27.705-4.275,26.882-17.284,26.884-17.943V84.304H39.622v62.392c0,0.656-0.818,13.667,26.883,17.943     C90.691,168.372,100,179.844,100,179.844z" /><path fill="#D50A0A" d="M71.852,129.904l-0.004-29.673l-3.726-2.087V90.45H85.08v7.694l-3.546,2.087v60.527l-9.064-2.717       l-13.104-38.663l-0.002,25.489l4.161,3.196v7.855l-18.193-4.624v-6.696l4.163-1.628v-42.714l-4.265-2.112V90.45h13.768       L71.852,129.904z" /><path fill="#D50A0A" d="M103.066,157.041l3.632,4.351v7.693l-18.021-6.733v-6.531l3.978-1.629v-53.96l-3.954-2V90.45h31.291       v15.659h-6.878l-2.084-6.429h-7.964v18.106h5.977l2.352-3.438h4.796v16.112h-4.796l-2.264-3.258h-6.064V157.041z" /><path fill="#D50A0A" d="M122.96,159.765v-7.694l4.476-3.674v-48.166l-3.822-2V90.45h18.044v7.782l-3.546,2v45.188l7.754-2.095       l1.487-11.843h7.418v19.785L122.96,159.765z" /><path fill="#FFFFFF" d="M42.761,71.54L37.3,67.431h6.674l2.204-6.323l2.204,6.323h6.674l-5.461,4.11l2.009,6.418l-5.426-3.812         l-5.426,3.812L42.761,71.54z" /><path fill="#FFFFFF" d="M66.456,71.54l-5.461-4.109h6.674l2.203-6.323l2.204,6.323h6.675l-5.461,4.11l2.009,6.418l-5.427-3.812         l-5.426,3.812L66.456,71.54z" /><path fill="#FFFFFF" d="M42.761,50.156L37.3,46.047h6.674l2.204-6.324l2.204,6.324h6.674l-5.461,4.109l2.009,6.418l-5.426-3.813         l-5.426,3.813L42.761,50.156z" /><path fill="#FFFFFF" d="M66.456,50.156l-5.461-4.109h6.674l2.203-6.324l2.204,6.324h6.675l-5.461,4.109l2.009,6.418l-5.427-3.813         l-5.426,3.813L66.456,50.156z" /><path fill="#FFFFFF" d="M159.25,77.959l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,77.959z" /><path fill="#FFFFFF" d="M135.555,77.959l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,77.959z" /><path fill="#FFFFFF" d="M159.25,56.575l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,56.575z" /><path fill="#FFFFFF" d="M135.555,56.575l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,56.575z" /><path fill="#FFFFFF" d="M115.044,33.001c0,0-2.829,1.474-6.776,5.356c0.891,0.503,1.893,1.208,2.467,1.684     c-0.596,0.529-1.206,1.099-1.772,1.636c-2.568-2.117-5.243-3.192-7.315-3.698c0.688-0.493,1.418-0.978,2.14-1.448     c1.114,0.284,2.19,0.722,2.19,0.722c4.459-3.394,8.315-4.858,8.315-4.858s-0.446-0.219-1.639-0.204     c-17.528,0.227-33.678,13.314-30.629,35.706c0.326,2.375,1.249,6.694,2.422,8.588c0.391-3.727,1.79-11.973,5.946-20.808     c-1.444-0.518-2.643-0.731-2.643-0.731c0.424-0.89,1.014-2.025,1.014-2.025c3.075,0.58,5.792,1.866,8.145,3.73     c-0.565,0.912-0.947,1.508-1.216,1.977c-0.825-0.649-1.658-1.21-2.577-1.719c-4.67,8.165-6.838,15.871-7.717,19.916     c0.048,0.249,0.557,0.378,0.966,0.325c19.207-2.486,28.199-12.283,30.935-22.9c1.8-6.965,1.211-14.248-1.199-20.187     C115.932,33.645,115.603,32.952,115.044,33.001z M98.529,54.153c-2.498-2.082-5.412-3.358-8.271-3.903c0,0,0.877-1.431,1.376-2.174     c3.497,0.692,6.336,2.327,8.321,4.007C99.445,52.812,98.961,53.51,98.529,54.153z M101.683,49.725     c-2.316-1.945-5.134-3.396-8.327-4.034c0.488-0.63,1.061-1.301,1.608-1.921c2.437,0.433,5.562,1.717,8.286,3.965     C102.692,48.401,102.173,49.08,101.683,49.725z M105.194,45.5c-2.431-2.001-5.156-3.297-8.052-3.914     c0.701-0.663,1.419-1.231,2.05-1.757c3.618,0.87,6.054,2.491,7.713,3.869C106.312,44.305,105.738,44.897,105.194,45.5z" /><g>
  <path fill="#FFFFFF" d="M167.963,22.75c-6.259,4.49-18.23,10.61-33.632,6.815C107.64,22.99,100,0,100,0     s-7.639,22.994-34.331,29.565C50.26,33.36,38.291,27.238,32.038,22.75h-4.819v123.974c0.002,3.137,0.766,9.309,5.852,15.488     c6.281,7.63,16.935,12.578,31.658,14.705c11.729,1.697,20.4,5.384,26.516,11.269C95.737,192.511,100,200,100,200     s4.426-7.646,8.756-11.813c6.115-5.885,14.791-9.571,26.516-11.269c14.723-2.127,25.377-7.075,31.659-14.705     c5.082-6.175,5.847-12.354,5.851-15.488V22.75H167.963z" />
  <path fill="#013369" d="M168.414,27.584c0,0-14.59,11.355-35.143,6.22C109.309,27.815,100,9.868,100,9.868     s-9.308,17.947-33.271,23.936c-20.552,5.136-35.142-6.22-35.142-6.22v119.134c0.003,4.049,1.695,21.243,33.768,25.878     c12.663,1.833,22.122,5.903,28.918,12.443c2.448,2.348,4.308,4.739,5.728,6.933c1.424-2.193,3.276-4.585,5.721-6.933     c6.804-6.54,16.263-10.61,28.926-12.443c32.07-4.635,33.762-21.829,33.768-25.878V27.584z" />
  <path fill="#FFFFFF" enable-background="new    " d="M100,179.844c0,0,9.308-11.473,33.495-15.204     c27.705-4.275,26.882-17.284,26.884-17.943V84.304H39.622v62.392c0,0.656-0.818,13.667,26.883,17.943     C90.691,168.372,100,179.844,100,179.844z" />
  <g>
    <path fill="#D50A0A" d="M71.852,129.904l-0.004-29.673l-3.726-2.087V90.45H85.08v7.694l-3.546,2.087v60.527l-9.064-2.717       l-13.104-38.663l-0.002,25.489l4.161,3.196v7.855l-18.193-4.624v-6.696l4.163-1.628v-42.714l-4.265-2.112V90.45h13.768       L71.852,129.904z" />
    <path fill="#D50A0A" d="M103.066,157.041l3.632,4.351v7.693l-18.021-6.733v-6.531l3.978-1.629v-53.96l-3.954-2V90.45h31.291       v15.659h-6.878l-2.084-6.429h-7.964v18.106h5.977l2.352-3.438h4.796v16.112h-4.796l-2.264-3.258h-6.064V157.041z" />
    <path fill="#D50A0A" d="M122.96,159.765v-7.694l4.476-3.674v-48.166l-3.822-2V90.45h18.044v7.782l-3.546,2v45.188l7.754-2.095       l1.487-11.843h7.418v19.785L122.96,159.765z" />
  </g>
  <g>
    <g>
      <path fill="#FFFFFF" d="M42.761,71.54L37.3,67.431h6.674l2.204-6.323l2.204,6.323h6.674l-5.461,4.11l2.009,6.418l-5.426-3.812         l-5.426,3.812L42.761,71.54z" />
    </g>
    <g>
      <path fill="#FFFFFF" d="M66.456,71.54l-5.461-4.109h6.674l2.203-6.323l2.204,6.323h6.675l-5.461,4.11l2.009,6.418l-5.427-3.812         l-5.426,3.812L66.456,71.54z" />
    </g>
    <g>
      <path fill="#FFFFFF" d="M42.761,50.156L37.3,46.047h6.674l2.204-6.324l2.204,6.324h6.674l-5.461,4.109l2.009,6.418l-5.426-3.813         l-5.426,3.813L42.761,50.156z" />
    </g>
    <g>
      <path fill="#FFFFFF" d="M66.456,50.156l-5.461-4.109h6.674l2.203-6.324l2.204,6.324h6.675l-5.461,4.109l2.009,6.418l-5.427-3.813         l-5.426,3.813L66.456,50.156z" />
    </g>
  </g>
  <g>
    <g>
      <path fill="#FFFFFF" d="M159.25,77.959l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,77.959z" />
    </g>
    <g>
      <path fill="#FFFFFF" d="M135.555,77.959l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,77.959z" />
    </g>
    <g>
      <path fill="#FFFFFF" d="M159.25,56.575l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,56.575z" />
    </g>
    <g>
      <path fill="#FFFFFF" d="M135.555,56.575l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,56.575z" />
    </g>
  </g>
  <path fill="#FFFFFF" d="M115.044,33.001c0,0-2.829,1.474-6.776,5.356c0.891,0.503,1.893,1.208,2.467,1.684     c-0.596,0.529-1.206,1.099-1.772,1.636c-2.568-2.117-5.243-3.192-7.315-3.698c0.688-0.493,1.418-0.978,2.14-1.448     c1.114,0.284,2.19,0.722,2.19,0.722c4.459-3.394,8.315-4.858,8.315-4.858s-0.446-0.219-1.639-0.204     c-17.528,0.227-33.678,13.314-30.629,35.706c0.326,2.375,1.249,6.694,2.422,8.588c0.391-3.727,1.79-11.973,5.946-20.808     c-1.444-0.518-2.643-0.731-2.643-0.731c0.424-0.89,1.014-2.025,1.014-2.025c3.075,0.58,5.792,1.866,8.145,3.73     c-0.565,0.912-0.947,1.508-1.216,1.977c-0.825-0.649-1.658-1.21-2.577-1.719c-4.67,8.165-6.838,15.871-7.717,19.916     c0.048,0.249,0.557,0.378,0.966,0.325c19.207-2.486,28.199-12.283,30.935-22.9c1.8-6.965,1.211-14.248-1.199-20.187     C115.932,33.645,115.603,32.952,115.044,33.001z M98.529,54.153c-2.498-2.082-5.412-3.358-8.271-3.903c0,0,0.877-1.431,1.376-2.174     c3.497,0.692,6.336,2.327,8.321,4.007C99.445,52.812,98.961,53.51,98.529,54.153z M101.683,49.725     c-2.316-1.945-5.134-3.396-8.327-4.034c0.488-0.63,1.061-1.301,1.608-1.921c2.437,0.433,5.562,1.717,8.286,3.965     C102.692,48.401,102.173,49.08,101.683,49.725z M105.194,45.5c-2.431-2.001-5.156-3.297-8.052-3.914     c0.701-0.663,1.419-1.231,2.05-1.757c3.618,0.87,6.054,2.491,7.713,3.869C106.312,44.305,105.738,44.897,105.194,45.5z" />
</g><g>
  <path fill="#D50A0A" d="M71.852,129.904l-0.004-29.673l-3.726-2.087V90.45H85.08v7.694l-3.546,2.087v60.527l-9.064-2.717       l-13.104-38.663l-0.002,25.489l4.161,3.196v7.855l-18.193-4.624v-6.696l4.163-1.628v-42.714l-4.265-2.112V90.45h13.768       L71.852,129.904z" />
  <path fill="#D50A0A" d="M103.066,157.041l3.632,4.351v7.693l-18.021-6.733v-6.531l3.978-1.629v-53.96l-3.954-2V90.45h31.291       v15.659h-6.878l-2.084-6.429h-7.964v18.106h5.977l2.352-3.438h4.796v16.112h-4.796l-2.264-3.258h-6.064V157.041z" />
  <path fill="#D50A0A" d="M122.96,159.765v-7.694l4.476-3.674v-48.166l-3.822-2V90.45h18.044v7.782l-3.546,2v45.188l7.754-2.095       l1.487-11.843h7.418v19.785L122.96,159.765z" />
</g><g>
  <g>
    <path fill="#FFFFFF" d="M42.761,71.54L37.3,67.431h6.674l2.204-6.323l2.204,6.323h6.674l-5.461,4.11l2.009,6.418l-5.426-3.812         l-5.426,3.812L42.761,71.54z" />
  </g>
  <g>
    <path fill="#FFFFFF" d="M66.456,71.54l-5.461-4.109h6.674l2.203-6.323l2.204,6.323h6.675l-5.461,4.11l2.009,6.418l-5.427-3.812         l-5.426,3.812L66.456,71.54z" />
  </g>
  <g>
    <path fill="#FFFFFF" d="M42.761,50.156L37.3,46.047h6.674l2.204-6.324l2.204,6.324h6.674l-5.461,4.109l2.009,6.418l-5.426-3.813         l-5.426,3.813L42.761,50.156z" />
  </g>
  <g>
    <path fill="#FFFFFF" d="M66.456,50.156l-5.461-4.109h6.674l2.203-6.324l2.204,6.324h6.675l-5.461,4.109l2.009,6.418l-5.427-3.813         l-5.426,3.813L66.456,50.156z" />
  </g>
</g><g>
  <path fill="#FFFFFF" d="M42.761,71.54L37.3,67.431h6.674l2.204-6.323l2.204,6.323h6.674l-5.461,4.11l2.009,6.418l-5.426-3.812         l-5.426,3.812L42.761,71.54z" />
</g><g>
  <path fill="#FFFFFF" d="M66.456,71.54l-5.461-4.109h6.674l2.203-6.323l2.204,6.323h6.675l-5.461,4.11l2.009,6.418l-5.427-3.812         l-5.426,3.812L66.456,71.54z" />
</g><g>
  <path fill="#FFFFFF" d="M42.761,50.156L37.3,46.047h6.674l2.204-6.324l2.204,6.324h6.674l-5.461,4.109l2.009,6.418l-5.426-3.813         l-5.426,3.813L42.761,50.156z" />
</g><g>
  <path fill="#FFFFFF" d="M66.456,50.156l-5.461-4.109h6.674l2.203-6.324l2.204,6.324h6.675l-5.461,4.109l2.009,6.418l-5.427-3.813         l-5.426,3.813L66.456,50.156z" />
</g><g>
  <g>
    <path fill="#FFFFFF" d="M159.25,77.959l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,77.959z" />
  </g>
  <g>
    <path fill="#FFFFFF" d="M135.555,77.959l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,77.959z" />
  </g>
  <g>
    <path fill="#FFFFFF" d="M159.25,56.575l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,56.575z" />
  </g>
  <g>
    <path fill="#FFFFFF" d="M135.555,56.575l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,56.575z" />
  </g>
</g><g>
  <path fill="#FFFFFF" d="M159.25,77.959l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,77.959z" />
</g><g>
  <path fill="#FFFFFF" d="M135.555,77.959l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,77.959z" />
</g><g>
  <path fill="#FFFFFF" d="M159.25,56.575l-5.427-3.813l-5.426,3.813l2.008-6.419l-5.46-4.11h6.674l2.204-6.323l2.203,6.323h6.674         l-5.46,4.11L159.25,56.575z" />
</g><g>
  <path fill="#FFFFFF" d="M135.555,56.575l-5.427-3.813l-5.427,3.813l2.01-6.419l-5.462-4.11h6.675l2.204-6.323l2.204,6.323h6.674         l-5.462,4.11L135.555,56.575z" />
</g></symbol><symbol id='carousel'><desc id='carouselDesc'>Carousel Icon</desc><polygon points="10 4 10 6 4 6 4 10 2 10 2 4 10 4" /><polygon points="2 20 10 20 10 18 4 18 4 14 2 14 2 20" /><polygon points="22 4 14 4 14 6 20 6 20 10 22 10 22 4" /><polygon points="14 18 14 20 22 20 22 14 20 14 20 18 14 18" /></symbol><symbol id='list'><desc id='listDesc'>List View</desc><rect x="4" y="4" width="16" height="7" /><rect x="4" y="13" width="16" height="7" /></symbol><symbol id='website'><desc id='websiteDesc'>Website</desc><path fill-rule="evenodd" d="M8.65,16.3A16.87,16.87,0,0,1,11.6,16v4C10.38,19.7,9.3,18.29,8.65,16.3Zm1,3.34a8.26,8.26,0,0,1-1.76-3.17,8.51,8.51,0,0,0-1.94.73A8,8,0,0,0,9.62,19.64Zm4.76,0a8,8,0,0,0,3.7-2.44,8.51,8.51,0,0,0-1.94-.73A8.26,8.26,0,0,1,14.38,19.64ZM11.6,15.19V12.38H8a14.8,14.8,0,0,0,.42,3.15A18.23,18.23,0,0,1,11.6,15.19Zm0-3.61V8.76a18.27,18.27,0,0,1-3.18-.33A14.71,14.71,0,0,0,8,11.58Zm-2-7.26a8,8,0,0,0-3.7,2.44,9,9,0,0,0,1.94.73A8.26,8.26,0,0,1,9.62,4.32Zm2-.32c-1.22.26-2.3,1.67-2.95,3.66A16.87,16.87,0,0,0,11.6,8Zm.8,16c1.22-.26,2.3-1.67,2.95-3.66A16.87,16.87,0,0,0,12.4,16Zm0-11.2v2.82H16a14.71,14.71,0,0,0-.42-3.15A18.27,18.27,0,0,1,12.4,8.76ZM7.64,15.7a15,15,0,0,1-.44-3.32H4a8,8,0,0,0,1.42,4.18A9.44,9.44,0,0,1,7.64,15.7Zm4.76-3.32v2.81a18.23,18.23,0,0,1,3.18.34A14.8,14.8,0,0,0,16,12.38Zm-5.2-.8a15.06,15.06,0,0,1,.44-3.33A9,9,0,0,1,5.42,7.4,8,8,0,0,0,4,11.58Zm9.6.8a15,15,0,0,1-.44,3.32,9.44,9.44,0,0,1,2.22.86A8,8,0,0,0,20,12.38Zm-.44-4.13a15.06,15.06,0,0,1,.44,3.33H20A8,8,0,0,0,18.58,7.4,9,9,0,0,1,16.36,8.25Zm-2-3.93a8.26,8.26,0,0,1,1.76,3.17,9,9,0,0,0,1.94-.73A8,8,0,0,0,14.38,4.32ZM12.4,4V8a16.87,16.87,0,0,0,2.95-.3C14.7,5.67,13.62,4.26,12.4,4Z" /><g id="Bell">
  <rect id="Bounds" fill="none" width="24" height="24" />
</g><rect id="Bounds" fill="none" width="24" height="24" /><rect fill="none" width="24" height="24" /><rect fill="none" width="24" height="24" /></symbol><symbol id='nfl-instagram'><desc id='nfl-instagramDesc'>Instagram</desc><path d="M12,7.77a4.24,4.24,0,0,1,3.46,1.79H19V7.19A2.19,2.19,0,0,0,16.84,5H7.19A2.19,2.19,0,0,0,5,7.19V9.57H8.55A4.24,4.24,0,0,1,12,7.77ZM16,6.53a.39.39,0,0,1,.39-.39h1.17a.39.39,0,0,1,.39.39V7.71a.39.39,0,0,1-.39.39H16.34A.39.39,0,0,1,16,7.71Z" /><path d="M16.26,12a4.24,4.24,0,1,1-8.4-.82H5v5.64A2.19,2.19,0,0,0,7.19,19h9.65A2.19,2.19,0,0,0,19,16.84V11.2H16.18A4.35,4.35,0,0,1,16.26,12Z" /><rect fill="none" width="24" height="24" /><circle cx="12.02" cy="12.01" r="2.61" /></symbol><symbol id='nfl-twitter'><desc id='nfl-twitterDesc'>Twitter</desc><path fill-rule="evenodd" d="M18.92,7.71a5.66,5.66,0,0,1-1.63.45,2.84,2.84,0,0,0,1.25-1.57,5.68,5.68,0,0,1-1.8.69,2.84,2.84,0,0,0-4.91,1.94,2.87,2.87,0,0,0,.07.65A8.06,8.06,0,0,1,6,6.89a2.84,2.84,0,0,0,.88,3.79,2.83,2.83,0,0,1-1.29-.36v0a2.84,2.84,0,0,0,2.28,2.78,2.82,2.82,0,0,1-1.28,0,2.84,2.84,0,0,0,2.65,2,5.7,5.7,0,0,1-3.53,1.22,5.84,5.84,0,0,1-.68,0A8.08,8.08,0,0,0,17.51,9.54c0-.12,0-.25,0-.37a5.78,5.78,0,0,0,1.42-1.47" /><rect fill="none" width="24" height="24" /><rect fill="none" width="24" height="24" /></symbol><symbol id='nfl-facebook'><desc id='nfl-facebookDesc'>Facebook</desc><path fill-rule="evenodd" d="M14.86,12l.32-2.33H12.72V8.54c0-.72.2-1.21,1.23-1.21h1.32V5.1A17.59,17.59,0,0,0,13.35,5a3,3,0,0,0-3.2,3.29V9.67H8V12h2.15v7h2.57V12h2.14" /><rect fill="none" width="24" height="24" /></symbol><symbol id='nfl-snapchat'><desc id='nfl-snapchatDesc'>Snapchat</desc><path d="M12.16,4.38h-.32A4.31,4.31,0,0,0,7.83,7a8.11,8.11,0,0,0-.2,3.28l0,.39a.65.65,0,0,1-.32.07,1.81,1.81,0,0,1-.75-.2.6.6,0,0,0-.26-.05.8.8,0,0,0-.79.54c0,.23.06.57.81.87l.24.08c.31.1.78.25.91.55a.72.72,0,0,1-.08.59v0a4.77,4.77,0,0,1-3.24,2.72.34.34,0,0,0-.28.35.45.45,0,0,0,0,.15c.17.39.86.67,2.13.87a1.58,1.58,0,0,1,.11.38c0,.12.05.25.09.38a.38.38,0,0,0,.39.29A2,2,0,0,0,7,18.2a4.28,4.28,0,0,1,.86-.1,3.81,3.81,0,0,1,.62.05,2.94,2.94,0,0,1,1.16.6,3.66,3.66,0,0,0,2.23.87h.2a3.66,3.66,0,0,0,2.23-.87,2.94,2.94,0,0,1,1.16-.6,3.8,3.8,0,0,1,.62-.05,4.31,4.31,0,0,1,.86.09,2,2,0,0,0,.37,0h0a.37.37,0,0,0,.37-.28c0-.13.07-.25.09-.38a1.57,1.57,0,0,1,.11-.38c1.27-.2,2-.48,2.13-.87a.45.45,0,0,0,0-.15.34.34,0,0,0-.28-.35,4.77,4.77,0,0,1-3.24-2.72v0a.72.72,0,0,1-.08-.59c.13-.3.6-.45.91-.55l.24-.08c.55-.22.83-.48.82-.79a.66.66,0,0,0-.49-.57h0a.9.9,0,0,0-.34-.06.75.75,0,0,0-.31.06,1.89,1.89,0,0,1-.7.2.62.62,0,0,1-.27-.07l0-.35v0A8.11,8.11,0,0,0,16.17,7a4.32,4.32,0,0,0-4-2.59Z" /><rect fill="none" width="24" height="24" /></symbol><symbol id='shop'><desc id='shopDesc'>Shop Icon</desc><path d="M12,6,9,4H6v6H7V20H17V10h1V4H15Zm0,1a.75.75,0,1,1-.75.75A.75.75,0,0,1,12,7Zm2,3.7a.35.35,0,0,1,0,.11l-2,5.11a.12.12,0,0,1-.13.08H10.66c-.06,0-.08,0-.06-.1l2-5.1v0H11.28a0,0,0,0,0-.06.06v.67a.09.09,0,0,1-.09.09h-1a.09.09,0,0,1-.09-.09V9.72a.09.09,0,0,1,.09-.09h3.81a.09.09,0,0,1,.09.09Z" /><rect x="3" y="4" width="2" height="6" /><rect x="19" y="4" width="2" height="6" /></symbol><symbol id='profile-header__overlay'><desc id='profile-header__overlayDesc'>Profile Overlay</desc><g>
  <polygon class="profile-header__overlay-stripe" points="109.818 0 16 225 0 225 93.818 0" />
  <polygon class="profile-header__overlay-polygon" points="16 225 109.818 0 800 0 800 225" />
</g><polygon class="profile-header__overlay-stripe" points="109.818 0 16 225 0 225 93.818 0" /><polygon class="profile-header__overlay-polygon" points="16 225 109.818 0 800 0 800 225" /></symbol><symbol id='nfl-avatar'><desc id='nfl-avatarDesc'>Avatar</desc><path d="M12,2A10,10,0,1,0,22,12,10,10,0,0,0,12,2Zm5.22,16-.88-2.19a2,2,0,0,0-1.22-1.15l-1.17-.39a6.16,6.16,0,0,0-3.9,0l-1.17.39a2,2,0,0,0-1.22,1.15L6.78,18a8,8,0,1,1,10.44,0Z" /><circle cx="12" cy="10" r="3" /></symbol><symbol id='nfl-add'><desc id='nfl-addDesc'>Add</desc><polygon points="19 11 13 11 13 5 11 5 11 11 5 11 5 13 11 13 11 19 13 19 13 13 19 13 19 11" /></symbol><symbol id='nfl-airplay'><desc id='nfl-airplayDesc'>Airplay</desc><polygon points="12 13 4 21 20 21 12 13" /><polygon points="2 4 22 4 22 18 19 18 17 16 20 16 20 6 4 6 4 16 7 16 5 18 2 18 2 4" /></symbol><symbol id='nfl-arrow-left'><desc id='nfl-arrow-leftDesc'>Arrow Left</desc><polygon points="19 11 7.83 11 11.12 7.71 9.71 6.29 4 12 9.71 17.71 11.12 16.29 7.83 13 19 13 19 11" /></symbol><symbol id='nfl-arrow-right'><desc id='nfl-arrow-rightDesc'>Arrow Right</desc><polygon points="14.29 6.29 12.88 7.71 16.17 11 5 11 5 13 16.17 13 12.88 16.29 14.29 17.71 20 12 14.29 6.29" /></symbol><symbol id='nfl-arrow-up'><desc id='nfl-arrow-upDesc'>Arrow Up</desc><polygon points="17.71 9.71 12 4 6.29 9.71 7.71 11.12 11 7.83 11 19 13 19 13 7.83 16.29 11.12 17.71 9.71" /></symbol><symbol id='nfl-arrow-down'><desc id='nfl-arrow-downDesc'>Arrow Down</desc><polygon points="16.29 12.88 13 16.17 13 5 11 5 11 16.17 7.71 12.88 6.29 14.29 12 20 17.71 14.29 16.29 12.88" /></symbol><symbol id='nfl-audio'><desc id='nfl-audioDesc'>Audio</desc><path d="M20,12A8,8,0,0,0,4,12a7.88,7.88,0,0,0,.83,3.53l-.24,0,.67,3.95,1.09-.19.16,1,2.92-.49-1-5.92-2,.34A5.84,5.84,0,0,1,6,12a6,6,0,0,1,12,0,5.84,5.84,0,0,1-.43,2.21l-2-.34-1,5.92,2.92.49.16-1,1.09.19.67-3.95-.24,0A7.88,7.88,0,0,0,20,12Z" /></symbol><symbol id='nfl-back-5s'><desc id='nfl-back-5sDesc'>Back 5s</desc><path d="M12.82,12.71c0-.49-.23-.62-.77-.62a1,1,0,0,0-.88.37l-1-.22V9.43h3.49v1.09H11.26v.87a1.69,1.69,0,0,1,1.15-.32,1.46,1.46,0,0,1,1.65,1.64v.22c0,1.18-.78,1.64-2.19,1.64a3.82,3.82,0,0,1-2-.48l.41-.92a3.07,3.07,0,0,0,1.49.4c.65,0,1-.11,1-.62Z" /><path d="M12,2A9.93,9.93,0,0,0,4.94,4.94L3,3V8H8L6.36,6.36a8,8,0,1,1,0,11.3L4.93,19.07A10,10,0,1,0,12,2Z" /></symbol><symbol id='nfl-back-10s'><desc id='nfl-back-10sDesc'>Back 10s</desc><path d="M10.47,14.56H9.2V10.43l-.82.06v-.8l1.23-.32h.85Z" /><path d="M15.49,11.44v1.11c0,1.49-.54,2.07-2,2.07s-2-.58-2-2.07V11.44c0-1.49.54-2.07,2-2.07S15.49,10,15.49,11.44Zm-2.74.16v.8c0,.84,0,1.18.73,1.18s.73-.34.73-1.18v-.8c0-.84,0-1.18-.73-1.18S12.75,10.76,12.75,11.6Z" /><path d="M12,2A9.93,9.93,0,0,0,4.94,4.94L3,3V8H8L6.36,6.36a8,8,0,1,1,0,11.3L4.93,19.07A10,10,0,1,0,12,2Z" /></symbol><symbol id='nfl-back-30s'><desc id='nfl-back-30sDesc'>Back 30s</desc><path d="M7.53,14.14l.41-.93a3.19,3.19,0,0,0,1.53.4c.69,0,1-.16,1-.58h0c0-.38-.17-.59-.76-.59H8.54v-1h1c.49,0,.65-.21.65-.52h0c0-.27-.12-.5-.7-.5a3.05,3.05,0,0,0-1.44.37l-.41-.93a3.79,3.79,0,0,1,2-.46c1.42,0,1.81.6,1.81,1.51v0a1.11,1.11,0,0,1-.64,1,1.13,1.13,0,0,1,.86,1.18h0c0,1-.73,1.54-2.19,1.54A3.84,3.84,0,0,1,7.53,14.14Z" /><path d="M16.29,11.44v1.11c0,1.49-.54,2.07-2,2.07s-2-.58-2-2.07V11.44c0-1.49.54-2.07,2-2.07S16.29,10,16.29,11.44Zm-2.74.16v.8c0,.84,0,1.18.73,1.18s.73-.34.73-1.18v-.8c0-.84,0-1.18-.73-1.18S13.55,10.76,13.55,11.6Z" /><path d="M12,2A9.93,9.93,0,0,0,4.94,4.94L3,3V8H8L6.36,6.36a8,8,0,1,1,0,11.3L4.93,19.07A10,10,0,1,0,12,2Z" /></symbol><symbol id='nfl-calendar'><desc id='nfl-calendarDesc'>Calendar</desc><path d="M18,5V3H16V5H8V3H6V5H4V19H20V5Zm1,13H5V8H19Z" /><rect x="7" y="10" width="2" height="2" /><rect x="11" y="10" width="2" height="2" /><rect x="15" y="10" width="2" height="2" /><rect x="7" y="14" width="2" height="2" /><rect x="11" y="14" width="2" height="2" /><rect x="15" y="14" width="2" height="2" /></symbol><symbol id='nfl-chart'><desc id='nfl-chartDesc'>Chart</desc><rect x="6" y="12" width="2" height="8" /><rect x="16" y="8" width="2" height="12" /><rect x="11" y="4" width="2" height="16" /></symbol><symbol id='nfl-check'><desc id='nfl-checkDesc'>Check</desc><path d="M12,2A10,10,0,1,0,22,12,10,10,0,0,0,12,2ZM10,16.414,6.293,12.707l1.414-1.414L10,13.586l6.293-6.293,1.414,1.414Z" /></symbol><symbol id='nfl-chevron-down'><desc id='nfl-chevron-downDesc'>Down</desc><path d="m12 17-6.71-6.71 1.42-1.41 5.29 5.29 5.29-5.29 1.42 1.41z" /><path d="m0 0h24v24h-24z" fill="none" /></symbol><symbol id='nfl-chevron-left'><desc id='nfl-chevron-leftDesc'>Left</desc><polygon points="13.71 18.71 7 12 13.71 5.29 15.12 6.71 9.83 12 15.12 17.29 13.71 18.71" /></symbol><symbol id='nfl-chevron-right'><desc id='nfl-chevron-rightDesc'>Right</desc><polygon points="10.29 18.71 8.88 17.29 14.17 12 8.88 6.71 10.29 5.29 17 12 10.29 18.71" /></symbol><symbol id='nfl-chevron-up'><desc id='nfl-chevron-upDesc'>Up</desc><polygon points="17.29 15.12 12 9.83 6.71 15.12 5.29 13.71 12 7 18.71 13.71 17.29 15.12" /></symbol><symbol id='nfl-chromecast-off'><desc id='nfl-chromecast-offDesc'>Chromecast Off</desc><path d="M6,21H8a6,6,0,0,0-6-6v2A4,4,0,0,1,6,21Z" /><path d="M10,21h2A10,10,0,0,0,2,11v2A8,8,0,0,1,10,21Z" /><path d="M3,3V9.05a11.9,11.9,0,0,1,2,.34V5H20V18H13.61a11.9,11.9,0,0,1,.34,2H22V3Z" /><path d="M4,21a2,2,0,0,0-2-2v2Z" /></symbol><symbol id='nfl-chromecast-on'><desc id='nfl-chromecast-onDesc'>Chromecast On</desc><path d="M6,21H8a6,6,0,0,0-6-6v2A4,4,0,0,1,6,21Z" /><path d="M10,21h2A10,10,0,0,0,2,11v2A8,8,0,0,1,10,21Z" /><path d="M3,3V9.05a11.9,11.9,0,0,1,2,.34V5H20V18H13.61a11.9,11.9,0,0,1,.34,2H22V3Z" /><path d="M7,7v3.11A12.06,12.06,0,0,1,12.89,16H18V7Z" /><path d="M4,21a2,2,0,0,0-2-2v2Z" /></symbol><symbol id='nfl-close'><desc id='nfl-closeDesc'>Close</desc><polygon points="13.41 12 18.21 7.21 16.79 5.79 12 10.59 7.21 5.79 5.79 7.21 10.59 12 5.79 16.79 7.21 18.21 12 13.41 16.79 18.21 18.21 16.79 13.41 12" /></symbol><symbol id='nfl-closed-captions'><desc id='nfl-closed-captionsDesc'>Closed Captions</desc><path d="M10,14.35a3.08,3.08,0,0,0,1.85-.53l-.46-1a2.5,2.5,0,0,1-1.41.41c-.87,0-1-.44-1-1V10.84c0-.56.18-1,1-1a2.5,2.5,0,0,1,1.41.41l.46-1A3.08,3.08,0,0,0,10,8.69c-1.77,0-2.35.78-2.35,2v1.57C7.6,13.57,8.18,14.35,10,14.35Z" /><path d="M14.55,14.35a3.08,3.08,0,0,0,1.85-.53l-.46-1a2.5,2.5,0,0,1-1.41.41c-.87,0-1-.44-1-1V10.84c0-.56.18-1,1-1a2.5,2.5,0,0,1,1.41.41l.46-1a3.08,3.08,0,0,0-1.85-.53c-1.77,0-2.35.78-2.35,2v1.57C12.2,13.57,12.78,14.35,14.55,14.35Z" /><path d="M3,5V18H15v3l3-3h3V5ZM19,16H5V7H19Z" /></symbol><symbol id='nfl-bench-off'><desc id='nfl-bench-offDesc'>Bench Off</desc><path d="M14,20.25a1.2,1.2,0,0,1-1.25-1.139V4.889a1.255,1.255,0,0,1,2.5,0V10.75h7v2.5h-7v5.861A1.2,1.2,0,0,1,14,20.25Zm0-16a.7.7,0,0,0-.75.639V19.111a.76.76,0,0,0,1.5,0V4.889A.7.7,0,0,0,14,4.25Zm1.25,8.5h6.5v-1.5h-6.5ZM10,20.25a1.2,1.2,0,0,1-1.25-1.139V4.889A1.2,1.2,0,0,1,10,3.75a1.2,1.2,0,0,1,1.25,1.139V19.111A1.2,1.2,0,0,1,10,20.25Zm0-16a.7.7,0,0,0-.75.639V19.111a.7.7,0,0,0,.75.639.7.7,0,0,0,.75-.639V4.889A.7.7,0,0,0,10,4.25Zm-4,16a1.2,1.2,0,0,1-1.25-1.139V13.25H3a1.25,1.25,0,0,1,0-2.5H4.75V4.889A1.2,1.2,0,0,1,6,3.75,1.2,1.2,0,0,1,7.25,4.889V19.111A1.2,1.2,0,0,1,6,20.25Zm0-16a.7.7,0,0,0-.75.639V19.111A.7.7,0,0,0,6,19.75a.7.7,0,0,0,.75-.639V4.889A.7.7,0,0,0,6,4.25Zm-3,7a.75.75,0,0,0,0,1.5H4.75v-1.5Z" /></symbol><symbol id='nfl-bench-on'><desc id='nfl-bench-onDesc'>Bench On</desc><path d="M7,4.889V19.111A.95.95,0,0,1,6,20a.95.95,0,0,1-1-.889V4.889A.95.95,0,0,1,6,4,.95.95,0,0,1,7,4.889ZM10,4a.95.95,0,0,0-1,.889V19.111A.95.95,0,0,0,10,20a.95.95,0,0,0,1-.889V4.889A.95.95,0,0,0,10,4Zm4,0a.95.95,0,0,0-1,.889V19.111a1.007,1.007,0,0,0,2,0V4.889A.95.95,0,0,0,14,4ZM2,12a1,1,0,0,0,1,1H4V11H3A1,1,0,0,0,2,12Zm14-1v2h6V11Z" /></symbol><symbol id='nfl-broad-off'><desc id='nfl-broad-offDesc'>Broad Off</desc><path d="M21.657,21.25H4.743a1.256,1.256,0,0,1-1.2-.891l-2.4-8a1.248,1.248,0,0,1,1.2-1.608H19.257a1.256,1.256,0,0,1,1.2.891l2.4,8a1.249,1.249,0,0,1-1.2,1.608Zm-19.314-10a.75.75,0,0,0-.718.964l2.4,8a.754.754,0,0,0,.717.534H21.657a.75.75,0,0,0,.718-.964l-2.4-8a.754.754,0,0,0-.717-.534ZM21,20.25a1.629,1.629,0,0,1-1.335-1.1l-2.3-6.127a1.062,1.062,0,0,1,.05-.98A.684.684,0,0,1,18,11.75a1.629,1.629,0,0,1,1.335,1.1l2.3,6.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,21,20.25Zm-3-8a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l2.3,6.127A1.159,1.159,0,0,0,21,19.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-2.3-6.127A1.159,1.159,0,0,0,18,12.25Zm-2,8a1.629,1.629,0,0,1-1.335-1.1l-2.3-6.127a1.062,1.062,0,0,1,.05-.98A.684.684,0,0,1,13,11.75a1.629,1.629,0,0,1,1.335,1.1l2.3,6.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,16,20.25Zm-3-8a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l2.3,6.127A1.159,1.159,0,0,0,16,19.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-2.3-6.127A1.159,1.159,0,0,0,13,12.25Zm-2,8a1.629,1.629,0,0,1-1.335-1.1l-2.3-6.127a1.062,1.062,0,0,1,.05-.98A.684.684,0,0,1,8,11.75a1.629,1.629,0,0,1,1.335,1.1l2.3,6.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,11,20.25Zm-3-8a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l2.3,6.127A1.159,1.159,0,0,0,11,19.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-2.3-6.127A1.159,1.159,0,0,0,8,12.25Zm-2,8a1.629,1.629,0,0,1-1.335-1.1l-2.3-6.127a1.062,1.062,0,0,1,.05-.98A.684.684,0,0,1,3,11.75a1.629,1.629,0,0,1,1.335,1.1l2.3,6.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,6,20.25Zm-3-8a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l2.3,6.127A1.159,1.159,0,0,0,6,19.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-2.3-6.127A1.159,1.159,0,0,0,3,12.25Zm14,4a1.629,1.629,0,0,1-1.335-1.1l-.8-2.127a1.062,1.062,0,0,1,.05-.98.684.684,0,0,1,.583-.294,1.629,1.629,0,0,1,1.335,1.1l.8,2.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,17,16.25Zm-1.5-4a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l.8,2.127A1.159,1.159,0,0,0,17,15.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-.8-2.127A1.159,1.159,0,0,0,15.5,12.25Zm-3.5,4a1.629,1.629,0,0,1-1.335-1.1l-.8-2.127a1.062,1.062,0,0,1,.05-.98.684.684,0,0,1,.583-.294,1.629,1.629,0,0,1,1.335,1.1l.8,2.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,12,16.25Zm-1.5-4a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l.8,2.127A1.159,1.159,0,0,0,12,15.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-.8-2.127A1.159,1.159,0,0,0,10.5,12.25Zm-3.5,4a1.629,1.629,0,0,1-1.335-1.1l-.8-2.127a1.062,1.062,0,0,1,.05-.98A.684.684,0,0,1,5.5,11.75a1.629,1.629,0,0,1,1.335,1.1l.8,2.127a1.06,1.06,0,0,1-.05.979A.683.683,0,0,1,7,16.25Zm-1.5-4a.19.19,0,0,0-.173.079.6.6,0,0,0,.008.52l.8,2.127A1.159,1.159,0,0,0,7,15.75a.19.19,0,0,0,.173-.079.6.6,0,0,0-.008-.52l-.8-2.127A1.159,1.159,0,0,0,5.5,12.25ZM3,10.472a.978.978,0,0,1-.438-.1A1,1,0,0,1,2.1,9.044,12.2,12.2,0,0,1,3.334,7.2a.993.993,0,0,1,.671-.37,1.009,1.009,0,0,1,.735.213,1,1,0,0,1,.158,1.4,10.267,10.267,0,0,0-1,1.473A1,1,0,0,1,3,10.472ZM4.117,7.323a.525.525,0,0,0-.056,0,.5.5,0,0,0-.336.186A11.435,11.435,0,0,0,2.549,9.261a.492.492,0,0,0,.232.66.507.507,0,0,0,.67-.228A10.466,10.466,0,0,1,4.507,8.135a.5.5,0,0,0-.39-.812Zm11.977.654a1.058,1.058,0,0,1-.991-.7.911.911,0,0,1,.641-1.192l.867-.224c-.281-.144-.541-.267-.787-.373A1,1,0,0,1,15.3,4.177a1,1,0,0,1,1.312-.527c.309.132.634.287.983.468l-.307-.894a.912.912,0,0,1,.641-1.193,1.058,1.058,0,0,1,1.239.666l1.1,3.209A.911.911,0,0,1,19.625,7.1l-3.284.846A.952.952,0,0,1,16.094,7.977Zm.123-3.907a.5.5,0,0,0-.46.3.494.494,0,0,0,0,.383.487.487,0,0,0,.268.272c.417.18.874.408,1.395.7l.547.3-2.094.54a.43.43,0,0,0-.271.206.422.422,0,0,0-.023.34.546.546,0,0,0,.641.345L19.5,6.614a.412.412,0,0,0,.294-.546l-1.1-3.208a.551.551,0,0,0-.641-.344.412.412,0,0,0-.294.546l.728,2.118-.571-.32a16.2,16.2,0,0,0-1.5-.75A.52.52,0,0,0,16.217,4.07ZM5.614,7.23A1,1,0,0,1,4.87,6.9a1,1,0,0,1,.076-1.412,11.545,11.545,0,0,1,3.21-2.069,1,1,0,0,1,.777,1.844A9.513,9.513,0,0,0,6.282,6.975,1,1,0,0,1,5.614,7.23ZM8.543,3.839a.508.508,0,0,0-.193.039,11.071,11.071,0,0,0-3.07,1.98.5.5,0,0,0-.038.706.516.516,0,0,0,.706.039A10.016,10.016,0,0,1,8.739,4.8a.5.5,0,0,0-.2-.961Zm5.582,1.047a1.017,1.017,0,0,1-.22-.025A8.452,8.452,0,0,0,10.8,4.729a.966.966,0,0,1-.74-.179,1,1,0,0,1,.438-1.8,10.419,10.419,0,0,1,3.85.157,1,1,0,0,1-.219,1.976Zm-2.146-.744a9.289,9.289,0,0,1,2.036.231.5.5,0,0,0,.22-.975,9.924,9.924,0,0,0-3.666-.151.5.5,0,0,0-.327.2.5.5,0,0,0,.109.7.489.489,0,0,0,.368.089A8.5,8.5,0,0,1,11.979,4.142Z" /></symbol><symbol id='nfl-broad-on'><desc id='nfl-broad-onDesc'>Broad On</desc><path d="M22.614,19.714l-2.4-8A1,1,0,0,0,19.257,11H2.343a1,1,0,0,0-.957,1.286l2.4,8A1,1,0,0,0,4.743,21H21.657A1,1,0,0,0,22.614,19.714ZM6,20a1.4,1.4,0,0,1-1.1-.936L2.6,12.936C2.408,12.421,2.588,12,3,12a1.393,1.393,0,0,1,1.1.936l2.3,6.128C6.592,19.579,6.413,20,6,20Zm1-4a1.4,1.4,0,0,1-1.1-.936l-.8-2.128c-.193-.515-.013-.936.4-.936a1.393,1.393,0,0,1,1.1.936l.8,2.128C7.592,15.579,7.413,16,7,16Zm4,4a1.4,1.4,0,0,1-1.1-.936L7.6,12.936C7.408,12.421,7.588,12,8,12a1.393,1.393,0,0,1,1.1.936l2.3,6.128C11.592,19.579,11.413,20,11,20Zm1-4a1.4,1.4,0,0,1-1.1-.936l-.8-2.128c-.193-.515-.013-.936.4-.936a1.393,1.393,0,0,1,1.1.936l.8,2.128C12.592,15.579,12.413,16,12,16Zm4,4a1.4,1.4,0,0,1-1.1-.936l-2.3-6.128c-.193-.515-.013-.936.4-.936a1.393,1.393,0,0,1,1.1.936l2.3,6.128C16.592,19.579,16.413,20,16,20Zm1-4a1.4,1.4,0,0,1-1.1-.936l-.8-2.128c-.193-.515-.013-.936.4-.936a1.393,1.393,0,0,1,1.1.936l.8,2.128C17.592,15.579,17.413,16,17,16Zm4,4a1.4,1.4,0,0,1-1.1-.936l-2.3-6.128c-.193-.515-.013-.936.4-.936a1.393,1.393,0,0,1,1.1.936l2.3,6.128C21.592,19.579,21.413,20,21,20ZM3,10.222a.748.748,0,0,1-.675-1.07,11.658,11.658,0,0,1,1.206-1.8A.75.75,0,0,1,4.7,8.291,10.4,10.4,0,0,0,3.674,9.806.758.758,0,0,1,3,10.222ZM5.614,6.98a.749.749,0,0,1-.5-1.307,11.3,11.3,0,0,1,3.14-2.026A.75.75,0,0,1,8.836,5.03,9.786,9.786,0,0,0,6.115,6.788.744.744,0,0,1,5.614,6.98Zm8.511-2.344a.727.727,0,0,1-.165-.019,8.686,8.686,0,0,0-3.2-.135A.75.75,0,0,1,10.532,3a10.15,10.15,0,0,1,3.758.154.75.75,0,0,1-.165,1.482ZM18,6.227a.747.747,0,0,1-.379-.1,15.292,15.292,0,0,0-1.7-.863.75.75,0,1,1,.589-1.379,16.611,16.611,0,0,1,1.868.949A.75.75,0,0,1,18,6.227Zm-2.661.97a.661.661,0,0,1,.467-.869l2.581-.665-.866-2.52a.661.661,0,0,1,.468-.869.8.8,0,0,1,.939.506l1.1,3.207a.661.661,0,0,1-.467.869L16.279,7.7A.8.8,0,0,1,15.339,7.2Z" /></symbol><symbol id='nfl-vertical-off'><desc id='nfl-vertical-offDesc'>Vertical Off</desc><path d="M12,10.25H3a1.25,1.25,0,0,1,0-2.5h9a1.25,1.25,0,0,1,0,2.5Zm-9-2a.75.75,0,0,0,0,1.5h9a.75.75,0,0,0,0-1.5Zm7,7H3a1.25,1.25,0,0,1,0-2.5h7a1.25,1.25,0,0,1,0,2.5Zm-7-2a.75.75,0,0,0,0,1.5h7a.75.75,0,0,0,0-1.5Zm5,7H3a1.25,1.25,0,0,1,0-2.5H8a1.25,1.25,0,0,1,0,2.5Zm-5-2a.75.75,0,0,0,0,1.5H8a.75.75,0,0,0,0-1.5Zm14-4a1,1,0,0,1-1-1V9.75a1,1,0,0,1,2,0v3.5A1,1,0,0,1,17,14.25Zm0-5a.5.5,0,0,0-.5.5v3.5a.5.5,0,0,0,1,0V9.75A.5.5,0,0,0,17,9.25Zm0,11a1,1,0,0,1-1-1v-3.5a1,1,0,0,1,2,0v3.5A1,1,0,0,1,17,20.25Zm0-5a.5.5,0,0,0-.5.5v3.5a.5.5,0,0,0,1,0v-3.5A.5.5,0,0,0,17,15.25Zm0-7a1,1,0,0,1-1-1V6.272l-.625.677a.935.935,0,0,1-1.354,0,1.053,1.053,0,0,1,0-1.407l2.3-2.491A.913.913,0,0,1,17,2.75h0a.914.914,0,0,1,.677.3l2.3,2.491a1.053,1.053,0,0,1,0,1.407.935.935,0,0,1-1.354,0L18,6.272V7.25A1,1,0,0,1,17,8.25ZM16.5,5V7.25a.5.5,0,0,0,1,0V5l1.492,1.614a.42.42,0,0,0,.62,0,.55.55,0,0,0,0-.727l-2.3-2.491A.422.422,0,0,0,17,3.25h0a.42.42,0,0,0-.309.141l-2.3,2.491a.55.55,0,0,0,0,.727.42.42,0,0,0,.62,0Zm-3.14.255H3a1.25,1.25,0,0,1,0-2.5H15.646ZM3,3.25a.75.75,0,0,0,0,1.5H13.116L14.5,3.25Zm18,2h-.36l-.119-.161L18.354,2.75H21a1.25,1.25,0,0,1,0,2.5Zm-.116-.5H21a.75.75,0,0,0,0-1.5H19.5Z" /></symbol><symbol id='nfl-vertical-on'><desc id='nfl-vertical-onDesc'>Vertical On</desc><path d="M13,9a1,1,0,0,1-1,1H3A1,1,0,0,1,3,8h9A1,1,0,0,1,13,9Zm-3,4H3a1,1,0,0,0,0,2h7a1,1,0,0,0,0-2ZM8,18H3a1,1,0,0,0,0,2H8a1,1,0,0,0,0-2Zm9-9a.75.75,0,0,0-.75.75v3.5a.75.75,0,0,0,1.5,0V9.75A.75.75,0,0,0,17,9Zm0,6a.75.75,0,0,0-.75.75v3.5a.75.75,0,0,0,1.5,0v-3.5A.75.75,0,0,0,17,15Zm.493-11.779a.66.66,0,0,0-.986,0L14.2,5.712a.8.8,0,0,0,0,1.067.661.661,0,0,0,.987,0L16.25,5.633V7.25a.75.75,0,0,0,1.5,0V5.633l1.059,1.146a.661.661,0,0,0,.987,0,.8.8,0,0,0,0-1.067ZM13.272,4.949,15.074,3H3A1,1,0,0,0,3,5H13.241C13.254,4.984,13.258,4.964,13.272,4.949ZM21,3H18.926l1.8,1.949c.014.015.018.035.031.051H21a1,1,0,0,0,0-2Z" /></symbol><symbol id='nfl-comment'><desc id='nfl-commentDesc'>Comment</desc><path d="M3,4V18H6l3,3V18H21V4ZM5,6H19V16H5Z" /><rect x="7" y="8" width="10" height="2" /><rect x="7" y="12" width="8" height="2" /></symbol><symbol id='nfl-dock'><desc id='nfl-dockDesc'>Dock</desc><path d="M2,4V20H22V4ZM20,18H4V6H20Z" /><rect x="11" y="11" width="7" height="5" /></symbol><symbol id='nfl-done'><desc id='nfl-doneDesc'>Done</desc><polygon points="9 17.414 4.293 12.707 5.707 11.293 9 14.586 17.293 6.293 18.707 7.707 9 17.414" /></symbol><symbol id='nfl-download'><desc id='nfl-downloadDesc'>Download</desc><polygon points="13 14 13 4 11 4 11 14 8 14 12 18 16 14 13 14" /><rect x="2" y="20" width="20" height="2" /></symbol><symbol id='nfl-draft'><desc id='nfl-draftDesc'>Draft</desc><path d="M21,7a26.91,26.91,0,0,0-5.25-1.84V8H21Z" /><path d="M22,9H15.71a2.3,2.3,0,0,1-.41,1,2.64,2.64,0,0,1-1.81,1,.94.94,0,0,0-.51.2,2.28,2.28,0,0,0-.32.32l-.66,1-.61-1a2.42,2.42,0,0,0-.36-.32,1,1,0,0,0-.52-.21A2.67,2.67,0,0,1,8.7,10a2.3,2.3,0,0,1-.41-1H2v6H22Z" /><path d="M18,16H6a21.91,21.91,0,0,0,5.65,4.8L12,21l.35-.2C15.42,19,17.2,17.35,18,16Z" /><path d="M8.25,5.17A31.83,31.83,0,0,0,3,7V8H8.25Z" /><path d="M10.59,4.18A1.73,1.73,0,0,1,9.2,3.91H9v5a1,1,0,0,0,.24.62,2,2,0,0,0,1.3.58,2,2,0,0,1,1.09.45A2.48,2.48,0,0,1,12,11a2.94,2.94,0,0,1,.36-.47,1.93,1.93,0,0,1,1.09-.45,2,2,0,0,0,1.3-.59A1,1,0,0,0,15,8.86v-5h-.2a1.7,1.7,0,0,1-1.38.27A2,2,0,0,1,12,3,2,2,0,0,1,10.59,4.18Z" /></symbol><symbol id='nfl-fantasy'><desc id='nfl-fantasyDesc'>Fantasy</desc><path d="M16.91,5,17,4H7l.09,1H3a9,9,0,0,0,4.81,8L8,15l3,1.5V19H9L7,21H17l-2-2H13V16.5L16,15l.19-2A9,9,0,0,0,21,5ZM5.29,7h2l.31,3.4A7,7,0,0,1,5.29,7Zm11.12,3.4L16.73,7h2A7,7,0,0,1,16.42,10.4Z" /></symbol><symbol id='nfl-filter'><desc id='nfl-filterDesc'>Filter</desc><polygon points="13 6 13 4 11 4 11 6 9.5 6 9.5 8 14.5 8 14.5 6 13 6" /><polygon points="7 15 7 4 5 4 5 15 3.5 15 3.5 17 8.5 17 8.5 15 7 15" /><polygon points="19 11 19 4 17 4 17 11 15.5 11 15.5 13 20.5 13 20.5 11 19 11" /><rect x="11" y="9" width="2" height="11" /><rect x="5" y="18" width="2" height="2" /><rect x="17" y="14" width="2" height="6" /></symbol><symbol id='nfl-forward-5s'><desc id='nfl-forward-5sDesc'>Forward 5s</desc><path d="M12.82,12.71c0-.49-.23-.62-.77-.62a1,1,0,0,0-.88.37l-1-.22V9.43h3.49v1.09H11.26v.87a1.69,1.69,0,0,1,1.15-.32,1.46,1.46,0,0,1,1.65,1.64v.22c0,1.18-.78,1.64-2.19,1.64a3.82,3.82,0,0,1-2-.48l.41-.92a3.07,3.07,0,0,0,1.49.4c.65,0,1-.11,1-.62Z" /><path d="M12,2a9.93,9.93,0,0,1,7.06,2.94L21,3V8H16l1.64-1.64a8,8,0,1,0,0,11.3l1.42,1.42A10,10,0,1,1,12,2Z" /></symbol><symbol id='nfl-forward-10s'><desc id='nfl-forward-10sDesc'>Forward 10s</desc><path d="M10.47,14.56H9.2V10.43l-.82.06v-.8l1.23-.32h.85Z" /><path d="M15.49,11.44v1.11c0,1.49-.54,2.07-2,2.07s-2-.58-2-2.07V11.44c0-1.49.54-2.07,2-2.07S15.49,10,15.49,11.44Zm-2.74.16v.8c0,.84,0,1.18.73,1.18s.73-.34.73-1.18v-.8c0-.84,0-1.18-.73-1.18S12.75,10.76,12.75,11.6Z" /><path d="M12,2a9.93,9.93,0,0,1,7.06,2.94L21,3V8H16l1.64-1.64a8,8,0,1,0,0,11.3l1.42,1.42A10,10,0,1,1,12,2Z" /></symbol><symbol id='nfl-forward-30s'><desc id='nfl-forward-30sDesc'>Forward 30s</desc><path d="M7.53,14.14l.41-.93a3.19,3.19,0,0,0,1.53.4c.69,0,1-.16,1-.58h0c0-.38-.17-.59-.76-.59H8.54v-1h1c.49,0,.65-.21.65-.52h0c0-.27-.12-.5-.7-.5a3.05,3.05,0,0,0-1.44.37l-.41-.93a3.79,3.79,0,0,1,2-.46c1.42,0,1.81.6,1.81,1.51v0a1.11,1.11,0,0,1-.64,1,1.13,1.13,0,0,1,.86,1.18h0c0,1-.73,1.54-2.19,1.54A3.84,3.84,0,0,1,7.53,14.14Z" /><path d="M16.29,11.44v1.11c0,1.49-.54,2.07-2,2.07s-2-.58-2-2.07V11.44c0-1.49.54-2.07,2-2.07S16.29,10,16.29,11.44Zm-2.74.16v.8c0,.84,0,1.18.73,1.18s.73-.34.73-1.18v-.8c0-.84,0-1.18-.73-1.18S13.55,10.76,13.55,11.6Z" /><path d="M12,2a9.93,9.93,0,0,1,7.06,2.94L21,3V8H16l1.64-1.64a8,8,0,1,0,0,11.3l1.42,1.42A10,10,0,1,1,12,2Z" /></symbol><symbol id='nfl-full-screen-off'><desc id='nfl-full-screen-offDesc'>Full Screen Off</desc><polygon points="22 14 22 16 16 16 16 20 14 20 14 14 22 14" /><polygon points="14 10 22 10 22 8 16 8 16 4 14 4 14 10" /><polygon points="10 14 2 14 2 16 8 16 8 20 10 20 10 14" /><polygon points="2 8 2 10 10 10 10 4 8 4 8 8 2 8" /></symbol><symbol id='nfl-full-screen-on'><desc id='nfl-full-screen-onDesc'>Full Screen On</desc><polygon points="10 4 10 6 4 6 4 10 2 10 2 4 10 4" /><polygon points="2 20 10 20 10 18 4 18 4 14 2 14 2 20" /><polygon points="22 4 14 4 14 6 20 6 20 10 22 10 22 4" /><polygon points="14 18 14 20 22 20 22 14 20 14 20 18 14 18" /></symbol><symbol id='nfl-gamepass'><desc id='nfl-gamepassDesc'>Gamepass</desc><path d="M13.63,7.14H9.56v9.72h2.26V13.72h1.8c2.36,0,3.29-1.19,3.29-3.28v0C16.92,8.33,16,7.14,13.63,7.14Zm-.28,4.71H11.82V9h1.53c.93,0,1.24.42,1.24,1.4S14.28,11.85,13.35,11.85Z" /><path d="M5,13.14H6.7V14.5a3.89,3.89,0,0,1-2,.47c-1.53,0-1.78-.78-1.78-1.76V10.79c0-1,.25-1.76,2-1.76a6,6,0,0,1,3,.75L8.74,7.9A6.84,6.84,0,0,0,4.93,7C1.61,7,.66,8.37.66,10.61v2.78c0,2.24,1,3.61,3.82,3.61a3.88,3.88,0,0,0,2.42-.78v.64H8.78V11.4H5Z" /><polygon points="17.59 7.14 17.59 9.68 20.24 11.85 17.59 14.15 17.59 16.86 19.01 15.63 23.34 11.85 17.59 7.14" /></symbol><symbol id='nfl-games'><desc id='nfl-gamesDesc'>Games</desc><path d="M10,15.5v-1h1.25v-2H10v-1h1.25v-2H10v-1h1.25V3.13C9.1,3.76,6,6.9,6,12s3.1,8.24,5.25,8.87V15.5Z" /><path d="M12.75,3.13V8.5H14v1H12.75v2H14v1H12.75v2H14v1H12.75v5.37C14.9,20.24,18,17.1,18,12S14.9,3.76,12.75,3.13Z" /><rect class="cls-1" x="10" y="11.5" width="1.25" height="1" /><rect class="cls-1" x="10" y="8.5" width="1.25" height="1" /><rect class="cls-1" x="12.75" y="14.5" width="1.25" height="1" /><rect class="cls-1" x="10" y="14.5" width="1.25" height="1" /><rect class="cls-1" x="12.75" y="11.5" width="1.25" height="1" /><rect class="cls-1" x="12.75" y="8.5" width="1.25" height="1" /></symbol><symbol id='nfl-insights'><desc id='nfl-insightsDesc'>Insights</desc><path d="M13,9V4H6V20H18V9Zm3,4.5H14.36l-1.32,4-2.09-5.23-.64,1.28H8v-1H9.69l1.36-2.72L13,14.55l.68-2H16Z" /><polygon points="18 8 14 4 14 8 18 8" /></symbol><symbol id='nfl-key'><desc id='nfl-keyDesc'>Key</desc><path d="M20,10H12.58a5,5,0,1,0,0,4H14l1-1,1,1h1l1-1,1,1h1l2-2ZM6.5,13.5A1.5,1.5,0,1,1,8,12,1.5,1.5,0,0,1,6.5,13.5Z" /></symbol><symbol id='nfl-leave'><desc id='nfl-leaveDesc'>Leave</desc><polygon points="14 15 14 18 6 18 6 10 10 10 10 8 4 8 4 18 4 20 6 20 16 20 16 15 14 15" /><polygon points="12 4 12 6 16.59 6 9.29 13.29 10.71 14.71 18 7.41 18 12 20 12 20 4 12 4" /></symbol><symbol id='nfl-live'><desc id='nfl-liveDesc'>Live</desc><path d="M2,6V18H22V6Zm5.75,8.5H5.25v-5h1v4h1.5Zm2.5,0h-1v-5h1Zm3.5,0h-1l-1.5-5h1l1,3.333,1-3.333h1Zm5-4h-1.5v1h1v1h-1v1h1.5v1h-2.5v-5h2.5Z" /></symbol><symbol id='nfl-menu-combine'><desc id='nfl-menu-combineDesc'>Combine</desc><path d="M15.23,7.8A6.983,6.983,0,0,0,13,7.079V5h2V3H9V5h2V7.08a6.97,6.97,0,1,0,4.23.716ZM12,18.5A4.5,4.5,0,1,1,16.5,14,4.5,4.5,0,0,1,12,18.5ZM16.273,7.252l1.561-1.561,2.475,2.475L18.733,9.742a7.9,7.9,0,0,0-.442-.67A8.015,8.015,0,0,0,16.273,7.252ZM13,14a1,1,0,0,1-2,0,1.013,1.013,0,0,1,.039-.277h0L12,10.84l.961,2.883h0A1.013,1.013,0,0,1,13,14Z" /></symbol><symbol id='nfl-menu-draft'><desc id='nfl-menu-draftDesc'>Draft</desc><path d="M5.5,21l1-2h11l1,2ZM21,3V5H3V3ZM5.5,6,7,18H17L18.5,6Zm9.25,7.875a.882.882,0,0,1-.223.569,1.846,1.846,0,0,1-1.2.541,1.77,1.77,0,0,0-1,.416,2.685,2.685,0,0,0-.328.432,2.256,2.256,0,0,0-.328-.432,1.808,1.808,0,0,0-1-.416,1.858,1.858,0,0,1-1.2-.535.884.884,0,0,1-.223-.57V9.331h.188a1.583,1.583,0,0,0,1.272.251,1.823,1.823,0,0,0,1.3-1.082,1.823,1.823,0,0,0,1.3,1.082,1.562,1.562,0,0,0,1.266-.251h.182Z" /></symbol><symbol id='nfl-menu-fantasy'><desc id='nfl-menu-fantasyDesc'>Fantasy</desc><path d="M16.909,5,17,4H7l.091,1H3a9.038,9.038,0,0,0,4.815,7.96L8,15l3,1.5V19H9L7,21H17l-2-2H13V16.5L16,15l.185-2.04A9.038,9.038,0,0,0,21,5ZM5.3,7H7.273l.309,3.4A7.01,7.01,0,0,1,5.3,7Zm11.123,3.4L16.727,7H18.7A7.01,7.01,0,0,1,16.418,10.4Z" /></symbol><symbol id='nfl-menu-games'><desc id='nfl-menu-gamesDesc'>Menu Games</desc><path d="M10,15.5v-1h1.25v-2H10v-1h1.25v-2H10v-1h1.25V3.131C9.1,3.763,6,6.9,6,12s3.1,8.237,5.25,8.869V15.5Z" /><path d="M12.75,3.131V8.5H14v1H12.75v2H14v1H12.75v2H14v1H12.75v5.369C14.9,20.237,18,17.1,18,12S14.9,3.763,12.75,3.131Z" /></symbol><symbol id='nfl-menu-network'><desc id='nfl-menu-networkDesc'>Menu Network</desc><path d="M3,4V16h8v2H6v2H18V18H13V16h8V4ZM20,15H4V5H20Z" /><rect x="5" y="6" width="14" height="8" /></symbol><symbol id='nfl-menu-news'><desc id='nfl-menu-newsDesc'>Menu News</desc><rect x="4" y="4" width="16" height="8" /><rect x="4" y="14" width="16" height="2" /><rect x="4" y="18" width="8" height="2" /></symbol><symbol id='nfl-menu-playoffs'><desc id='nfl-menu-playoffsDesc'>Menu Playoffs</desc><polygon points="22 6 22 4 17.5 4 17.5 6.25 13.5 6.25 13.5 11 10.5 11 10.5 6.25 6.5 6.25 6.5 4 2 4 2 6 4.5 6 4.5 8.5 2 8.5 2 10.5 6.5 10.5 6.5 8.25 8.5 8.25 8.5 15.75 6.5 15.75 6.5 13.5 2 13.5 2 15.5 4.5 15.5 4.5 18 2 18 2 20 6.5 20 6.5 17.75 10.5 17.75 10.5 13 13.5 13 13.5 17.75 17.5 17.75 17.5 20 22 20 22 18 19.5 18 19.5 15.5 22 15.5 22 13.5 17.5 13.5 17.5 15.75 15.5 15.75 15.5 8.25 17.5 8.25 17.5 10.5 22 10.5 22 8.5 19.5 8.5 19.5 6 22 6" /></symbol><symbol id='nfl-menu-probowl'><desc id='nfl-menu-probowlDesc'>Menu Pro Bowl</desc><path d="M6.924,8.2a7.451,7.451,0,0,0-.213.8,8.98,8.98,0,0,0,1.141,6.639c.076.114-.377.789-.377.789H6.721L6,18H18l-.721-1.579h-.754s-.453-.675-.377-.789a8.98,8.98,0,0,0,1.141-6.639,7.451,7.451,0,0,0-.213-.8A8.311,8.311,0,0,0,12,3,8.311,8.311,0,0,0,6.924,8.2ZM9.412,9.236a4.6,4.6,0,0,1,1.8-3.754A6.388,6.388,0,0,1,12,4.974a6.388,6.388,0,0,1,.786.508,4.6,4.6,0,0,1,1.8,3.754,4.471,4.471,0,0,1-2.048,4.1l-.54.323-.54-.323A4.471,4.471,0,0,1,9.412,9.236Z" /><rect x="4" y="19" width="16" height="2" /></symbol><symbol id='nfl-menu-shop'><desc id='nfl-menu-shopDesc'>Menu Shop</desc><path d="M12,6,9,4H6v6H7V20H17V10h1V4H15Zm0,1a.75.75,0,1,1-.75.75A.75.75,0,0,1,12,7Zm2,3.7a.328.328,0,0,1-.019.112l-2.023,5.106a.122.122,0,0,1-.131.084H10.656c-.056,0-.085-.037-.056-.1l2.023-5.1v-.028h-1.34a.05.05,0,0,0-.056.056v.665a.089.089,0,0,1-.094.094H10.094A.089.089,0,0,1,10,11.494V9.724a.088.088,0,0,1,.094-.094h3.812A.088.088,0,0,1,14,9.724Z" /><rect x="3" y="4" width="2" height="6" /><rect x="19" y="4" width="2" height="6" /></symbol><symbol id='nfl-menu-standings'><desc id='nfl-menu-standingsDesc'>Menu Standings</desc><rect x="10" y="5" width="10" height="2" /><rect x="10" y="11" width="10" height="2" /><rect x="10" y="17" width="10" height="2" /><rect x="4" y="4" width="4" height="4" /><rect x="4" y="10" width="4" height="4" /><rect x="4" y="16" width="4" height="4" /></symbol><symbol id='nfl-menu-stats'><desc id='nfl-menu-statsDesc'>Menu Stats</desc><rect x="15.5" y="8" width="3" height="12" /><rect x="10.5" y="4" width="3" height="16" /><rect x="5.5" y="12" width="3" height="8" /></symbol><symbol id='nfl-menu-superbowl'><desc id='nfl-menu-superbowlDesc'>Menu Super Bowl</desc><path d="M7.99,15.54l-.74,5.473a10.118,10.118,0,0,0,3.855,1.133V15.4a10.953,10.953,0,0,1-2.025.191C8.718,15.592,8.354,15.572,7.99,15.54Z" /><path d="M16.387,4.387A12.1,12.1,0,0,0,14.939,4.3a8.852,8.852,0,0,0-5.71,1.864,8.765,8.765,0,0,0-2.86,6.612A1.125,1.125,0,0,0,7.323,13.9a10.875,10.875,0,0,0,1.757.145,8.075,8.075,0,0,0,5.243-1.763,9.535,9.535,0,0,0,3.046-6.717A1.12,1.12,0,0,0,16.387,4.387Z" /><path d="M12.652,14.983v7.175a10.125,10.125,0,0,0,4.07-1.132L15.383,13.4c-.025.022-.044.047-.07.068A8.866,8.866,0,0,1,12.652,14.983Z" /></symbol><symbol id='nfl-menu-teams'><desc id='nfl-menu-teamsDesc'>Menu Teams</desc><path d="M15,10h6c0-2-2-7-9-7S3,9,3,13l2,5c2,0,3,2,5,2s2.854-1.146,3.854-2.146L16,20h5V13H13.875Zm-3.939,7.646a1.207,1.207,0,1,1,0-1.707A1.206,1.206,0,0,1,11.061,17.646ZM19,15v3H17l-3-3Z" /></symbol><symbol id='nfl-menu-tickets'><desc id='nfl-menu-ticketsDesc'>Menu Tickets</desc><path d="M14,3a2,2,0,0,1-4,0H7V21h3a2,2,0,0,1,4,0h3V3Zm-.772,9.722.759,2.336L12,13.614l-1.987,1.444.759-2.336L8.784,11.278h2.457L12,8.942l.759,2.336h2.457Z" /></symbol><symbol id='nfl-menu'><desc id='nfl-menuDesc'>Menu</desc><rect x="4" y="6" width="16" height="2" /><rect x="4" y="11" width="16" height="2" /><rect x="4" y="16" width="16" height="2" /></symbol><symbol id='nfl-more-horizontal'><desc id='nfl-more-horizontalDesc'>More Horizontal</desc><circle cx="12" cy="12" r="2" /><circle cx="19" cy="12" r="2" /><circle cx="5" cy="12" r="2" /></symbol><symbol id='nfl-more-vertical'><desc id='nfl-more-verticalDesc'>More Vertical</desc><circle cx="12" cy="12" r="2" /><circle cx="12" cy="19" r="2" /><circle cx="12" cy="5" r="2" /></symbol><symbol id='nfl-my-location'><desc id='nfl-my-locationDesc'>My Location</desc><path d="M20,11H17.91A6,6,0,0,0,13,6.09V4H11V6.09A6,6,0,0,0,6.09,11H4v2H6.09A6,6,0,0,0,11,17.91V20h2V17.91A6,6,0,0,0,17.91,13H20Zm-8,5a4,4,0,1,1,4-4A4,4,0,0,1,12,16Z" /></symbol><symbol id='nfl-network'><desc id='nfl-networkDesc'>Network</desc><path d="M3,4V16h8v2H6v2H18V18H13V16h8V4ZM20,15H4V5H20Z" /><rect x="5" y="6" width="14" height="8" /></symbol><symbol id='nfl-news'><desc id='nfl-newsDesc'>News</desc><rect x="4" y="4" width="16" height="8" /><rect x="4" y="14" width="16" height="2" /><rect x="4" y="18" width="8" height="2" /></symbol><symbol id='nfl-pause'><desc id='nfl-pauseDesc'>Pause</desc><rect x="6" y="4" width="4" height="16" /><rect x="14" y="4" width="4" height="16" /></symbol><symbol id='nfl-play'><desc id='nfl-playDesc'>play</desc><polygon points="7 4 7 20 19 12 7 4" /></symbol><symbol id='nfl-player-multi'><desc id='nfl-player-multiDesc'>Multiple Players</desc><path d="M9,8A1,1,0,0,0,8,7H5A1,1,0,0,0,4,8v4H5v4H8V9.91A1.9,1.9,0,0,1,9,8.24Z" /><path d="M19,7H16a1,1,0,0,0-1,1v.24a1.9,1.9,0,0,1,1,1.67V16h3V12h1V8A1,1,0,0,0,19,7Z" /><path d="M14,9H10a1,1,0,0,0-1,1v5h1v5h4V15h1V10A1,1,0,0,0,14,9Z" /><circle cx="12" cy="6" r="2" /><circle cx="17.5" cy="4.5" r="1.5" /><circle cx="6.5" cy="4.5" r="1.5" /></symbol><symbol id='nfl-player-single'><desc id='nfl-player-singleDesc'>Single Player</desc><path d="M14,9H10a1,1,0,0,0-1,1v5h1v5h4V15h1V10A1,1,0,0,0,14,9Z" /><circle cx="12" cy="6" r="2" /></symbol><symbol id='nfl-playlist'><desc id='nfl-playlistDesc'>Playlist</desc><path class="cls-2" d="M12,1A11,11,0,1,0,23,12,11,11,0,0,0,12,1ZM7.5,8.75l2,1-2,1Zm9,6.5h-9v-1h9Zm0-2.5h-9v-1h9Zm0-2.5h-6v-1h6Z" /><path class="cls-3" d="M16.5,9.25v1h-6v-1Zm-9,3.5h9v-1h-9Zm0,2.5h9v-1h-9Zm0-6.5v2l2-1Z" /></symbol><symbol id='nfl-playoffs'><desc id='nfl-playoffsDesc'>Playoffs</desc><path d="M12.95,8.79A1.33,1.33,0,0,1,12,8a1.33,1.33,0,0,1-.94.79,1.15,1.15,0,0,1-.93-.18H10v3.31a.64.64,0,0,0,.16.41,1.35,1.35,0,0,0,.87.39,1.31,1.31,0,0,1,.73.3,1.65,1.65,0,0,1,.24.31,2,2,0,0,1,.24-.31,1.29,1.29,0,0,1,.73-.3,1.34,1.34,0,0,0,.87-.39.64.64,0,0,0,.16-.41V8.6h-.13A1.14,1.14,0,0,1,12.95,8.79Z" /><path d="M7.19,6.73C4.27,10.42,4.82,16.13,5.71,18h7.46a9.37,9.37,0,0,0,3.64-2.73C20.1,11.12,19,4.4,18,3.49S10.48,2.58,7.19,6.73Zm9.06,6.55c-1.52,2.63-4.63,3.68-7,2.34s-3-4.56-1.45-7.19,4.63-3.68,7-2.34S17.77,10.64,16.26,13.27Z" /><rect x="4" y="19" width="16" height="2" /></symbol><symbol id='nfl-probowl'><desc id='nfl-probowlDesc'>Pro Bowl</desc><path d="M6.92,8.19a7.52,7.52,0,0,0-.21.8,9,9,0,0,0,1.14,6.64c.08.11-.38.79-.38.79H6.72L6,18H18l-.72-1.58h-.75s-.45-.68-.38-.79A9,9,0,0,0,17.29,9a7.52,7.52,0,0,0-.21-.8A8.31,8.31,0,0,0,12,3,8.31,8.31,0,0,0,6.92,8.19Zm2.49,1a4.6,4.6,0,0,1,1.8-3.75A6.34,6.34,0,0,1,12,5a6.34,6.34,0,0,1,.79.51,4.6,4.6,0,0,1,1.8,3.75,4.47,4.47,0,0,1-2,4.1l-.54.32-.54-.32A4.47,4.47,0,0,1,9.41,9.24Z" /><rect x="4" y="19" width="16" height="2" /></symbol><symbol id='nfl-purge'><desc id='nfl-purgeDesc'>Purge</desc><polygon points="13.41 12 18.21 7.21 16.79 5.79 12 10.59 7.21 5.79 5.79 7.21 10.59 12 5.79 16.79 7.21 18.21 12 13.41 16.79 18.21 18.21 16.79 13.41 12" /></symbol><symbol id='nfl-refresh'><desc id='nfl-refreshDesc'>Refresh</desc><path d="M12,2A9.93,9.93,0,0,0,4.94,4.94L3,3V8H8L6.36,6.36a8,8,0,1,1,0,11.3L4.93,19.07A10,10,0,1,0,12,2Z" /></symbol><symbol id='nfl-remove'><desc id='nfl-removeDesc'>Remove</desc><rect x="5" y="11" width="14" height="2" /></symbol><symbol id='nfl-replay'><desc id='nfl-replayDesc'>Replay</desc><path d="M12,2A9.93,9.93,0,0,0,4.94,4.94L3,3V8H8L6.36,6.36a8,8,0,1,1,0,11.3L4.93,19.07A10,10,0,1,0,12,2Z" /><polygon points="10 8 10 16 16 12 10 8" /></symbol><symbol id='nfl-search'><desc id='nfl-searchDesc'>Search</desc><path d="M19.71,18.29l-4-4a6.52,6.52,0,1,0-1.41,1.41l4,4ZM6,10.5A4.5,4.5,0,1,1,10.5,15,4.51,4.51,0,0,1,6,10.5Z" /></symbol><symbol id='nfl-settings'><desc id='nfl-settingsDesc'>Settings</desc><path d="M18.94,12.89h0a6.33,6.33,0,0,0,0-1.78h0l1.86-1a8.94,8.94,0,0,0-1.23-3l-2,.6-.18-.23h0a7,7,0,0,0-.84-.84h0l-.23-.18h0l.6-2a8.94,8.94,0,0,0-3-1.23l-1,1.86a6.34,6.34,0,0,0-1.78,0l-1-1.86a8.94,8.94,0,0,0-3,1.23l.6,2h0l-.23.18h0a7,7,0,0,0-.84.84h0l-.18.23-2-.6a8.94,8.94,0,0,0-1.23,3l1.86,1h0a6.33,6.33,0,0,0,0,1.78h0l-1.86,1a8.94,8.94,0,0,0,1.23,3l2-.6.18.23h0a7,7,0,0,0,.84.84h0l.23.18h0l-.6,2a8.94,8.94,0,0,0,3,1.23l1-1.86a6.34,6.34,0,0,0,1.78,0l1,1.86a8.94,8.94,0,0,0,3-1.23l-.6-2h0l.23-.18h0a7.06,7.06,0,0,0,.84-.84h0l.18-.23,2,.6a8.94,8.94,0,0,0,1.23-3l-1.86-1ZM12,14a2,2,0,1,1,2-2A2,2,0,0,1,12,14Z" /></symbol><symbol id='nfl-share-android'><desc id='nfl-share-androidDesc'>Share Android</desc><path d="M17,15a3,3,0,0,0-2.08.84l-6-3a2.69,2.69,0,0,0,0-1.64l6-3A3,3,0,1,0,14,6a3,3,0,0,0,0,.36L7.7,9.53a3,3,0,1,0,0,4.94L14,17.64A3,3,0,0,0,14,18a3,3,0,1,0,3-3Z" /></symbol><symbol id='nfl-share-copy-url'><desc id='nfl-share-copy-urlDesc'>Share Copy URL</desc><path d="M8,4V16H20V4ZM18,14H10V6h8Z" /><polygon points="6 8 4 8 4 18 4 20 6 20 16 20 16 18 6 18 6 8" /></symbol><symbol id='nfl-share-email'><desc id='nfl-share-emailDesc'>Share Email</desc><polygon points="19 9 19 7 5 7 5 9 12 13 19 9" /><polygon points="5 17 19 17 19 10 12 14 5 10 5 17" /></symbol><symbol id='nfl-share-facebook'><desc id='nfl-share-facebookDesc'>Share Facebook</desc><path class="cls-2" d="M14.86,12l.32-2.33H12.72V8.54c0-.72.2-1.21,1.23-1.21h1.32V5.1A17.59,17.59,0,0,0,13.35,5a3,3,0,0,0-3.2,3.29V9.67H8V12h2.15v7h2.57V12h2.14" /></symbol><symbol id='nfl-share-instagram'><desc id='nfl-share-instagramDesc'>Share Instagram</desc><path d="M12,7.77a4.24,4.24,0,0,1,3.46,1.79H19V7.19A2.19,2.19,0,0,0,16.84,5H7.19A2.19,2.19,0,0,0,5,7.19V9.57H8.55A4.24,4.24,0,0,1,12,7.77ZM16,6.53a.39.39,0,0,1,.39-.39h1.17a.39.39,0,0,1,.39.39V7.71a.39.39,0,0,1-.39.39H16.34A.39.39,0,0,1,16,7.71Z" /><path d="M16.26,12a4.24,4.24,0,1,1-8.4-.82H5v5.64A2.19,2.19,0,0,0,7.19,19h9.65A2.19,2.19,0,0,0,19,16.84V11.2H16.18A4.35,4.35,0,0,1,16.26,12Z" /><circle cx="12.02" cy="12.01" r="2.61" /></symbol><symbol id='nfl-share-ios'><desc id='nfl-share-iosDesc'>Share iOS</desc><polygon points="11 14 13 14 13 5 16 5 12 1 8 5 11 5 11 14" /><polygon points="15 8 15 10 20 10 20 19 4 19 4 10 9 10 9 8 2 8 2 21 22 21 22 8 15 8" /></symbol><symbol id='nfl-share-snapchat'><desc id='nfl-share-snapchatDesc'>Share Snapchat</desc><path d="M12.16,4.38h-.32A4.31,4.31,0,0,0,7.83,7a8.11,8.11,0,0,0-.2,3.28l0,.39a.65.65,0,0,1-.32.07,1.81,1.81,0,0,1-.75-.2.6.6,0,0,0-.26-.05.8.8,0,0,0-.79.54c0,.23.06.57.81.87l.24.08c.31.1.78.25.91.55a.72.72,0,0,1-.08.59v0a4.77,4.77,0,0,1-3.24,2.72.34.34,0,0,0-.28.35.45.45,0,0,0,0,.15c.17.39.86.67,2.13.87a1.58,1.58,0,0,1,.11.38c0,.12.05.25.09.38a.38.38,0,0,0,.39.29A2,2,0,0,0,7,18.2a4.28,4.28,0,0,1,.86-.1,3.81,3.81,0,0,1,.62.05,2.94,2.94,0,0,1,1.16.6,3.66,3.66,0,0,0,2.23.87h.2a3.66,3.66,0,0,0,2.23-.87,2.94,2.94,0,0,1,1.16-.6,3.8,3.8,0,0,1,.62-.05,4.31,4.31,0,0,1,.86.09,2,2,0,0,0,.37,0h0a.37.37,0,0,0,.37-.28c0-.13.07-.25.09-.38a1.57,1.57,0,0,1,.11-.38c1.27-.2,2-.48,2.13-.87a.45.45,0,0,0,0-.15.34.34,0,0,0-.28-.35,4.77,4.77,0,0,1-3.24-2.72v0a.72.72,0,0,1-.08-.59c.13-.3.6-.45.91-.55l.24-.08c.55-.22.83-.48.82-.79a.66.66,0,0,0-.49-.57h0a.9.9,0,0,0-.34-.06.75.75,0,0,0-.31.06,1.89,1.89,0,0,1-.7.2.62.62,0,0,1-.27-.07l0-.35v0A8.11,8.11,0,0,0,16.17,7a4.32,4.32,0,0,0-4-2.59Z" /></symbol><symbol id='nfl-share-twitter'><desc id='nfl-share-twitterDesc'>Share Twitter</desc><path class="cls-2" d="M18.92,7.71a5.66,5.66,0,0,1-1.63.45,2.84,2.84,0,0,0,1.25-1.57,5.68,5.68,0,0,1-1.8.69,2.84,2.84,0,0,0-4.91,1.94,2.87,2.87,0,0,0,.07.65A8.06,8.06,0,0,1,6,6.89a2.84,2.84,0,0,0,.88,3.79,2.83,2.83,0,0,1-1.29-.36v0a2.84,2.84,0,0,0,2.28,2.78,2.82,2.82,0,0,1-1.28,0,2.84,2.84,0,0,0,2.65,2,5.7,5.7,0,0,1-3.53,1.22,5.84,5.84,0,0,1-.68,0A8.08,8.08,0,0,0,17.51,9.54c0-.12,0-.25,0-.37a5.78,5.78,0,0,0,1.42-1.47" /></symbol><symbol id='nfl-skip-next'><desc id='nfl-skip-nextDesc'>Skip Next</desc><polygon points="14 4 14 10 5 4 5 20 14 14 14 20 18 20 18 4 14 4" /></symbol><symbol id='nfl-skip-previous'><desc id='nfl-skip-previousDesc'>Skip Previous</desc><polygon points="19 4 10 10 10 4 6 4 6 20 10 20 10 14 19 20 19 4" /></symbol><symbol id='nfl-standings'><desc id='nfl-standingsDesc'>Standings</desc><rect x="10" y="5" width="10" height="2" /><rect x="10" y="11" width="10" height="2" /><rect x="10" y="17" width="10" height="2" /><rect x="4" y="4" width="4" height="4" /><rect x="4" y="10" width="4" height="4" /><rect x="4" y="16" width="4" height="4" /></symbol><symbol id='nfl-star'><desc id='nfl-starDesc'>Star</desc><polygon points="12 3 14.11 9.49 20.94 9.49 15.41 13.51 17.52 20 12 15.99 6.48 20 8.59 13.51 3.06 9.49 9.89 9.49 12 3" /></symbol><symbol id='nfl-stats'><desc id='nfl-statsDesc'>Stats</desc><rect x="15.5" y="8" width="3" height="12" /><rect x="10.5" y="4" width="3" height="16" /><rect x="5.5" y="12" width="3" height="8" /></symbol><symbol id='nfl-swap'><desc id='nfl-swapDesc'>Swap</desc><polygon points="19 8 7 8 7 6 4 9 7 12 7 10 19 10 19 8" /><polygon points="20 15 17 12 17 14 5 14 5 16 17 16 17 18 20 15" /></symbol><symbol id='nfl-teams'><desc id='nfl-teamsDesc'>Teams</desc><path d="M15,10h6c0-2-2-7-9-7S3,9,3,13l2,5c2,0,3,2,5,2s2.85-1.15,3.85-2.15L16,20h5V13H13.88Zm-3.94,7.65a1.21,1.21,0,1,1,0-1.71A1.21,1.21,0,0,1,11.06,17.65ZM19,15v3H17l-3-3Z" /></symbol><symbol id='nfl-tickets'><desc id='nfl-ticketsDesc'>Tickets</desc><path d="M14,3a2,2,0,0,1-4,0H7V21h3a2,2,0,0,1,4,0h3V3Zm-.77,9.72L14,15.06l-2-1.44-2,1.44.76-2.34-2-1.44h2.46L12,8.94l.76,2.34h2.46Z" /></symbol><symbol id='nfl-video'><desc id='nfl-videoDesc'>Video</desc><path class="cls-1" d="M12,1A11,11,0,1,0,23,12,11,11,0,0,0,12,1ZM10,16V8l6,4Z" /><path class="cls-2" d="M12,2A10,10,0,1,1,2,12,10,10,0,0,1,12,2m0-1A11,11,0,1,0,23,12,11,11,0,0,0,12,1Z" /><polygon class="cls-2" points="10 16 16 12 10 8 10 16" /><rect class="cls-3" width="24" height="24" /></symbol><symbol id='nfl-visibility-off'><desc id='nfl-visibility-offDesc'>Visibility Off</desc><path d="M8.5,12A3.5,3.5,0,0,1,12,8.5a3.46,3.46,0,0,1,1.6.4l1.34-1.34A8.17,8.17,0,0,0,12,7a9.53,9.53,0,0,0-8,5,9.7,9.7,0,0,0,3.1,3.4l1.8-1.8A3.46,3.46,0,0,1,8.5,12Z" /><path d="M12,10a2,2,0,0,0-1.95,2.45l2.39-2.39A2,2,0,0,0,12,10Z" /><path d="M16.86,8.57,20,5.47l-.73-.73L4.68,19.29h0l.73.73L9,16.42A8.17,8.17,0,0,0,12,17a9.53,9.53,0,0,0,8-5A9.71,9.71,0,0,0,16.86,8.57ZM12,15.5a3.46,3.46,0,0,1-1.64-.43l1.14-1.14a2,2,0,0,0,2.43-2.43l1.14-1.14A3.46,3.46,0,0,1,15.5,12,3.5,3.5,0,0,1,12,15.5Z" /></symbol><symbol id='nfl-visibility-on'><desc id='nfl-visibility-onDesc'>Visibility On</desc><path d="M12,7a9.53,9.53,0,0,0-8,5,9.53,9.53,0,0,0,8,5,9.53,9.53,0,0,0,8-5A9.53,9.53,0,0,0,12,7Zm0,8.5A3.5,3.5,0,1,1,15.5,12,3.5,3.5,0,0,1,12,15.5Z" /><circle cx="12" cy="12" r="2" /></symbol><symbol id='nfl-volume-hi'><desc id='nfl-volume-hiDesc'>Volume Hi</desc><path d="M17.55,3.68,16.44,5.35a8,8,0,0,1,0,13.31l1.11,1.67a10,10,0,0,0,0-16.64Z" /><path d="M15.33,7,14.22,8.67a4,4,0,0,1,0,6.65L15.33,17a6,6,0,0,0,0-10Z" /><polygon points="3 8 3 16 6 16 12 21 12 3 6 8 3 8" /></symbol><symbol id='nfl-volume-low'><desc id='nfl-volume-lowDesc'>Volume Low</desc><polygon points="12 3 6 8 3 8 3 16 6 16 12 21 12 3" /></symbol><symbol id='nfl-volume-medium'><desc id='nfl-volume-mediumDesc'>Volume Medium</desc><path d="M18,12a6,6,0,0,0-2.67-5L14.22,8.67a4,4,0,0,1,0,6.65L15.33,17A6,6,0,0,0,18,12Z" /><polygon points="12 3 6 8 3 8 3 16 6 16 12 21 12 3" /></symbol><symbol id='nfl-volume-mute'><desc id='nfl-volume-muteDesc'>Volume Mute</desc><polygon points="12 3 6 8 3 8 3 16 6 16 12 21 12 3" /><polygon points="18.41 12 20.19 10.22 18.78 8.81 17 10.59 15.21 8.79 13.79 10.21 15.59 12 13.79 13.79 15.21 15.21 17 13.41 18.81 15.22 20.22 13.81 18.41 12" /></symbol><symbol id='nfl-warning'><desc id='nfl-warningDesc'>Warning</desc><path d="M12,2,2,20H22Zm1,15H11V15h2Zm-2-4V8h2v5Z" /></symbol><symbol id='nfl-website'><desc id='nfl-websiteDesc'>Website</desc><path class="cls-2" d="M8.65,16.3A16.87,16.87,0,0,1,11.6,16v4C10.38,19.7,9.3,18.29,8.65,16.3Zm1,3.34a8.26,8.26,0,0,1-1.76-3.17,8.51,8.51,0,0,0-1.94.73A8,8,0,0,0,9.62,19.64Zm4.76,0a8,8,0,0,0,3.7-2.44,8.51,8.51,0,0,0-1.94-.73A8.26,8.26,0,0,1,14.38,19.64ZM11.6,15.19V12.38H8a14.8,14.8,0,0,0,.42,3.15A18.23,18.23,0,0,1,11.6,15.19Zm0-3.61V8.76a18.27,18.27,0,0,1-3.18-.33A14.71,14.71,0,0,0,8,11.58Zm-2-7.26a8,8,0,0,0-3.7,2.44,9,9,0,0,0,1.94.73A8.26,8.26,0,0,1,9.62,4.32Zm2-.32c-1.22.26-2.3,1.67-2.95,3.66A16.87,16.87,0,0,0,11.6,8Zm.8,16c1.22-.26,2.3-1.67,2.95-3.66A16.87,16.87,0,0,0,12.4,16Zm0-11.2v2.82H16a14.71,14.71,0,0,0-.42-3.15A18.27,18.27,0,0,1,12.4,8.76ZM7.64,15.7a15,15,0,0,1-.44-3.32H4a8,8,0,0,0,1.42,4.18A9.44,9.44,0,0,1,7.64,15.7Zm4.76-3.32v2.81a18.23,18.23,0,0,1,3.18.34A14.8,14.8,0,0,0,16,12.38Zm-5.2-.8a15.06,15.06,0,0,1,.44-3.33A9,9,0,0,1,5.42,7.4,8,8,0,0,0,4,11.58Zm9.6.8a15,15,0,0,1-.44,3.32,9.44,9.44,0,0,1,2.22.86A8,8,0,0,0,20,12.38Zm-.44-4.13a15.06,15.06,0,0,1,.44,3.33H20A8,8,0,0,0,18.58,7.4,9,9,0,0,1,16.36,8.25Zm-2-3.93a8.26,8.26,0,0,1,1.76,3.17,9,9,0,0,0,1.94-.73A8,8,0,0,0,14.38,4.32ZM12.4,4V8a16.87,16.87,0,0,0,2.95-.3C14.7,5.67,13.62,4.26,12.4,4Z" /><g id="Bell">
  <rect id="Bounds" class="cls-1" width="24" height="24" />
</g><rect id="Bounds" class="cls-1" width="24" height="24" /></symbol><symbol id='nfl-caret-down'><desc id='nfl-caret-downDesc'>Caret down</desc><polygon points="6 9 18 9 12 15"></polygon></symbol><symbol id='nfl-caret-up'><desc id='nfl-caret-upDesc'>Caret up</desc><polygon transform="translate(12.000000, 12.000000) scale(1, -1) translate(-12.000000, -12.000000)" points="6 9 18 9 12 15"></polygon></symbol><symbol id='nfl-at'><desc id='nfl-atDesc'>At</desc><path d="M10.188,21h0C5.814,21,3.476,18.808,4.1,15.686L5.572,8.314C6.2,5.192,9.4,3,13.8,3s6.72,2.192,6.1,5.314l-1.547,7.722a1.686,1.686,0,0,1-1.8,1.461H15.139c-1.265,0-1.812-.326-2.035-.918A4.77,4.77,0,0,1,9.96,17.661c-3.048,0-3.477-2.261-3.137-3.922l.7-3.486a4.67,4.67,0,0,1,4.706-3.938A3.372,3.372,0,0,1,15,7.439l.62-.96h2.257l-1.879,9.358c-.058.315.051.461.368.461h.072A.536.536,0,0,0,17,15.844l1.486-7.43C18.971,6,17.246,4.237,13.566,4.237c-3.55,0-6.108,1.657-6.613,4.177L5.516,15.585C5.029,18,6.77,19.763,10.444,19.763a7.152,7.152,0,0,0,4.682-1.516l.7.918A8.656,8.656,0,0,1,10.188,21ZM11.2,15.406a2.9,2.9,0,0,0,1.925-.738L14.2,9.324a1.853,1.853,0,0,0-1.624-.754,1.821,1.821,0,0,0-1.832,1.717l-.682,3.4c-.231,1.192.22,1.718,1.143,1.718Z" /></symbol></svg>


<script src="https://www.49ers.com/compiledassets/js/vendor/requirejs/require-2.3.5.min.js?_t=bebd45d1f406bbe61424136b03e50895" type="text/javascript" async defer data-main="https://www.49ers.com/compiledassets/js/main.js?_t=4fd7fa52fafaa8710075c619292ea569"></script><script src="https://www.49ers.com/compiledassets/js/vendor/picturefill/picturefill.min.js?_t=c927ae7f55dda5652225c00240e24232" type="text/javascript" async defer></script>
</div>

  <div id="fb-root"></div>
  <script type="text/javascript">
    window._taboola = window._taboola || [];
    _taboola.push({ flush: true });
  </script>
    <!-- Interstitial 1x4/3x4 SLOT  -->
    <div id="interstitial" style="text-align: center;"></div>

  
  

<script type="text/javascript">
  var googletag = googletag || {};
  googletag.cmd = googletag.cmd || [];

  googletag.cmd.push(function() {
    if (document.getElementById('adv_league')) {
      gptSlots.push(
        googletag
        .defineSlot(leagueAdvId, gptconfig.adSizes, 'adv_league')
        .setTargeting('slot', 'top')
        .setTargeting('club', 'sf')
        .setTargeting('s1', adUnitName === 'home' ? 'home' : undefined)
        .defineSizeMapping(gptconfig.responsiveMappings['leaderboardleague'])
        .addService(googletag.pubads())
      );
    }

    if (document.getElementById('adv_club_premium')) {
      gptSlots.push(googletag.defineSlot(adUnit, gptconfig.adSizes, 'adv_club_premium').setTargeting('slot', 'top')
        .setTargeting('club', 'sf')
        .setTargeting('s1', adUnitName === 'home' ? 'home' : undefined)
        .defineSizeMapping(gptconfig.responsiveMappings['club-premium']).addService(googletag.pubads()));
    }

    if (document.getElementById('adv_club')) {
      gptSlots.push(googletag.defineSlot(adUnit, gptconfig.adSizes, 'adv_club').setTargeting('slot', 'bottom')
        .defineSizeMapping(gptconfig.responsiveMappings['leaderboard']).addService(googletag.pubads()));
    }

    if (document.getElementById('adv_inpage_1')) {
      gptSlots.push(googletag.defineSlot(adUnit, gptconfig.adSizes, 'adv_inpage_1').setTargeting('slot', 'inpage_1')
        .defineSizeMapping(gptconfig.responsiveMappings['leaderboard']).addService(googletag.pubads()));
    }
    if (document.getElementById('adv_inpage_2')) {
      gptSlots.push(googletag.defineSlot(adUnit, gptconfig.adSizes, 'adv_inpage_2').setTargeting('slot', 'inpage_2')
        .defineSizeMapping(gptconfig.responsiveMappings['leaderboard']).addService(googletag.pubads()));
    }
    if (document.getElementById('adv_inpage_3')) {
      gptSlots.push(googletag.defineSlot(adUnit, gptconfig.adSizes, 'adv_inpage_3').setTargeting('slot', 'inpage_3')
        .defineSizeMapping(gptconfig.responsiveMappings['leaderboard']).addService(googletag.pubads()));
    }

    if (document.getElementById('interstitial')) {
      gptSlots.push(googletag.defineSlot(adUnit, gptconfig.adSizes, 'interstitial').setTargeting('slot', 'interstitial')
        .defineSizeMapping(gptconfig.responsiveMappings['interstitials']).addService(googletag.pubads()));
    }

    var sectionsLength;
    if (adUnitName === 'home') {
      googletag.pubads().setTargeting('s1', 'home');
    } else if (sectionsLength = advPageInfo.Sections.length) {
      for (var i = 0; i < sectionsLength; i++) {
        googletag.pubads().setTargeting('s' + (i + 1), advPageInfo.Sections[i]);
      }
    }

    if (testParameter) {
      googletag.pubads().setTargeting('test', testParameter);
    }

    var disableOOP = false;
    if (!disableOOP) {
      googletag.defineOutOfPageSlot(adUnit, 'oop').setTargeting('slot', 'oop').addService(googletag.pubads());
    }

    var modulePartnerCampaignID = '';
    if (modulePartnerCampaignID) {
      googletag.pubads().setTargeting('partner', modulePartnerCampaignID);
    }

    var pagePartnerCampaignID = window.partnerCampaignID;
    if (pagePartnerCampaignID && pagePartnerCampaignID.length) {
      googletag.pubads().setTargeting('partner', pagePartnerCampaignID);
    }

    var seriesName = '';
    if (seriesName) {
      googletag.pubads().setTargeting('s3', seriesName);
    }

    googletag.pubads().enableAsyncRendering();
    googletag.pubads().enableSingleRequest();
    googletag.pubads().collapseEmptyDivs();
    googletag.pubads().disableInitialLoad();
    googletag.enableServices();

    if (googletag && googletag.pubadsReady) {
      googletag.pubads().refresh();
    } else {
      window.requestAnimationFrame(awaitPub);
    }
  });

  function awaitPub() {
    var googletagFooter = googletag || {};
    googletagFooter.cmd = googletag.cmd || [];

    if (googletagFooter && googletagFooter.pubadsReady) {
      googletagFooter.pubads().refresh();
      if (window.animationFrameIdAD) {
        window.cancelAnimationFrame(animationFrameIdAD);
      }
    } else {
      window.animationFrameIdAD = window.requestAnimationFrame(awaitPub);
    }
  }
</script>

  

<div class="nfl-c-cookie-consent" data-require="modules/cookieConsent" data-cookiepolicyurl="https://www.49ers.com/about-us/privacy-policy" data-cookiedomain=".49ers.com"></div>

  <div class="nfl-c-unsupported-browsers nfl-c-unsupported-browsers--hide" data-unsupported-browsers="parent" aria-hidden="true" data-require="modules/unsupportedBrowsers">
  <div role="dialog" aria-described-by="unsupportedBrowsers:desc" class="cc-window d3-o-cookie cc-banner cc-type-info cc-theme-block cc-bottom ">
    <span id="unsupportedBrowsers:desc" class="d3-o-cookie-message">The browser you are using is no longer supported on this site. It is highly recommended that you use the latest versions of a supported browser in order to receive an optimal viewing experience. The following browsers are supported: Chrome, Edge (v80 and later), Firefox and Safari.</span>
    <div class="cc-compliance">
      <button data-unsupported-browsers="button" aria-label="dismiss cookie message" tabindex="0" class="cc-btn cc-dismiss d3-o-cookie-btn d3-o-cookie-btn--dismiss">Got it!</button>
    </div>
  </div>
</div>
</body>
</html>
```